In [1]:
import contextlib
import functools
import gc
import inspect
import io
import json
import logging
import lzma
import math
import os
import pickle
import psutil
import random
import re
import sys
import time
import warnings
import zipfile

import joblib
import lightgbm as lgm
import matplotlib.pyplot as plt
import os, gc, warnings, typing as tp
import mplfinance as mpf
import numpy as np
import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytorch_lightning as pl
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import seaborn as sns
import shap
import statsmodels.api as sm
import torch
import torch.nn as nn
import torch.nn.functional as F

from catboost import CatBoostRegressor
from collections import defaultdict, deque
from collections.abc import Mapping, Sequence
from typing import Optional, Literal
from contextlib import redirect_stderr, redirect_stdout
from io import StringIO
from joblib import Parallel, delayed
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm import LGBMRanker
from numba import jit, njit
from optuna.trial import FrozenTrial
from pandas.tseries.frequencies import to_offset
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data.encoders import EncoderNormalizer
from pytorch_forecasting.metrics import Metric, QuantileLoss
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.progress import RichProgressBar, TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
from scipy import stats
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter
from scipy.stats import norm, pearsonr
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score, get_scorer, ndcg_score
from sklearn.model_selection import BaseCrossValidator, GroupKFold, KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, PowerTransformer, QuantileTransformer, RobustScaler, StandardScaler
from torchmetrics import MeanSquaredError
from torchmetrics.functional import mean_squared_error as torchmetrics_mse
from tqdm.auto import tqdm as tqdm_class
from tqdm.notebook import tqdm
from typing import Any, Dict, Iterable, Iterator, List, Literal, Mapping, Optional, Tuple
from xgboost import XGBRegressor
os.environ['OMP_NUM_THREADS'] = '3'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['OPENBLAS_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'
#torch.set_num_threads(8)
#torch.set_num_interop_threads(8)

warnings.simplefilter("ignore", FutureWarning)

def to_dense(X):
    """Преобразует разреженную матрицу в плотную."""
    if issparse(X):
        return X.toarray()
    return X
    
class ToDenseTransformer(BaseEstimator, TransformerMixin):
    """Преобразует разреженную матрицу в плотную numpy-массив."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        if issparse(X):
            return X.toarray()
        return X
    def get_feature_names_out(self, input_features: tp.Sequence[str] | None = None):
        return np.asarray(input_features) if input_features is not None else np.array([])


PARALLEL_FILES = 1                         # ← меняйте при желании
N_JOBS  = max(os.cpu_count() // PARALLEL_FILES-1, 1)
warnings.filterwarnings("ignore")

# ──────────────────────────────────────────────────────────────
# 1.  RareCategoryGrouper – вместо FunctionTransformer("rare")
# ──────────────────────────────────────────────────────────────
class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    """
    Объединяет редкие категории (freq < threshold) в '__RARE__'.
    Приводит все категории к str, чтобы не было смешения типов.
    """
    def __init__(self, threshold: float = .01):
        self.threshold = threshold
        self._levels_: list[pd.Index] = []

    def fit(self, X: pd.DataFrame, y=None):
        # Обязательно приводим к строкам
        X_str = X.astype(str)
        self._levels_ = [
            X_str[col].value_counts(normalize=True)[lambda s: s >= self.threshold].index
            for col in X_str.columns
        ]
        return self

    def transform(self, X: pd.DataFrame):
        X_new = X.astype(str).copy()          # ← всё превращаем в str
        for col, keep in zip(X_new.columns, self._levels_):
            X_new[col] = np.where(
                X_new[col].isin(keep), X_new[col], "__RARE__"
            )
        return X_new

    def get_feature_names_out(self, input_features=None):
        return np.asarray(input_features) if input_features is not None else np.array([])

# ──────────────────────────────────────────────────────────────
# 2.  FrequencyEncoder с методом get_feature_names_out
# ──────────────────────────────────────────────────────────────
class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.maps_ = [X[c].value_counts(normalize=True).to_dict() for c in X.columns]
        self.feature_names_in_ = list(X.columns)
        return self
    def transform(self, X):
        cols = [X[c].map(m).fillna(0.).to_numpy(float) for c, m in zip(X.columns, self.maps_)]
        return np.vstack(cols).T
    def get_feature_names_out(self, input_features=None):
        return np.array(self.feature_names_in_)

# ──────────────────────────────────────────────────────────────
# 3.  Конструктор препроцессора
# ──────────────────────────────────────────────────────────────

to_dense_tr = FunctionTransformer(to_dense, feature_names_out="one-to-one")

# -------------------------------------------------------------
def build_preprocessor(
    X: pd.DataFrame,
    rare_thr: float = .01,
    high_card_thr: int = 20
) -> ColumnTransformer:

    num_cols = [c for c in make_column_selector(dtype_include=np.number)(X) if c != "batch"]
    cat_cols = [c for c in make_column_selector(dtype_include=object)(X)  if c != "batch"]

    low_card, high_card = [], []
    for c in cat_cols:
        (high_card if X[c].nunique() >= high_card_thr else low_card).append(c)

    num_pipe = Pipeline([
        ("imp", SimpleImputer(strategy="median")),
        ("sc",  StandardScaler())
    ])

    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)

    low_cat_pipe = Pipeline([
        ("to_str", FunctionTransformer(lambda df: df.astype(str), feature_names_out="one-to-one")),
        #("rare",  RareCategoryGrouper(threshold=rare_thr)),
        ("imp",   SimpleImputer(strategy="most_frequent")),
        ("ohe",   ohe)
    ])

    high_cat_pipe = Pipeline([
        ("to_str", FunctionTransformer(lambda df: df.astype(str), feature_names_out="one-to-one")),
        #("rare",  RareCategoryGrouper(threshold=rare_thr)),
        ("imp",   SimpleImputer(strategy="most_frequent")),
        ("freq",  FrequencyEncoder())          # → dense вектор размера 1
    ])

    return ColumnTransformer(
        [("num",   num_pipe,     num_cols),
         ("lowc",  low_cat_pipe, low_card),
         ("highc", high_cat_pipe, high_card)],
        remainder="drop",
        n_jobs=1,
        verbose_feature_names_out=False,
    )

def _to_dense(x):
    if issparse(x):
        return x.toarray()
    return np.asarray(x)

# Modernized feature_ranking for multioutput (optimized for memory)
def feature_ranking_multioutput(
    df: pd.DataFrame,
    target_cols: Sequence[str],
    group_col: str = "batch",
    top_k: int = 50,
    *,
    top_pi_feats: Optional[int] = 300,  # Reduced default to save memory/compute
    val_frac: float = .10,  # Reduced to save memory in SHAP
    max_shap_samples: int = 1000,  # Limit SHAP computation to this many samples
    parallel_fit: bool = False,
    n_estimators: int = 100  # Reduced for faster fitting and less memory
) -> pd.DataFrame:
    """
    Ранжирование признаков для multi-output регрессии.
    Возвращает DataFrame с важностью и подробными Δ-метриками.
    Оптимизировано для снижения потребления памяти.
    """

    # --- отделяем таргеты
    df = df.copy().dropna(subset=target_cols)
    y = df[target_cols].reset_index(drop=True).to_numpy(float)   # (n, K)
    df.drop(columns=target_cols, inplace=True)
    groups = df[group_col] if group_col in df.columns else None

    # --- препроцессинг (keep sparse if possible, convert to dense only for folds)
    pre = build_preprocessor(df)
    Xall = pre.fit_transform(df)
    feat_names = pre.get_feature_names_out()

    # --- модели (only necessary ones: LGBM for perm/SHAP, Ridge for coef)
    MODELS: dict[str, object] = {
        "LGBM": MultiOutputRegressor(
                    LGBMRegressor(
                        n_estimators=n_estimators, learning_rate=.05,
                        subsample=.8, colsample_bytree=.8,
                        random_state=RANDOM_STATE, n_jobs=1,
                        verbosity=-1
                    ), n_jobs=1),
        "Ridge": Ridge(alpha=1.0)
    }

    # --- контейнеры
    pi_mix, pi_mse, pi_r2, pi_corr = [], [], [], []
    shap_all, coef_all = [], []
    gkf = GroupKFold(n_splits=3)

    # функция корреляции по каждому таргету
    def _corr_multi(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        corrs = []
        for j in range(y_true.shape[1]):
            c = np.corrcoef(y_true[:, j], y_pred[:, j])[0, 1]
            corrs.append(-1. if np.isnan(c) else c)
        return np.mean(corrs)

    # --- кросс-валидация
    for fold_idx, (tr_idx, val_idx) in enumerate(gkf.split(Xall, y, groups=groups)):
        # Convert to dense per fold to manage memory
        Xtr = _to_dense(Xall[tr_idx])
        Xval = _to_dense(Xall[val_idx])
        ytr, yval = y[tr_idx], y[val_idx]

        # ---------- ФИТ МОДЕЛЕЙ --------------
        if parallel_fit:
            trained = dict(joblib.Parallel(
                              n_jobs=min(3, len(MODELS)), backend="threading")(
                joblib.delayed(lambda nm, mdl: (nm, clone(mdl).fit(Xtr, ytr)))(
                    nm, mdl) for nm, mdl in MODELS.items()
            ))
        else:
            trained = {nm: clone(mdl).fit(Xtr, ytr) for nm, mdl in MODELS.items()}

        # ------------- SHAP -----------------
        # усредняем |SHAP| по таргетам и объектам, limit sample size
        shap_fold = np.zeros(Xval.shape[1])
        m = min(max_shap_samples, math.ceil(len(Xval) * val_frac))
        X_shap = Xval[:m]
        for est_idx, est in enumerate(trained["LGBM"].estimators_):
            expl = shap.TreeExplainer(est, feature_perturbation="tree_path_dependent")
            sv = expl.shap_values(X_shap, check_additivity=False)  # (m, F)
            shap_fold += np.abs(sv).mean(axis=0)
            del expl, sv  # Free memory immediately
            gc.collect()
        shap_all.append(shap_fold / len(trained["LGBM"].estimators_))

        # ------------- Ridge коэффициенты ----
        coef = np.abs(trained["Ridge"].coef_)
        if coef.ndim == 2:
            coef = coef.mean(axis=0)
        coef_all.append(coef)

        # ------------- baseline метрики -------
        y_pred_base = trained["LGBM"].predict(Xval)                    # (n, K)
        base_mse = mean_squared_error(yval, y_pred_base, multioutput="raw_values").mean()
        base_r2 = r2_score(yval, y_pred_base, multioutput="raw_values").mean()
        base_corr = _corr_multi(yval, y_pred_base)
        del y_pred_base
        gc.collect()

        # ------------- permutation importance -
        variances = np.array(Xtr.var(axis=0)).ravel()
        order = np.argsort(variances)[::-1][: (top_pi_feats or len(variances))]

        fold_mix = np.zeros(Xval.shape[1])
        fold_mse = np.zeros_like(fold_mix)
        fold_r2 = np.zeros_like(fold_mix)
        fold_corr = np.zeros_like(fold_mix)

        rng = np.random.default_rng(RANDOM_STATE)
        for pi_idx, idx in enumerate(order):
            original_col = Xval[:, idx].copy()
            rng.shuffle(Xval[:, idx])  # Shuffle in place
            y_perm = trained["LGBM"].predict(Xval)

            new_mse = mean_squared_error(yval, y_perm, multioutput="raw_values").mean()
            new_r2 = r2_score(yval, y_perm, multioutput="raw_values").mean()
            new_corr = _corr_multi(yval, y_perm)

            d_mse = new_mse - base_mse
            d_r2 = base_r2 - new_r2
            d_corr = base_corr - new_corr

            fold_mse[idx] = d_mse / (abs(base_mse) + 1e-12)
            fold_r2[idx] = d_r2
            fold_corr[idx] = d_corr
            fold_mix[idx] = (fold_mse[idx] + d_r2 + d_corr) / 3

            # Restore original column
            Xval[:, idx] = original_col

            # Periodic garbage collection to manage memory
            if (pi_idx + 1) % 50 == 0:
                del y_perm
                gc.collect()

        pi_mix.append(old_mix)
        pi_mse.append(fold_mse)
        pi_r2.append(fold_r2)
        pi_corr.append(fold_corr)

        del trained, Xtr, Xval, ytr, yval
        gc.collect()

    # --- агрегируем фолды
    mean_ = lambda lst: np.stack(lst).mean(axis=0)
    mix_m, mse_m, r2_m, corr_m = map(mean_, (pi_mix, pi_mse, pi_r2, pi_corr))
    shap_m, coef_m = map(mean_, (shap_all, coef_all))

    # --- финальный комбинированный скор
    z = lambda x: (x - x.mean()) / (x.std() + 1e-9)
    final_score = np.nanmean(np.vstack([z(mix_m), z(shap_m), z(coef_m)]), axis=0)

    res = (pd.DataFrame(dict(
             feature    = feat_names,
             Δmse       = mse_m,
             Δr2        = r2_m,
             Δcorr      = corr_m,
             perm_mix   = mix_m,
             shap       = shap_m,
             ridge_coef = coef_m,
             importance = final_score))
           .sort_values("importance", ascending=False)
           .reset_index(drop=True))

    return res

def feature_ranking(
    df: pd.DataFrame,
    target_col : str = "target",
    group_col  : str = "batch",
    top_k      : int = 50,
    *,
    top_pi_feats : Optional[int] = 800,
    val_frac     : float = .30
) -> pd.DataFrame:
    """
    Важность = усреднение четырёх каналов:
        1. perm_mix  – средний ущерб трёх метрик (MSE, R², Corr)
        2. SHAP      – |SHAP|
        3. Ridge      |coef|
        4. (опционально) можно добавить любой другой канал
    Возвращает top_k фичей + подробные Δ-метрики.
    """
    df = df.copy().dropna(subset=[target_col])
    y  = df.pop(target_col).reset_index(drop=True)
    g  = df[group_col].reset_index(drop=True) if group_col in df.columns else None

    pre = build_preprocessor(df)
    Xt  = pre.fit_transform(df, y)
    f_names = pre.get_feature_names_out()

    MODELS: Dict[str, object] = {
        "LGBM": LGBMRegressor(
                    n_estimators=250, learning_rate=.05,
                    subsample=.8, colsample_bytree=.8,
                    random_state=RANDOM_STATE, n_jobs=1, verbosity=-1),
        "HGBR": HistGradientBoostingRegressor(
                    max_depth=None, learning_rate=.06, l2_regularization=1.,
                    max_iter=250, random_state=RANDOM_STATE,
                    categorical_features=None),       # ← фиксация бага
        "RF"  : RandomForestRegressor(
                    n_estimators=350, max_features="sqrt",
                    random_state=RANDOM_STATE, n_jobs=1),
        "Ridge": Ridge(alpha=1.0)
    }

    # контейнеры
    pi_mix, pi_mse, pi_r2, pi_corr = [], [], [], []
    shap_all, coef_all = [], []
    gkf = GroupKFold(n_splits=3)

    # вспом. функция корреляции
    def _corr(a, b):
        c = np.corrcoef(a, b)[0, 1]
        return -1. if np.isnan(c) else c

    # параллельный фит одного алгоритма
    def _fit(name, mdl, Xtr, ytr):
        return name, mdl.fit(Xtr, ytr)

    for tr_idx, val_idx in gkf.split(Xt, y, groups=g):
        Xtr, Xval = Xt[tr_idx], Xt[val_idx]
        ytr, yval = y.iloc[tr_idx], y.iloc[val_idx]

        trained = dict(joblib.Parallel(n_jobs=len(MODELS))(
            joblib.delayed(_fit)(nm, mdl, Xtr, ytr)
            for nm, mdl in MODELS.items()
        ))

        # ─── SHAP
        expl = shap.TreeExplainer(trained["LGBM"], feature_perturbation="tree_path_dependent")
        m = math.ceil(len(Xval) * val_frac)
        shap_vals = expl.shap_values(Xval[:m], check_additivity=False)
        shap_all.append(np.abs(shap_vals).mean(axis=0))

        # ─── Ridge
        coef_all.append(np.abs(trained["Ridge"].coef_))

        # ─── Baseline метрики
        y_pred_base = trained["LGBM"].predict(Xval)
        base_mse  = mean_squared_error(yval, y_pred_base)
        base_r2   = r2_score(yval, y_pred_base)
        base_corr = _corr(yval, y_pred_base)

        # ─── Permutation importance по top-N
        variances = np.array(Xtr.var(axis=0)).ravel()
        order = np.argsort(variances)[::-1][: (top_pi_feats or len(variances))]

        fold_mix  = np.zeros(Xval.shape[1])
        fold_mse  = np.zeros_like(fold_mix)
        fold_r2   = np.zeros_like(fold_mix)
        fold_corr = np.zeros_like(fold_mix)

        rng = np.random.default_rng(RANDOM_STATE)
        for idx in order:
            col = Xval[:, idx].copy()
            rng.shuffle(col)
            Xperm = Xval.copy()
            Xperm[:, idx] = col
            y_perm = trained["LGBM"].predict(Xperm)

            new_mse  = mean_squared_error(yval, y_perm)
            new_r2   = r2_score(yval, y_perm)
            new_corr = _corr(yval, y_perm)

            d_mse  = new_mse  - base_mse          # ↑ плохое
            d_r2   = base_r2  - new_r2            # ↑ плохое
            d_corr = base_corr - new_corr         # ↑ плохое

            fold_mse[idx]  = d_mse  / (abs(base_mse) + 1e-12)   # нормируем
            fold_r2[idx]   = d_r2
            fold_corr[idx] = d_corr
            fold_mix[idx]  = (fold_mse[idx] + d_r2 + d_corr) / 3

        pi_mix.append(fold_mix)
        pi_mse.append(fold_mse)
        pi_r2.append(fold_r2)
        pi_corr.append(fold_corr)

        del trained, expl, shap_vals
        gc.collect()

    # ─── агрегируем по фолдам
    mean_ = lambda lst: np.stack(lst).mean(axis=0)
    mix_m, mse_m, r2_m, corr_m = map(mean_, (pi_mix, pi_mse, pi_r2, pi_corr))
    shap_m, coef_m             = map(mean_, (shap_all, coef_all))

    # ─── финальный комбинированный скор
    z = lambda x: (x - x.mean()) / (x.std() + 1e-9)
    final_score = np.nanmean(np.vstack([z(mix_m), z(shap_m), z(coef_m)]), axis=0)

    res = (pd.DataFrame(dict(
             feature=f_names,
             Δmse = mse_m,
             Δr2  = r2_m,
             Δcorr= corr_m,
             perm_mix = mix_m,
             shap = shap_m,
             ridge = coef_m,
             importance = final_score))
           .sort_values("importance", ascending=False)
           .reset_index(drop=True))

    return res

def find_best_trial_by_weighted_three_score(
    trials: List[FrozenTrial],
    pnl_score = 0.45,
    diff_score = 0.45,
    weight_score = 0.10
) -> Tuple[float, int, float, float, int, Dict]:
    """
    Ищет лучший трейл по взвешенной сумме двух метрик:
        - pnl (trial.values[0]), вес 0.65, диапазон 0..600
        - diff (trial.values[1]), вес 0.35, диапазон 0..1

    Возвращает:
        - score: float — итоговый взвешенный скор
        - trial_number: int — номер трейла
        - pnl: float — значение pnl
        - diff: float — значение diff
        - params: dict — параметры трейла
    """
    WEIGHT_PNL = pnl_score
    WEIGHT_DIFF = diff_score
    WEIGHT_SCORE = weight_score
    MAX_PNL = np.max([i.values[0] for i in [trial for trial in trials if trial.values is not None]])*2.5  # для нормализации
    MAX_SCORE = np.max([i.values[2] for i in [trial for trial in study.trials if trial.values is not None]])  # для нормализации
    MAX_DIFF = 1   # для нормализации

    best_score = float('-inf')
    best_trial_number = -1
    best_pnl = None
    best_diff = None
    best_score_n = None
    best_params = None

    for trial in trials:
        # Проверяем что трейл валидный и содержит обе метрики
        if not trial.values or len(trial.values) < 3:
            continue

        pnl = trial.values[0]
        diff = trial.values[1]
        score_n = trial.values[2]

        # Нормализуем значения
        norm_pnl = pnl / MAX_PNL if MAX_PNL else 0
        norm_diff = diff / MAX_DIFF if MAX_DIFF else 0
        norm_score = score_n / MAX_SCORE if WEIGHT_SCORE else 0

        # Взвешенная сумма
        score = WEIGHT_PNL * norm_pnl + WEIGHT_DIFF * norm_diff + WEIGHT_SCORE * norm_score

        if score > best_score:
            best_score = score
            best_trial_number = trial.number
            best_pnl = pnl
            best_diff = diff
            best_score_n = score_n
            best_params = trial.params

    if best_trial_number == -1:
        raise ValueError("Нет подходящих трейлов с двумя метриками (pnl и diff).")

    return best_score, best_trial_number, best_pnl, best_diff, best_score_n, best_params

def find_best_trial_by_weighted_score(
    trials: List[FrozenTrial]
) -> Tuple[float, int, float, float, Dict]:
    """
    Ищет лучший трейл по взвешенной сумме двух метрик:
        - pnl (trial.values[0]), вес 0.65, диапазон 0..600
        - diff (trial.values[1]), вес 0.35, диапазон 0..1

    Возвращает:
        - score: float — итоговый взвешенный скор
        - trial_number: int — номер трейла
        - pnl: float — значение pnl
        - diff: float — значение diff
        - params: dict — параметры трейла
    """
    WEIGHT_PNL = 0.60
    WEIGHT_DIFF = 0.40
    MAX_PNL = np.max([i.values for i in [trial for trial in trials if trial.values is not None]])*2.5  # для нормализации
    MAX_DIFF = 1   # для нормализации

    best_score = float('-inf')
    best_trial_number = -1
    best_pnl = None
    best_diff = None
    best_params = None

    for trial in trials:
        # Проверяем что трейл валидный и содержит обе метрики
        if not trial.values or len(trial.values) < 2:
            continue

        pnl = trial.values[0]
        diff = trial.values[1]

        # Нормализуем значения
        norm_pnl = pnl / MAX_PNL if MAX_PNL else 0
        norm_diff = diff / MAX_DIFF if MAX_DIFF else 0

        # Взвешенная сумма
        score = WEIGHT_PNL * norm_pnl + WEIGHT_DIFF * norm_diff

        if score > best_score:
            best_score = score
            best_trial_number = trial.number
            best_pnl = pnl
            best_diff = diff
            best_params = trial.params

    if best_trial_number == -1:
        raise ValueError("Нет подходящих трейлов с двумя метриками (pnl и diff).")

    return best_score, best_trial_number, best_pnl, best_diff, best_params


def to_dense(X):
    # LightGBM «любит» плотные матрицы
    return X.toarray() if hasattr(X, 'toarray') else X

def corrcoef(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]

def find_best_trial_by_weighted_score_extended(
    trials: List[FrozenTrial]
) -> Tuple[float, int, Dict, str, float, float, float, float]:
    """
    Ищет лучший трейл по взвешенной сумме метрик для двух стратегий:
        - Стратегия RSI: pnl_rsi и diff_rsi
        - Стратегия RSI+SMA/EMA: pnl_full и diff_full
    
    Возвращает:
        - best_score: float - наивысший взвешенный скор среди всех стратегий
        - best_trial_number: int - номер лучшего трейла
        - best_params: dict - параметры лучшего трейла
        - best_strategy: str - название лучшей стратегии ('RSI' или 'RSI_SMA_EMA')
        - best_pnl: float - лучшее значение PnL (из выбранной стратегии)
        - best_diff: float - лучшее значение diff (из выбранной стратегии)
        - pnl_rsi: float - значение PnL для стратегии RSI
        - diff_rsi: float - значение diff для стратегии RSI
        - pnl_full: float - значение PnL для стратегии RSI+SMA/EMA
        - diff_full: float - значение diff для стратегии RSI+SMA/EMA
    """
    # Веса для метрик
    WEIGHT_PNL = 0.80
    WEIGHT_DIFF = 0.20
    
    # Максимальные значения для нормализации (можно настроить)
    MAX_PNL = np.max([i.values for i in [trial for trial in study.trials if trial.values is not None]])  # предполагаемый максимум PnL
    MAX_DIFF = 1    # максимум для diff (уже нормализован)
    
    best_score = float('-inf')
    best_trial_number = -1
    best_params = None
    best_strategy = None
    best_pnl = None
    best_diff = None
    
    # Для хранения всех метрик (для отладки/анализа)
    full_results = []

    for trial in trials:
        # Проверяем что трейл валидный и содержит все 4 метрики
        if not trial.values or len(trial.values) < 4 and trial.values:
            continue
            
        pnl_rsi, pnl_full, diff_rsi, diff_full = trial.values
        
        # Нормализация значений (защита от деления на 0)
        norm_pnl_rsi = pnl_rsi / MAX_PNL if MAX_PNL != 0 else 0
        norm_pnl_full = pnl_full / MAX_PNL if MAX_PNL != 0 else 0
        norm_diff_rsi = diff_rsi / MAX_DIFF if MAX_DIFF != 0 else 0
        norm_diff_full = diff_full / MAX_DIFF if MAX_DIFF != 0 else 0
        
        # Вычисляем скоринг для обеих стратегий
        score_rsi = WEIGHT_PNL * norm_pnl_rsi + WEIGHT_DIFF * norm_diff_rsi
        score_full = WEIGHT_PNL * norm_pnl_full + WEIGHT_DIFF * norm_diff_full
        
        # Определяем какая стратегия лучше в этом трейле
        if score_rsi > score_full:
            current_score = score_rsi
            current_strategy = 'С трендовой линией'
            current_pnl = pnl_rsi
            current_diff = diff_rsi
        else:
            current_score = score_full
            current_strategy = 'Трендовая + недельные скользящие'
            current_pnl = pnl_full
            current_diff = diff_full
        
        # Сохраняем все метрики для анализа
        full_results.append({
            'trial_number': trial.number,
            'score_rsi': score_rsi,
            'score_full': score_full,
            'strategy': current_strategy,
            'score': current_score,
            'params': trial.params
        })
        
        # Обновляем лучший результат
        if current_score > best_score:
            best_score = current_score
            best_trial_number = trial.number
            best_params = trial.params
            best_strategy = current_strategy
            best_pnl = current_pnl
            best_diff = current_diff

    if best_trial_number == -1:
        raise ValueError("Нет подходящих трейлов с четырьмя метриками (pnl_rsi, pnl_full, diff_rsi, diff_full).")

    return (
        best_score,
        best_trial_number,
        best_params,
        best_strategy,
        best_pnl,
        best_diff,
        # Возвращаем также все метрики для лучшего трейла
        #next(t.values[0] for t in trials if t.number == best_trial_number),  # pnl_rsi
        #next(t.values[2] for t in trials if t.number == best_trial_number),  # diff_rsi
        #next(t.values[1] for t in trials if t.number == best_trial_number),  # pnl_full
        #next(t.values[3] for t in trials if t.number == best_trial_number)   # diff_full
    )

def find_best_trial_by_weighted_score_extended1(
    trials: List[FrozenTrial],
    pnl_score: float = 0.45,
    diff_score: float = 0.45,
    weight_score: float = 0.10
) -> Tuple[float, int, Dict, str, float, float, float, float, float, float]:
    """
    Ищет лучший трейл по взвешенной сумме метрик для двух стратегий:
        - Стратегия RSI: pnl_rsi, diff_rsi, score_rsi
        - Стратегия RSI+SMA/EMA: pnl_full, diff_full, score_full
    
    Параметры:
        - pnl_score: вес для метрики PnL (по умолчанию 0.45)
        - diff_score: вес для метрики diff (по умолчанию 0.45)
        - weight_score: вес для дополнительной метрики score (по умолчанию 0.10)
    
    Возвращает:
        - best_score: float - наивысший взвешенный скор среди всех стратегий
        - best_trial_number: int - номер лучшего трейла
        - best_params: dict - параметры лучшего трейла
        - best_strategy: str - название лучшей стратегии ('RSI' или 'RSI_SMA_EMA')
        - best_pnl: float - лучшее значение PnL (из выбранной стратегии)
        - best_diff: float - лучшее значение diff (из выбранной стратегии)
        - best_score_n: float - лучшее значение score (из выбранной стратегии)
        - pnl_rsi: float - значение PnL для стратегии RSI
        - diff_rsi: float - значение diff для стратегии RSI
        - pnl_full: float - значение PnL для стратегии RSI+SMA/EMA
        - diff_full: float - значение diff для стратегии RSI+SMA/EMA
        - score_rsi: float - значение score для стратегии RSI
        - score_full: float - значение score для стратегии RSI+SMA/EMA
    """
    # Проверка корректности весов
    total_weight = pnl_score + diff_score + weight_score
    if not np.isclose(total_weight, 1.0):
        raise ValueError(f"Сумма весов должна быть равна 1.0 (получено {total_weight})")
    
    # Максимальные значения для нормализации
    MAX_PNL = np.max([max(trial.values[0], trial.values[1]) for trial in trials if trial.values is not None]) * 2.5
    MAX_DIFF = 1.0    # максимум для diff (уже нормализован)
    MAX_SCORE = np.max([max(trial.values[2], trial.values[3]) for trial in trials if trial.values is not None]) if weight_score > 0 else 1.0
    
    best_score = float('-inf')
    best_trial_number = -1
    best_params = None
    best_strategy = None
    best_pnl = None
    best_diff = None
    best_score_n = None
    
    # Для хранения всех метрик (для отладки/анализа)
    full_results = []

    for trial in trials:
        # Проверяем что трейл валидный и содержит все 6 метрик
        if not trial.values or len(trial.values) < 6:
            continue
            
        pnl_rsi, pnl_full, score_rsi, score_full, diff_rsi, diff_full = trial.values
        
        # Нормализация значений (защита от деления на 0)
        norm_pnl_rsi = pnl_rsi / MAX_PNL if MAX_PNL != 0 else 0
        norm_pnl_full = pnl_full / MAX_PNL if MAX_PNL != 0 else 0
        norm_diff_rsi = diff_rsi / MAX_DIFF if MAX_DIFF != 0 else 0
        norm_diff_full = diff_full / MAX_DIFF if MAX_DIFF != 0 else 0
        norm_score_rsi = score_rsi / MAX_SCORE if MAX_SCORE != 0 and weight_score > 0 else 0
        norm_score_full = score_full / MAX_SCORE if MAX_SCORE != 0 and weight_score > 0 else 0
        
        # Вычисляем скоринг для обеих стратегий
        score_rsi_weighted = (pnl_score * norm_pnl_rsi + 
                             diff_score * norm_diff_rsi + 
                             weight_score * norm_score_rsi)
        
        score_full_weighted = (pnl_score * norm_pnl_full + 
                              diff_score * norm_diff_full + 
                              weight_score * norm_score_full)
        
        # Определяем какая стратегия лучше в этом трейле
        if score_rsi_weighted > score_full_weighted:
            current_score = score_rsi_weighted
            current_strategy = 'С трендовой линией'
            current_pnl = pnl_rsi
            current_diff = diff_rsi
            current_score_n = score_rsi
        else:
            current_score = score_full_weighted
            current_strategy = 'Трендовая + недельные скользящие'
            current_pnl = pnl_full
            current_diff = diff_full
            current_score_n = score_full
        
        # Сохраняем все метрики для анализа
        full_results.append({
            'trial_number': trial.number,
            'score_rsi': score_rsi_weighted,
            'score_full': score_full_weighted,
            'strategy': current_strategy,
            'score': current_score,
            'params': trial.params
        })
        
        # Обновляем лучший результат
        if current_score > best_score:
            best_score = current_score
            best_trial_number = trial.number
            best_params = trial.params
            best_strategy = current_strategy
            best_pnl = current_pnl
            best_diff = current_diff
            best_score_n = current_score_n

    if best_trial_number == -1:
        raise ValueError("Нет подходящих трейлов с шестью метриками (pnl_rsi, pnl_full, score_rsi, score_full, diff_rsi, diff_full).")

    # Получаем все метрики для лучшего трейла
    best_trial = next(t for t in trials if t.number == best_trial_number)
    pnl_rsi, pnl_full, score_rsi, score_full, diff_rsi, diff_full = best_trial.values

    return (
        best_score,
        best_trial_number,
        best_params,
        best_strategy,
        best_pnl,
        best_diff,
        best_score_n,
        pnl_rsi,
        diff_rsi,
        score_rsi,
        pnl_full,
        diff_full,
        score_full
    )

def plot_price_with_indicators_mplfinance(df, ticker, regime = 'P',save_path=None):
    """
    Отрисовывает график движения цены с индикаторами и сигналами покупки/продажи, используя mplfinance.

    Args:
        df (pd.DataFrame): DataFrame с данными о цене и индикаторах.
        ticker (str): Тикер акции для заголовка графика.
        save_path (str, optional): Путь для сохранения графика. Если None, то график покажется.
    """
    # Копируем DataFrame, чтобы не менять оригинал
    df = df.copy()

    # Преобразуем столбец time в datetime и делаем индексом
    df["time"] = pd.to_datetime(df["time"])
    df = df.set_index("time")

    # Создаем список дополнительных панелей для индикаторов (без объема)
    apds = []


    if "pmax" in df.columns:
        apds.append(mpf.make_addplot(df["pmax"], color="red", ylabel="PMAX", panel=0))
    if "ma" in df.columns:
        apds.append(mpf.make_addplot(df["ma"], color="blue", ylabel="VAR", panel=0))
    if "var" in df.columns:
        apds.append(mpf.make_addplot(df["var"], color="blue", ylabel="VAR", panel=0))
    if "ema" in df.columns:
        apds.append(mpf.make_addplot(df["ema"], color="purple", ylabel="EMA", panel=0))
    if "adaptive_ma" in df.columns:
        apds.append(mpf.make_addplot(df["adaptive_ma"], color="blue", ylabel="VAR", panel=0))
    if "adaptive_pmax" in df.columns:
        apds.append(mpf.make_addplot(df["adaptive_pmax"], color="purple", ylabel="EMA", panel=0))

    if "regime" in df.columns and regime == 'R':
        # Генерация цветовой карты по количеству уникальных режимов
        unique_regimes = sorted(df['regime'].unique())
        colors = plt.cm.get_cmap('tab10', len(unique_regimes))
        
        for i, reg in enumerate(unique_regimes):
            # Создаем линию на уровне 5% от минимума
            reg_line = (df["low"] * 1.05).where(df["regime"] == reg)
            apds.append(mpf.make_addplot(
                reg_line,
                type='line',
                color=colors(i),
                panel=0
            ))
    if "normalized_target" in df.columns and regime == 'P':
        # Генерация цветовой карты по количеству уникальных режимов
        apds.append(mpf.make_addplot(df["normalized_target"], color="blue", ylabel="Regime", panel=2))
    if "predicted_p" in df.columns and regime == 'P':
        # Генерация цветовой карты по количеству уникальных режимов
        apds.append(mpf.make_addplot(df["predicted_p"], color="black", ylabel="Regime", panel=2))
    if "reg_pred" in df.columns and regime == 'P':
        # Генерация цветовой карты по количеству уникальных режимов
        apds.append(mpf.make_addplot(df["reg_pred"], color="yellow", ylabel="Regime", panel=2))
    if "rank_pred" in df.columns and regime == 'P':
        # Генерация цветовой карты по количеству уникальных режимов
        apds.append(mpf.make_addplot(df["rank_pred"], color="green", ylabel="Regime", panel=2))
    if "multi_target_10" in df.columns and regime == 'P':
        # Генерация цветовой карты по количеству уникальных режимов
        apds.append(mpf.make_addplot(df["multi_target_10"], color="grey", ylabel="Regime", panel=2))
    if "predicted_p_old" in df.columns and regime == 'P':
        # Генерация цветовой карты по количеству уникальных режимов
        apds.append(mpf.make_addplot(df["predicted_p_old"], color="purple", ylabel="Regime", panel=2))

    # Подготовка сигналов для отрисовки
    buy_signals = df[df["buy_signal"]]
    sell_signals = df[df["sell_signal"]]

    # Создаем Series для сигналов, выровненные по индексу основного DataFrame
    buy_series = pd.Series(index=df.index, dtype='float64')
    buy_series[buy_signals.index] = buy_signals['open']

    sell_series = pd.Series(index=df.index, dtype='float64')
    sell_series[sell_signals.index] = sell_signals['open']

    # Добавляем сигналы покупки
    if not buy_signals.empty:
        apds.append(mpf.make_addplot(
            buy_series,
            type='scatter',
            markersize=50,
            marker='^',
            color='green',
            label='Buy Signal',
            panel=0
        ))

    # Добавляем сигналы продажи
    if not sell_signals.empty:
        apds.append(mpf.make_addplot(
            sell_series,
            type='scatter',
            markersize=50,
            marker='v',
            color='red',
            label='Sell Signal',
            panel=0
        ))

    # Подготовка vlines (единый словарь)
    vlines_dict = {}
    if not buy_signals.empty:
        vlines_dict['vlines'] = buy_signals.index.to_list()
        vlines_dict['linewidths'] = 0.5
        vlines_dict['colors'] = ['green'] * len(buy_signals)
        vlines_dict['alpha'] = 0.5

    if not sell_signals.empty:
        if 'vlines' in vlines_dict:
            vlines_dict['vlines'].extend(sell_signals.index.to_list())
            vlines_dict['colors'].extend(['red'] * len(sell_signals))
        else:
            vlines_dict['vlines'] = sell_signals.index.to_list()
            vlines_dict['linewidths'] = 0.5
            vlines_dict['colors'] = ['red'] * len(sell_signals)
            vlines_dict['alpha'] = 0.5

    # Отрисовка графика с mplfinance
    plot_kwargs = dict(
        type="candle",
        style="yahoo",
        title=f"График цены {ticker} с индикаторами",
        ylabel="Цена",
        addplot=apds,
        show_nontrading=False,
        figsize=(18, 10),
        warn_too_much_data=len(df) + 1,
    )

    if "volume" in df.columns:
        plot_kwargs["volume"] = True
        plot_kwargs["panel_ratios"] = (6, 3)
    if vlines_dict:
        plot_kwargs["vlines"] = vlines_dict

    if save_path:
        plot_kwargs["savefig"] = save_path

    mpf.plot(df, **plot_kwargs)
    
def plot_3d_metrics(trials, num_trial = [0,1,2],x_label='среднее значение f1', y_label='среднее значение AUC ROC', z_label='стандартное отклонение f1', deffs=None, directions=['максимизировать', 'максимизировать', 'минимизировать']):
    """
    Функция для построения 3D-графика на основе результатов Optuna.

    Параметры:
    -----------
    trials : list
        Список объектов `optuna.trial.FrozenTrial` из study.trials.
    x_label : str, optional
        Название для оси X (по умолчанию 'среднее значение f1').
    y_label : str, optional
        Название для оси Y (по умолчанию 'среднее значение AUC ROC').
    z_label : str, optional
        Название для оси Z (по умолчанию 'стандартное отклонение f1').
    deffs : list, optional
        Пороговые значения для фильтрации trials (по умолчанию None).
    directions : list, optional
        Направления оптимизации для каждой метрики (по умолчанию ['максимизировать', 'максимизировать', 'минимизировать']).
    """
    print(len(trials))

    # Сопоставление направлений с операторами сравнения
    direction_to_operator = {
        'максимизировать': lambda a, b: a > b,
        'минимизировать': lambda a, b: a < b
    }

    # Фильтрация trials
    if deffs is None:
        trials = [trial for trial in trials if trial.values is not None]
    else:
        trials = [
            trial for trial in trials
            if trial.values is not None
            and direction_to_operator[directions[0]](trial.values[0], deffs[0])
            and direction_to_operator[directions[1]](trial.values[1], deffs[1])
            and direction_to_operator[directions[2]](trial.values[2], deffs[2])
        ]

    print(len(trials))

    # Извлечение значений метрик
    x_vals = [trial.values[num_trial[0]] for trial in trials]
    y_vals = [trial.values[num_trial[1]] for trial in trials]
    z_vals = [trial.values[num_trial[2]] for trial in trials]

    # Форматирование параметров для hover text
    def format_params(params):
        return '<br>'.join([f"{key}: {value}" for key, value in params.items()])

    # Создание текста для hover
    hover_texts = [
        f"Number: {trial.number}<br>"
        f"{x_label}: {trial.values[num_trial[0]]:.4f}<br>"
        f"{y_label}: {trial.values[num_trial[1]]:.4f}<br>"
        f"{z_label}: {trial.values[num_trial[2]]:.4f}<br>"
        f"Params:<br>{format_params(trial.params)}"
        for trial in trials
    ]

    # Создание 3D-графика
    fig = go.Figure(data=[go.Scatter3d(
        x=x_vals, y=y_vals, z=z_vals,
        mode='markers',
        marker=dict(
            size=4,
            color=x_vals,  # Цветовая шкала может быть привязана к одной из метрик
            colorscale='Viridis',
            opacity=0.8
        ),
        text=hover_texts,  # Добавляем hover text
        hoverinfo='text'   # Указываем, что при наведении нужно показывать текст
    )])

    # Добавление меток к осям
    fig.update_layout(
        scene=dict(
            xaxis_title=x_label,
            yaxis_title=y_label,
            zaxis_title=z_label
        ),
        title="3 метрики через Optuna"
    )

    # Отображение графика
    fig.show()

def prepare_regime_params(optuna_params):
    """
    Преобразует параметры из формата Optuna в два словаря: базовые параметры режимов и параметры расчета.
    
    Args:
        optuna_params (dict): Словарь с параметрами из Optuna
        
    Returns:
        dict: Словарь с двумя ключами: 'base_params' (параметры режимов) и 'calc_params' (остальные параметры)
    """
    # Инициализируем словари для базовых параметров и параметров расчета
    start_params = {}
    base_params = {}
    calc_params = {}
    
    # Сначала обрабатываем параметры режимов (0-4)

    start_params['moving_average_length'] = optuna_params.get('moving_average_length', 14)
    start_params['atr_period'] = optuna_params.get('atr_period', 10)
    for regime in range(5):
        regime_key = f'regime_{regime}_'
        regime_params = {}
        
        # Основные параметры режима
        regime_params['average_type'] = optuna_params.get(f'{regime_key}average_type', 'SMA')
        regime_params['moving_average_length'] = optuna_params.get(f'{regime_key}ma_length', 50)
        regime_params['atr_period'] = optuna_params.get(f'{regime_key}atr_period', 14)
        regime_params['atr_multiplier'] = optuna_params.get(f'{regime_key}atr_multiplier', 3.0)
        
        # Параметры AMA, если они есть
        ama_atr_period = optuna_params.get(f'{regime_key}ama_atr_period')
        ama_min_period = optuna_params.get(f'{regime_key}ama_min_period')
        ama_max_period = optuna_params.get(f'{regime_key}ama_max_period')
        
        if regime_params['average_type'] == 'AMA' and all(p is not None for p in [ama_atr_period, ama_min_period, ama_max_period]):
            regime_params['ama_params'] = {
                'atr_period': int(ama_atr_period),
                'min_period': int(ama_min_period),
                'max_period': int(ama_max_period)
            }
        
        base_params[regime] = regime_params
    
    # Теперь собираем все остальные параметры в calc_params
    other_params = [
        'rsi_length', 'use_smoothing', 'smoothing_length', 'smoothing_type',
        'alma_sigma', 'rsi_overbought', 'rsi_oversold', 'use_knn',
        'knn_neighbors', 'knn_lookback', 'knn_weight', 'feature_count',
        'use_filter', 'filter_method', 'filter_strength', 'sma_length',
        'ema_length', 'rsi_helbuth'
    ]
    
    for param in other_params:
        if param in optuna_params:
            calc_params[param] = optuna_params[param]
    
    return {
        'start_params': start_params,
        'base_params': base_params,
        'calc_params': calc_params
    }

def pair_and_clean_signals(df: pd.DataFrame) -> pd.DataFrame:
    """
    Оставляет только первый buy после flat и первый sell после buy.
    Гарантирует, что каждая покупка имеет свою продажу.
    """
    df = df.copy()
    # пусть V5 уже записал нам в df['buy_signal'], df['sell_signal'] — raw-сигналы
    buy_raw  = df['buy_signal'].values
    sell_raw = df['sell_signal'].values

    # обнуляем
    df['buy_signal']  = False
    df['sell_signal'] = False

    in_pos = False
    for i in range(len(df)):
        if not in_pos and buy_raw[i]:
            # открываем новую сделку
            df.iat[i, df.columns.get_loc('buy_signal')] = True
            in_pos = True
        elif in_pos and sell_raw[i]:
            # закрываем
            df.iat[i, df.columns.get_loc('sell_signal')] = True
            in_pos = False
        # все остальные raw-сигналы игнорируем

    # если позиция осталась открытой — принудительный выход на последнем баре
    if in_pos and len(df)>0:
        df.iat[len(df)-1, df.columns.get_loc('sell_signal')] = True

    return df

class MachineLearningRSI:
    def __init__(self,
                 rsi_length=300,
                 use_smoothing=True,
                 smoothing_length=268,
                 smoothing_type='ALMA',
                 alma_sigma=6,
                 rsi_overbought=70,
                 rsi_oversold=30,
                 use_knn=True,
                 knn_neighbors=7,
                 knn_lookback=500,
                 knn_weight=0.6,
                 feature_count=5,
                 use_filter=True,
                 filter_method='Kalman',
                 filter_strength=0.7,
                 sma_length=20 + 7*24*4*3,
                 ema_length=21 + 7*24*4*3
                 ):

        # Базовые параметры
        self.rsi_length = rsi_length
        self.use_smoothing = use_smoothing
        self.smoothing_length = smoothing_length
        self.smoothing_type = smoothing_type
        self.alma_sigma = alma_sigma

        # Пороговые уровни
        self.rsi_overbought = rsi_overbought
        self.rsi_oversold = rsi_oversold

        # Параметры KNN
        self.use_knn = use_knn
        self.knn_neighbors = knn_neighbors
        self.knn_lookback = knn_lookback
        self.knn_weight = knn_weight
        self.feature_count = feature_count

        # Фильтрация
        self.use_filter = use_filter
        self.filter_method = filter_method
        self.filter_strength = filter_strength

        self.sma_length = sma_length
        self.ema_length = ema_length

    def calculate_rsi(self, close: pd.Series, length: int) -> pd.Series:
        """Расчет RSI через RMA аналогично PineScript ta.rsi"""
        delta = close.diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def smooth(self, series: pd.Series) -> pd.Series:
        """Корректный ALMA"""
        if self.smoothing_type == 'SMA':
            return series.rolling(self.smoothing_length).mean()
        elif self.smoothing_type == 'EMA':
            return series.ewm(span=self.smoothing_length, adjust=False).mean()
        elif self.smoothing_type == 'ALMA':
            m = self.smoothing_length
            offset = 0.85
            sigma = self.alma_sigma

            def alma(series):
                window = np.arange(m)
                weights = np.exp(-((window - offset * (m-1))**2) / (2*(sigma**2)))
                weights /= weights.sum()
                return np.convolve(series, weights, mode='valid')

            def alma_causal(series: pd.Series, length: int = 9, offset: float = 0.85, sigma: float = 6) -> pd.Series:
                """
                Казуальная реализация ALMA (Arnaud Legoux Moving Average)
                Использует только прошлые и текущие значения, без lookahead bias.
                """
                if length > len(series):
                    return pd.Series(np.nan, index=series.index)

                # Предвычисление весов ALMA
                window = np.arange(length)
                m = offset * (length - 1)
                s = length / sigma
                weights = np.exp(-((window - m) ** 2) / (2 * s ** 2))
                weights /= weights.sum()

                # Применяем ALMA казуально (rolling + dot product)
                alma_vals = []
                for i in range(length - 1, len(series)):
                    window_data = series.iloc[i - length + 1:i + 1]
                    if window_data.isnull().any():
                        alma_vals.append(np.nan)
                    else:
                        alma_vals.append(np.dot(weights, window_data.values))

                # Паддинг NaN в начало, чтобы сохранить индекс
                alma_series = pd.Series([np.nan] * (length - 1) + alma_vals, index=series.index)

                return alma_series

            alma_series = alma_causal(series.fillna(method='ffill'), m, offset, sigma)#, index=series.index[pad:-pad])
            #alma_series = alma_series.reindex(series.index, method='nearest')
            return alma_series
        else:
            return series

    def feature_extraction(self, close: pd.Series, rsi: pd.Series) -> pd.DataFrame:
        """Извлечение признаков для KNN"""
        features = pd.DataFrame(index=close.index)
        features['rsi'] = self.normalize(rsi, self.knn_lookback)

        if self.feature_count >= 2:
            features['momentum_rsi'] = self.normalize(rsi.diff(3), self.knn_lookback)
        if self.feature_count >= 3:
            features['volatility_rsi'] = self.normalize(rsi.rolling(10).std(), self.knn_lookback)
        if self.feature_count >= 4:
            features['slope_rsi'] = self.normalize(self.get_slope(rsi, 5), self.knn_lookback)
        if self.feature_count >= 5:
            features['momentum_price'] = self.normalize(close.diff(5), self.knn_lookback)

        return features.dropna()

    def normalize(self, series: pd.Series, period: int) -> pd.Series:
        """Мин-макс нормализация"""
        min_val = series.rolling(period).min()
        max_val = series.rolling(period).max()
        norm = (series - min_val) / (max_val - min_val)
        return norm.clip(0, 1)

    def get_slope(self, series: pd.Series, window: int) -> pd.Series:
        """Расчет наклона линейной регрессии"""
        idx = np.arange(window)
        def linreg(x):
            A = np.vstack([idx, np.ones(len(idx))]).T
            m, c = np.linalg.lstsq(A, x, rcond=None)[0]
            return m
        return series.rolling(window).apply(linreg, raw=True)

    def apply_knn(self, features: pd.DataFrame, rsi: pd.Series) -> pd.Series:
        """
        Оптимизированная, но идентичная оригиналу версия KNN.
        Сохраняет точную математику оригинального apply_knn_b с ускорением через BallTree.
        """
        # Сохраняем структуру как в оригинале
        full_index = rsi.index
        common_index = features.index.intersection(rsi.index)
        features = features.loc[common_index]
        rsi = rsi.loc[common_index]

        enhanced_rsi = pd.Series(index=full_index, data=np.nan)
        enhanced_rsi.loc[rsi.index] = rsi

        if len(features) < self.knn_lookback:
            return enhanced_rsi

        feature_array = features.values
        rsi_array = rsi.values

        # Основное изменение: BallTree строится на скользящем окне
        for t in range(self.knn_lookback, len(feature_array)):
            window_start = t - self.knn_lookback
            window_end = t
            X_window = feature_array[window_start:window_end]
            y_window = rsi_array[window_start:window_end]

            # Строим дерево только на текущем окне
            tree = BallTree(X_window, metric='euclidean')
            distances, indices = tree.query(feature_array[t].reshape(1, -1),
                                          k=self.knn_neighbors)

            # Точное воспроизведение оригинальной логики взвешивания
            weights = np.where(distances[0] < 1e-6, 1.0, 1.0 / distances[0])
            prediction = np.average(y_window[indices[0]], weights=weights)

            idx = common_index[t]
            enhanced_rsi.loc[idx] = (1 - self.knn_weight) * rsi.loc[idx] + self.knn_weight * prediction

        return enhanced_rsi

    def kalman_filter(self, series: pd.Series) -> pd.Series:
        """Калман-фильтр с параметрами ближе к PineScript"""
        n = len(series)
        xhat = np.full(n, np.nan)
        P = np.zeros(n)
        R = self.filter_strength * 0.1  # Очень маленький measurement noise
        Q = self.filter_strength * 0.01  # Очень маленький process noise

        first_valid_idx = series.first_valid_index()
        if first_valid_idx is None:
            return pd.Series(xhat, index=series.index)

        first_idx = series.index.get_loc(first_valid_idx)
        xhat[first_idx] = series.iloc[first_idx]
        P[first_idx] = 1.0

        for k in range(first_idx + 1, n):
            if np.isnan(series.iloc[k]):
                xhat[k] = xhat[k - 1]
                P[k] = P[k - 1] + Q
            else:
                xhatminus = xhat[k-1]
                Pminus = P[k-1] + Q
                K = Pminus / (Pminus + R)
                xhat[k] = xhatminus + K * (series.iloc[k] - xhatminus)
                P[k] = (1 - K) * Pminus

        return pd.Series(xhat, index=series.index)

    def filter_series(self, series: pd.Series) -> pd.Series:
        """Применение фильтрации к финальному RSI"""
        if self.filter_method == 'None':
            return series
        elif self.filter_method == 'Kalman':
            return self.kalman_filter(series)
        elif self.filter_method == 'DoubleEMA':
            ema1 = series.ewm(span=int(self.filter_strength * 10)).mean()
            ema2 = ema1.ewm(span=int(self.filter_strength * 5)).mean()
            return ema2
        elif self.filter_method == 'ALMA':
            return self.smooth(series)
        else:
            return series

    def week_level(self, close):
        sma_length = self.sma_length
        ema_length = self.ema_length

        # Вычисление 20-недельной SMA
        SMA_20w = close.rolling(window=sma_length, min_periods=1).mean()

        # Вычисление 21-недельной EMA
        MA_21w = close.ewm(span=ema_length, adjust=False).mean()

        return SMA_20w, MA_21w


    def fit(self, close: pd.Series) -> pd.Series:
        """Основная функция расчёта"""
        rsi = self.calculate_rsi(close, self.rsi_length)
        if self.use_smoothing:
            rsi = self.smooth(rsi)
        if self.use_knn:
            features = self.feature_extraction(close, rsi)

            rsi = self.apply_knn(features, rsi)

        if self.use_filter:
            rsi = self.filter_series(rsi)

        sma, ma = self.week_level(close)

        return rsi.clip(0, 100), sma, ma


class TinkoffHistoricalDataCollector:
    def __init__(self):
        self.sma_state = {}

    def generateVar(self, high_array, low_array, moving_average_length=10):
        valpha = 2 / (moving_average_length + 1)
        hl2 = (high_array + low_array) / 2

        before_val = hl2[0] if len(hl2) > 0 else 0

        vud1 = []
        vdd1 = []
        for current_hl2 in hl2:
            if current_hl2 > before_val:
                vud1.append(current_hl2 - before_val)
                vdd1.append(0)
            elif current_hl2 < before_val:
                vdd1.append(before_val - current_hl2)
                vud1.append(0)
            else:
                vud1.append(0)
                vdd1.append(0)
            before_val = current_hl2

        def calculate_window_sums(arr, window_size=9):
          return [sum(arr[max(0, i - window_size + 1):i+1]) for i in range(len(arr))]

        vUD = calculate_window_sums(vud1, 9)
        vDD = calculate_window_sums(vdd1, 9)

        vUD_ar = np.array(vUD)
        vDD_ar = np.array(vDD)

        epsilon = 1e-10
        with np.errstate(divide='ignore', invalid='ignore'):
            vCMO = np.divide(vUD_ar - vDD_ar, vUD_ar + vDD_ar + epsilon)

        vCMO = np.nan_to_num(vCMO, nan=0.0)

        var = []
        var_before = 0.0
        for i in range(len(hl2)):
            if i < len(vCMO):
                cmo = abs(vCMO[i])
                var_current = (valpha * cmo * hl2[i]) + (1 - valpha * cmo) * var_before
            else:
                var_current = var_before
            var.append(var_current)
            var_before = var_current

        return np.array(var)

    def generateAma(self, high_array, low_array, close_array, atr_period=14, min_period=5, max_period=50):
        """
        Генерация адаптивного скользящего среднего на основе волатильности.

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param close_array: Массив значений close.
        :param atr_period: Период для расчета ATR.
        :param min_period: Минимальный период скользящего среднего.
        :param max_period: Максимальный период скользящего среднего.
        :return: Массив значений адаптивного скользящего среднего.
        """
        # Рассчитываем ATR
        atr = self._calculate_atr(high_array, low_array, close_array, atr_period)

        # Нормализуем ATR для использования в качестве коэффициента
        normalized_atr = (atr - np.min(atr)) / (np.max(atr) - np.min(atr) + 1e-10)

        # Рассчитываем динамический период
        dynamic_period = min_period + (max_period - min_period) * normalized_atr

        # Рассчитываем адаптивное скользящее среднее (гибрид SMA и EMA)
        adaptive_ma = np.zeros_like(close_array)
        for i in range(len(close_array)):
            if i < int(dynamic_period[i]):
                adaptive_ma[i] = np.mean(close_array[:i+1])  # SMA для начальных значений
            else:
                period = int(dynamic_period[i])
                alpha = 2 / (period + 1)
                adaptive_ma[i] = alpha * close_array[i] + (1 - alpha) * adaptive_ma[i-1]  # EMA

        return adaptive_ma

    def _calculate_atr(self, high_array, low_array, close_array, period=14):
        """
        Рассчитывает Average True Range (ATR).

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param close_array: Массив значений close.
        :param period: Период для расчета ATR.
        :return: Массив значений ATR.
        """
        tr = np.zeros_like(high_array)
        tr[0] = high_array[0] - low_array[0]

        for i in range(1, len(high_array)):
            hl = high_array[i] - low_array[i]
            hc = abs(high_array[i] - close_array[i-1])
            lc = abs(low_array[i] - close_array[i-1])
            tr[i] = max(hl, hc, lc)

        atr = np.zeros_like(tr)
        atr[period-1] = np.mean(tr[:period])

        for i in range(period, len(tr)):
            atr[i] = (atr[i-1] * (period-1) + tr[i]) / period

        return atr

    def generateAtr(self, high_array, low_array, close_array, period=14):

        # Рассчитываем True Range (TR)
        tr1 = high_array - low_array
        tr2 = np.abs(high_array - np.roll(close_array, 1))
        tr3 = np.abs(low_array - np.roll(close_array, 1))

        tr = np.maximum(tr1, np.maximum(tr2, tr3))

        # Рассчитываем ATR
        atr = np.zeros_like(tr)
        atr[period - 1] = np.mean(tr[:period])

        for i in range(period, len(tr)):
            atr[i] = (atr[i - 1] * (period - 1) + tr[i]) / period

        return atr

    def generateSma(self, high_array, low_array, window=10):
        """
        Генерация Simple Moving Average (SMA).

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param window: Период SMA.
        :return: Массив значений SMA.
        """
        hl2 = (high_array + low_array) * 0.5

        if window <= 1:
            return hl2

        # Создаем массив для результатов с NaN
        sma = np.full_like(hl2, np.nan)

        # Рассчитываем кумулятивную сумму
        cumsum = np.cumsum(hl2)

        # Создаем сдвинутый кумулятивный массив
        shifted_cumsum = np.zeros_like(cumsum)
        shifted_cumsum[window:] = cumsum[:-window]

        # Вычисляем SMA для валидных периодов
        valid = slice(window - 1, None)
        sma[valid] = (cumsum[valid] - shifted_cumsum[valid]) / window

        return sma

    def generatePMax(self, var_array, close_array, high_array, low_array, atr_period, atr_multiplier):
        """
        Генерация PMax (Profit Maximizer).

        :param var_array: Массив значений скользящего среднего.
        :param close_array: Массив значений close.
        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param atr_period: Период для расчета ATR.
        :param atr_multiplier: Множитель ATR.
        :return: Массив значений PMax.
        """
        try:
            atr = self.generateAtr(high_array, low_array, close_array, period=atr_period)
        except Exception as exp:
            print('exception in atr:', str(exp), flush=True)
            return []

        previous_final_upperband = 0
        previous_final_lowerband = 0
        final_upperband = 0
        final_lowerband = 0
        previous_var = 0
        previous_pmax = 0
        pmax = []
        pmaxc = 0

        for i in range(0, len(close_array)):
            if np.isnan(close_array[i]):
                pass
            else:
                atrc = atr[i]
                varc = var_array[i]

                if math.isnan(atrc):
                    atrc = 0

                basic_upperband = varc + atr_multiplier * atrc
                basic_lowerband = varc - atr_multiplier * atrc

                if basic_upperband < previous_final_upperband or previous_var > previous_final_upperband:
                    final_upperband = basic_upperband
                else:
                    final_upperband = previous_final_upperband

                if basic_lowerband > previous_final_lowerband or previous_var < previous_final_lowerband:
                    final_lowerband = basic_lowerband
                else:
                    final_lowerband = previous_final_lowerband

                if previous_pmax == previous_final_upperband and varc <= final_upperband:
                    pmaxc = final_upperband
                else:
                    if previous_pmax == previous_final_upperband and varc >= final_upperband:
                        pmaxc = final_lowerband
                    else:
                        if previous_pmax == previous_final_lowerband and varc >= final_lowerband:
                            pmaxc = final_lowerband
                        elif previous_pmax == previous_final_lowerband and varc <= final_lowerband:
                            pmaxc = final_upperband

                pmax.append(pmaxc)

                previous_var = varc

                previous_final_upperband = final_upperband

                previous_final_lowerband = final_lowerband

                previous_pmax = pmaxc

        return pmax

    def generate_signals(self, df, moving_average_length=10, atr_period=10, atr_multiplier=3, average_type='SMA',
                        ama_params=None):
        """
        Генерация сигналов на основе SMA или AMA.

        :param df: DataFrame с данными.
        :param moving_average_length: Период скользящего среднего.
        :param atr_period: Период ATR.
        :param atr_multiplier: Множитель ATR.
        :param average_type: Тип скользящего среднего ('SMA' или 'AMA').
        :param ama_params: Параметры для AMA (если используется).
        :return: DataFrame с добавленными сигналами.
        """
        high_array = df["high"].values
        low_array = df["low"].values
        close_array = df["close"].values
        df = df.copy()

        if average_type == 'SMA':
            ma_arr = self.generateSma(high_array, low_array, moving_average_length)
        elif average_type == 'VAR':
            ma_arr = self.generateVar(high_array, low_array, moving_average_length)
        elif average_type == 'AMA':
            if ama_params is None:
                raise ValueError("Для AMA необходимо указать параметры ama_params.")
            ma_arr = self.generateAma(high_array, low_array, close_array, **ama_params)
        else:
            raise ValueError("Неподдерживаемый тип скользящего среднего.")

        pmax = self.generatePMax(ma_arr, close_array, high_array, low_array, atr_period, atr_multiplier)
        df["pmax"] = pmax
        df["ma"] = ma_arr
        df["buy_signal"] = (df["ma"] > df["pmax"]) & (df["ma"].shift(1) < df["pmax"].shift(1))
        df["sell_signal"] = (df["ma"] < df["pmax"]) & (df["ma"].shift(1) > df["pmax"].shift(1))

        return df

def calculate_target(df, threshold=3.0):
    # Проверка необходимых колонок
    required_columns = ['event_price', 'event_sell_price']
    missing_cols = [col for col in required_columns if col not in df.columns]

    if missing_cols:
        raise ValueError(f"Отсутствуют необходимые колонки: {missing_cols}")

    # Копируем DataFrame чтобы не менять оригинал
    result_df = df.copy()

    # Рассчитываем процентное изменение
    result_df['price_change_pct'] = (
        (result_df['event_sell_price'] / result_df['event_price'] - 1) * 100
    )

    # Создаем целевой признак
    result_df['target'] = (result_df['price_change_pct'] >= threshold).astype(int)

    # Обработка случаев с отсутствующими данными
    result_df['target'] = result_df['target'].where(
        result_df[['event_price', 'event_sell_price']].notnull().all(axis=1),
        other=0
    )

    # Обработка случаев с нулевой ценой покупки (если такие есть)
    result_df['target'] = result_df['target'].where(
        result_df['event_price'] != 0,
        other=0
    )

    # Удаляем временную колонку
    result_df.drop('price_change_pct', axis=1, inplace=True)

    return result_df

class FastRollingMode:
    def __init__(self, window_size):
        self.window = deque(maxlen=window_size)
        self.counts = {}
        
    def update(self, new_val):
        if len(self.window) == self.window.maxlen:
            old_val = self.window.popleft()
            self.counts[old_val] -= 1
            if self.counts[old_val] == 0:
                del self.counts[old_val]
        
        self.window.append(new_val)
        self.counts[new_val] = self.counts.get(new_val, 0) + 1
        return max(self.counts.items(), key=lambda x: x[1])[0]

def extract_features(df: pd.DataFrame, window: int = 126):
    """
    Вычисляет устойчивые признаки для кластеризации рыночных режимов.
    """

    def calculate_macd(df, macd_fast_periods=[12], macd_slow_periods=[26], macd_signal_periods=[9]):
        """
        Быстрый расчет нормализованного MACD с использованием векторизованных операций
        """
        close = df['close']

        # Создаем множества для уникальных периодов
        unique_fast = set(macd_fast_periods)
        unique_slow = set(macd_slow_periods)


        # Предварительно вычисляем все необходимые EMA и скользящие средние
        ema_cache = {}
        rolling_cache = {}

        # Кешируем быстрые EMA
        for fp in unique_fast:
            ema_cache[f'ema_{fp}'] = close.ewm(span=fp, adjust=False).mean()

        # Кешируем медленные EMA и скользящие средние
        for sp in unique_slow:
            ema_cache[f'ema_{sp}'] = close.ewm(span=sp, adjust=False).mean()
            rolling_cache[f'rolling_{sp}'] = close.rolling(window=sp).mean()

        # Основной цикл вычислений
        for fp in macd_fast_periods:
            ema_fast = ema_cache[f'ema_{fp}']
            for sp in macd_slow_periods:
                ema_slow = ema_cache[f'ema_{sp}']
                rolling_mean = rolling_cache[f'rolling_{sp}']

                # Вычисляем MACD и нормализацию
                macd = ema_fast - ema_slow
                macd_norm = macd / rolling_mean

                # Сохраняем MACD только один раз для комбинации fp/sp

                # Обрабатываем сигнальные периоды
                for sig in macd_signal_periods:
                    # Вычисляем сигнальную линию
                    signal = macd.ewm(span=sig, adjust=False).mean()
                    signal_norm = signal / rolling_mean

        return pd.DataFrame([macd_norm, signal_norm, macd_norm - signal_norm]).T.fillna(0)

    def calculate_atr(df, atr_window=14):
        """
        Расчет ATR и его сдвигов.
        """
        high = df['high']
        low = df['low']
        close = df['close']
    
        tr1 = high - low
        tr2 = np.abs(high - close.shift(1))
        tr3 = np.abs(low - close.shift(1))
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(atr_window).mean()
    
        return pd.Series(atr).fillna(0)
    
    def calculate_rsi(df, rsi_period=14):
        """
        Расчет RSI и его сдвиги.
        """
        close = df['close']
        delta = close.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(rsi_period).mean()
        avg_loss = loss.rolling(rsi_period).mean()
        rs = avg_gain / (avg_loss + 1e-10)
        rsi = 100 - (100 / (1 + rs))
        
        return pd.Series(rsi).fillna(0)
    
    def calculate_bollinger_bands(df, bollinger_window=20):
        """
        Расчет Bollinger Bands (ширины полос) и сдвигов.
        """
        close = df['close']
        ma = close.rolling(bollinger_window).mean()
        std = close.rolling(bollinger_window).std()
        bb_width = (2 * std) / ma
    
        return pd.Series(bb_width).fillna(0)
    
    def detect_market_regime(df: pd.DataFrame, window: int = 30, n_clusters: int = 3) -> pd.Series:
        """
        Классифицирует рыночную фазу на основе кластеризации признаков: волатильность, автокорреляция, наклон тренда.
        Возвращает метку режима рынка для каждого окна.
        """
        from sklearn.cluster import KMeans
        from sklearn.preprocessing import StandardScaler
        import pandas as pd
        import numpy as np
    
        features = []
    
        for i in range(len(df) - window + 1):
            window_df = df.iloc[i:i+window]
            close = window_df['close'].values
    
            # Волатильность (стандартное отклонение)
            volatility = np.std(np.diff(close))
    
            # Наклон тренда (регрессия по времени)
            x = np.arange(window)
            y = close
            slope = np.polyfit(x, y, deg=1)[0]
    
            # Автокорреляция лаг-1
            autocorr = np.corrcoef(close[:-1], close[1:])[0, 1]
    
            features.append([volatility, slope, autocorr])
    
        features = np.array(features)
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
    
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(features_scaled)
    
        # Расширим метки до длины df
        regime_series = pd.Series(np.nan, index=df.index)
        regime_series.iloc[window - 1:] = labels
    
        return regime_series.fillna(0).ffill().astype(int)
    macd_trend = calculate_macd(df, macd_slow_periods=[window], macd_fast_periods=[window//3], 
                                 macd_signal_periods=[window//6])
    atr = calculate_atr(df, atr_window=window)
    rel_volatility = atr / df["close"]
    rsi_ind = calculate_rsi(df, rsi_period=window//2)
    volume_ratio = df['volume'].rolling(window).apply(
        lambda x: x[-1]/x.mean(), raw=True
    ).fillna(1).values

    features = np.column_stack([
        macd_trend,
        rel_volatility,
        rsi_ind,
        volume_ratio
    ])

    return features

def find_best_trial_by_weighted_score_extended1(
    trials: List[FrozenTrial],
    pnl_weight: float = 0.45,
    diff_weight: float = 0.45,
    score_weight: float = 0.10
) -> Tuple[float, int, Dict, str, float, float, float, float, float, float]:
    """
    Ищет лучший трейл по взвешенной сумме метрик для двух стратегий.
    
    Порядок метрик в trial.values:
    0: pnl_rsi
    1: pnl_full
    2: diff_rsi
    3: diff_full
    4: score_rsi
    5: score_full
    
    Параметры:
        - pnl_weight: вес для метрики PnL (по умолчанию 0.45)
        - diff_weight: вес для метрики diff (по умолчанию 0.45)
        - score_weight: вес для метрики score (по умолчанию 0.10)
    
    Возвращает:
        - best_score: наивысший взвешенный скор
        - best_trial_number: номер лучшего трейла
        - best_params: параметры лучшего трейла
        - best_strategy: лучшая стратегия ('RSI' или 'RSI_SMA_EMA')
        - best_pnl: лучшее PnL
        - best_diff: лучшее diff
        - best_score_n: лучшее score
        - pnl_rsi: PnL RSI для лучшего трейла
        - diff_rsi: diff RSI для лучшего трейла
        - score_rsi: score RSI для лучшего трейла
        - pnl_full: PnL полной стратегии для лучшего трейла
        - diff_full: diff полной стратегии для лучшего трейла
        - score_full: score полной стратегии для лучшего трейла
    """
    # Проверка корректности весов
    total_weight = pnl_weight + diff_weight + score_weight
    if not np.isclose(total_weight, 1.0):
        raise ValueError(f"Сумма весов должна быть равна 1.0 (получено {total_weight})")
    
    # Находим максимальные значения для нормализации
    all_pnls = []
    all_diffs = []
    all_scores = []
    
    for trial in trials:
        if trial.values and len(trial.values) >= 6:
            all_pnls.extend([trial.values[0], trial.values[1]])  # pnl_rsi, pnl_full
            all_diffs.extend([trial.values[2], trial.values[3]]) # diff_rsi, diff_full
            all_scores.extend([trial.values[4], trial.values[5]]) # score_rsi, score_full
    
    if not all_pnls:
        raise ValueError("Нет подходящих трейлов с шестью метриками.")
    
    MAX_PNL = max(all_pnls) * 1.5  # Добавляем запас 50%
    MAX_DIFF = max(all_diffs) * 1.2 if max(all_diffs) > 0 else 1.0
    MAX_SCORE = max(all_scores) * 1.2 if max(all_scores) > 0 else 1.0
    
    best_score = float('-inf')
    best_trial_number = -1
    best_params = None
    best_strategy = None
    best_pnl = None
    best_diff = None
    best_score_n = None

    for trial in trials:
        if not trial.values or len(trial.values) < 6:
            continue
            
        pnl_rsi, pnl_full, diff_rsi, diff_full, score_rsi, score_full = trial.values
        
        # Нормализация (с защитой от деления на 0)
        norm_pnl_rsi = pnl_rsi / MAX_PNL if MAX_PNL > 0 else 0
        norm_pnl_full = pnl_full / MAX_PNL if MAX_PNL > 0 else 0
        norm_diff_rsi = diff_rsi / MAX_DIFF if MAX_DIFF > 0 else 0
        norm_diff_full = diff_full / MAX_DIFF if MAX_DIFF > 0 else 0
        norm_score_rsi = score_rsi / MAX_SCORE if MAX_SCORE > 0 and score_weight > 0 else 0
        norm_score_full = score_full / MAX_SCORE if MAX_SCORE > 0 and score_weight > 0 else 0
        
        # Вычисляем взвешенные scores для обеих стратегий
        score_rsi_weighted = (pnl_weight * norm_pnl_rsi + 
                             diff_weight * norm_diff_rsi + 
                             score_weight * norm_score_rsi)
        
        score_full_weighted = (pnl_weight * norm_pnl_full + 
                              diff_weight * norm_diff_full + 
                              score_weight * norm_score_full)
        
        # Выбираем лучшую стратегию в этом трейле
        if score_rsi_weighted > score_full_weighted:
            current_score = score_rsi_weighted
            current_strategy = 'RSI'
            current_pnl = pnl_rsi
            current_diff = diff_rsi
            current_score_n = score_rsi
        else:
            current_score = score_full_weighted
            current_strategy = 'RSI_SMA_EMA'
            current_pnl = pnl_full
            current_diff = diff_full
            current_score_n = score_full
        
        # Обновляем глобальный лучший результат
        if current_score > best_score:
            best_score = current_score
            best_trial_number = trial.number
            best_params = trial.params
            best_strategy = current_strategy
            best_pnl = current_pnl
            best_diff = current_diff
            best_score_n = current_score_n

    # Получаем все метрики для лучшего трейла
    best_trial = next(t for t in trials if t.number == best_trial_number)
    pnl_rsi, pnl_full, diff_rsi, diff_full, score_rsi, score_full = best_trial.values

    return (
        best_score,
        best_trial_number,
        best_params,
        best_strategy,
        best_pnl,
        best_diff,
        best_score_n,
        pnl_rsi,
        diff_rsi,
        score_rsi,
        pnl_full,
        diff_full,
        score_full
    )

def fast_generate_var(high: np.ndarray,
                      low:  np.ndarray,
                      L:    int = 10,
                      W:    int = 9) -> np.ndarray:
    """
    Vectorized generation of your VAR (CMO‐smoothed) series.
    """
    α0 = 2.0 / (L + 1.0)
    hl2 = (high + low) * 0.5

    # 1) first diffs
    diff = np.empty_like(hl2)
    diff[0] = 0.0
    diff[1:] = hl2[1:] - hl2[:-1]

    # 2) up/down
    up = np.where(diff > 0, diff, 0.0)
    dn = np.where(diff < 0, -diff, 0.0)

    # 3) W‐period sums via convolution
    kernel = np.ones(W, dtype=np.float64)
    sum_up = np.convolve(up, kernel, mode="full")[: len(up)]
    sum_dn = np.convolve(dn, kernel, mode="full")[: len(dn)]

    # 4) CMO
    denom = sum_up + sum_dn + 1e-10
    cmo   = np.abs((sum_up - sum_dn) / denom)

    # 5) final EMA‐like smoothing
    var = np.empty_like(hl2)
    var[0] = 0.0
    for i in range(1, len(hl2)):
        α      = α0 * cmo[i]
        var[i] = α * hl2[i] + (1 - α) * var[i - 1]

    return var


class MachineLearningRSI:
    def __init__(self,
                 rsi_length=300,
                 use_smoothing=True,
                 smoothing_length=268,
                 smoothing_type='ALMA',
                 alma_sigma=6,
                 rsi_overbought=70,
                 rsi_oversold=30,
                 use_knn=True,
                 knn_neighbors=7,
                 knn_lookback=500,
                 knn_weight=0.6,
                 feature_count=5,
                 use_filter=True,
                 filter_method='Kalman',
                 filter_strength=0.7,
                 sma_length=20 + 7*24*4*3,
                 ema_length=21 + 7*24*4*3
                 ):

        # Базовые параметры
        self.rsi_length = rsi_length
        self.use_smoothing = use_smoothing
        self.smoothing_length = smoothing_length
        self.smoothing_type = smoothing_type
        self.alma_sigma = alma_sigma

        # Пороговые уровни
        self.rsi_overbought = rsi_overbought
        self.rsi_oversold = rsi_oversold

        # Параметры KNN
        self.use_knn = use_knn
        self.knn_neighbors = knn_neighbors
        self.knn_lookback = knn_lookback
        self.knn_weight = knn_weight
        self.feature_count = feature_count

        # Фильтрация
        self.use_filter = use_filter
        self.filter_method = filter_method
        self.filter_strength = filter_strength

        self.sma_length = sma_length
        self.ema_length = ema_length

    def calculate_rsi(self, close: pd.Series, length: int) -> pd.Series:
        """Расчет RSI через RMA аналогично PineScript ta.rsi"""
        delta = close.diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def smooth(self, series: pd.Series) -> pd.Series:
        """Корректный ALMA"""
        if self.smoothing_type == 'SMA':
            return series.rolling(self.smoothing_length).mean()
        elif self.smoothing_type == 'EMA':
            return series.ewm(span=self.smoothing_length, adjust=False).mean()
        elif self.smoothing_type == 'ALMA':
            m = self.smoothing_length
            offset = 0.85
            sigma = self.alma_sigma

            def alma(series):
                window = np.arange(m)
                weights = np.exp(-((window - offset * (m-1))**2) / (2*(sigma**2)))
                weights /= weights.sum()
                return np.convolve(series, weights, mode='valid')

            def alma_causal(series: pd.Series, length: int = 9, offset: float = 0.85, sigma: float = 6) -> pd.Series:
                """
                Казуальная реализация ALMA (Arnaud Legoux Moving Average)
                Использует только прошлые и текущие значения, без lookahead bias.
                """
                if length > len(series):
                    return pd.Series(np.nan, index=series.index)

                # Предвычисление весов ALMA
                window = np.arange(length)
                m = offset * (length - 1)
                s = length / sigma
                weights = np.exp(-((window - m) ** 2) / (2 * s ** 2))
                weights /= weights.sum()

                # Применяем ALMA казуально (rolling + dot product)
                alma_vals = []
                for i in range(length - 1, len(series)):
                    window_data = series.iloc[i - length + 1:i + 1]
                    if window_data.isnull().any():
                        alma_vals.append(np.nan)
                    else:
                        alma_vals.append(np.dot(weights, window_data.values))

                # Паддинг NaN в начало, чтобы сохранить индекс
                alma_series = pd.Series([np.nan] * (length - 1) + alma_vals, index=series.index)

                return alma_series

            alma_series = alma_causal(series.fillna(method='ffill'), m, offset, sigma)#, index=series.index[pad:-pad])
            #alma_series = alma_series.reindex(series.index, method='nearest')
            return alma_series
        else:
            return series

    def feature_extraction(self, close: pd.Series, rsi: pd.Series) -> pd.DataFrame:
        """Извлечение признаков для KNN"""
        features = pd.DataFrame(index=close.index)
        features['rsi'] = self.normalize(rsi, self.knn_lookback)

        if self.feature_count >= 2:
            features['momentum_rsi'] = self.normalize(rsi.diff(3), self.knn_lookback)
        if self.feature_count >= 3:
            features['volatility_rsi'] = self.normalize(rsi.rolling(10).std(), self.knn_lookback)
        if self.feature_count >= 4:
            features['slope_rsi'] = self.normalize(self.get_slope(rsi, 5), self.knn_lookback)
        if self.feature_count >= 5:
            features['momentum_price'] = self.normalize(close.diff(5), self.knn_lookback)

        return features.dropna()

    def normalize(self, series: pd.Series, period: int) -> pd.Series:
        """Мин-макс нормализация"""
        min_val = series.rolling(period).min()
        max_val = series.rolling(period).max()
        norm = (series - min_val) / (max_val - min_val)
        return norm.clip(0, 1)

    def get_slope(self, series: pd.Series, window: int) -> pd.Series:
        """Расчет наклона линейной регрессии"""
        idx = np.arange(window)
        def linreg(x):
            A = np.vstack([idx, np.ones(len(idx))]).T
            m, c = np.linalg.lstsq(A, x, rcond=None)[0]
            return m
        return series.rolling(window).apply(linreg, raw=True)

    def apply_knn(self, features: pd.DataFrame, rsi: pd.Series) -> pd.Series:
        """
        Оптимизированная, но идентичная оригиналу версия KNN.
        Сохраняет точную математику оригинального apply_knn_b с ускорением через BallTree.
        """
        # Сохраняем структуру как в оригинале
        full_index = rsi.index
        common_index = features.index.intersection(rsi.index)
        features = features.loc[common_index]
        rsi = rsi.loc[common_index]

        enhanced_rsi = pd.Series(index=full_index, data=np.nan)
        enhanced_rsi.loc[rsi.index] = rsi

        if len(features) < self.knn_lookback:
            return enhanced_rsi

        feature_array = features.values
        rsi_array = rsi.values

        # Основное изменение: BallTree строится на скользящем окне
        for t in range(self.knn_lookback, len(feature_array)):
            window_start = t - self.knn_lookback
            window_end = t
            X_window = feature_array[window_start:window_end]
            y_window = rsi_array[window_start:window_end]

            # Строим дерево только на текущем окне
            tree = BallTree(X_window, metric='euclidean')
            distances, indices = tree.query(feature_array[t].reshape(1, -1),
                                          k=self.knn_neighbors)

            # Точное воспроизведение оригинальной логики взвешивания
            weights = np.where(distances[0] < 1e-6, 1.0, 1.0 / distances[0])
            prediction = np.average(y_window[indices[0]], weights=weights)

            idx = common_index[t]
            enhanced_rsi.loc[idx] = (1 - self.knn_weight) * rsi.loc[idx] + self.knn_weight * prediction

        return enhanced_rsi

    def kalman_filter(self, series: pd.Series) -> pd.Series:
        """Калман-фильтр с параметрами ближе к PineScript"""
        n = len(series)
        xhat = np.full(n, np.nan)
        P = np.zeros(n)
        R = self.filter_strength * 0.1  # Очень маленький measurement noise
        Q = self.filter_strength * 0.01  # Очень маленький process noise

        first_valid_idx = series.first_valid_index()
        if first_valid_idx is None:
            return pd.Series(xhat, index=series.index)

        first_idx = series.index.get_loc(first_valid_idx)
        xhat[first_idx] = series.iloc[first_idx]
        P[first_idx] = 1.0

        for k in range(first_idx + 1, n):
            if np.isnan(series.iloc[k]):
                xhat[k] = xhat[k - 1]
                P[k] = P[k - 1] + Q
            else:
                xhatminus = xhat[k-1]
                Pminus = P[k-1] + Q
                K = Pminus / (Pminus + R)
                xhat[k] = xhatminus + K * (series.iloc[k] - xhatminus)
                P[k] = (1 - K) * Pminus

        return pd.Series(xhat, index=series.index)

    def filter_series(self, series: pd.Series) -> pd.Series:
        """Применение фильтрации к финальному RSI"""
        if self.filter_method == 'None':
            return series
        elif self.filter_method == 'Kalman':
            return self.kalman_filter(series)
        elif self.filter_method == 'DoubleEMA':
            ema1 = series.ewm(span=int(self.filter_strength * 10)).mean()
            ema2 = ema1.ewm(span=int(self.filter_strength * 5)).mean()
            return ema2
        elif self.filter_method == 'ALMA':
            return self.smooth(series)
        else:
            return series

    def week_level(self, close):
        sma_length = self.sma_length
        ema_length = self.ema_length

        # Вычисление 20-недельной SMA
        SMA_20w = close.rolling(window=sma_length, min_periods=1).mean()

        # Вычисление 21-недельной EMA
        MA_21w = close.ewm(span=ema_length, adjust=False).mean()

        return SMA_20w, MA_21w


    def fit(self, close: pd.Series) -> pd.Series:
        """Основная функция расчёта"""
        rsi = self.calculate_rsi(close, self.rsi_length)
        if self.use_smoothing:
            rsi = self.smooth(rsi)
        if self.use_knn:
            features = self.feature_extraction(close, rsi)

            rsi = self.apply_knn(features, rsi)

        if self.use_filter:
            rsi = self.filter_series(rsi)

        sma, ma = self.week_level(close)

        return rsi.clip(0, 100), sma, ma


class TinkoffHistoricalDataCollector:
    def __init__(self):
        self.sma_state = {}

    def generateVar(self, high_array, low_array, moving_average_length=10):
        valpha = 2 / (moving_average_length + 1)
        hl2 = (high_array + low_array) / 2

        before_val = hl2[0] if len(hl2) > 0 else 0

        vud1 = []
        vdd1 = []
        for current_hl2 in hl2:
            if current_hl2 > before_val:
                vud1.append(current_hl2 - before_val)
                vdd1.append(0)
            elif current_hl2 < before_val:
                vdd1.append(before_val - current_hl2)
                vud1.append(0)
            else:
                vud1.append(0)
                vdd1.append(0)
            before_val = current_hl2

        def calculate_window_sums(arr, window_size=9):
          return [sum(arr[max(0, i - window_size + 1):i+1]) for i in range(len(arr))]

        vUD = calculate_window_sums(vud1, 9)
        vDD = calculate_window_sums(vdd1, 9)

        vUD_ar = np.array(vUD)
        vDD_ar = np.array(vDD)

        epsilon = 1e-10
        with np.errstate(divide='ignore', invalid='ignore'):
            vCMO = np.divide(vUD_ar - vDD_ar, vUD_ar + vDD_ar + epsilon)

        vCMO = np.nan_to_num(vCMO, nan=0.0)

        var = []
        var_before = 0.0
        for i in range(len(hl2)):
            if i < len(vCMO):
                cmo = abs(vCMO[i])
                var_current = (valpha * cmo * hl2[i]) + (1 - valpha * cmo) * var_before
            else:
                var_current = var_before
            var.append(var_current)
            var_before = var_current

        return np.array(var)

    def generateAma(self, high_array, low_array, close_array, atr_period=14, min_period=5, max_period=50):
        """
        Генерация адаптивного скользящего среднего на основе волатильности.

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param close_array: Массив значений close.
        :param atr_period: Период для расчета ATR.
        :param min_period: Минимальный период скользящего среднего.
        :param max_period: Максимальный период скользящего среднего.
        :return: Массив значений адаптивного скользящего среднего.
        """
        # Рассчитываем ATR
        atr = self._calculate_atr(high_array, low_array, close_array, atr_period)

        # Нормализуем ATR для использования в качестве коэффициента
        normalized_atr = (atr - np.min(atr)) / (np.max(atr) - np.min(atr) + 1e-10)

        # Рассчитываем динамический период
        dynamic_period = min_period + (max_period - min_period) * normalized_atr

        # Рассчитываем адаптивное скользящее среднее (гибрид SMA и EMA)
        adaptive_ma = np.zeros_like(close_array)
        for i in range(len(close_array)):
            if i < int(dynamic_period[i]):
                adaptive_ma[i] = np.mean(close_array[:i+1])  # SMA для начальных значений
            else:
                period = int(dynamic_period[i])
                alpha = 2 / (period + 1)
                adaptive_ma[i] = alpha * close_array[i] + (1 - alpha) * adaptive_ma[i-1]  # EMA

        return adaptive_ma

    def _calculate_atr(self, high_array, low_array, close_array, period=14):
        """
        Рассчитывает Average True Range (ATR).

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param close_array: Массив значений close.
        :param period: Период для расчета ATR.
        :return: Массив значений ATR.
        """
        tr = np.zeros_like(high_array)
        tr[0] = high_array[0] - low_array[0]

        for i in range(1, len(high_array)):
            hl = high_array[i] - low_array[i]
            hc = abs(high_array[i] - close_array[i-1])
            lc = abs(low_array[i] - close_array[i-1])
            tr[i] = max(hl, hc, lc)

        atr = np.zeros_like(tr)
        atr[period-1] = np.mean(tr[:period])

        for i in range(period, len(tr)):
            atr[i] = (atr[i-1] * (period-1) + tr[i]) / period

        return atr

    def generateAtr(self, high_array, low_array, close_array, period=14):

        # Рассчитываем True Range (TR)
        tr1 = high_array - low_array
        tr2 = np.abs(high_array - np.roll(close_array, 1))
        tr3 = np.abs(low_array - np.roll(close_array, 1))

        tr = np.maximum(tr1, np.maximum(tr2, tr3))

        # Рассчитываем ATR
        atr = np.zeros_like(tr)
        atr[period - 1] = np.mean(tr[:period])

        for i in range(period, len(tr)):
            atr[i] = (atr[i - 1] * (period - 1) + tr[i]) / period

        return atr

    def generateSma(self, high_array, low_array, window=10):
        """
        Генерация Simple Moving Average (SMA).

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param window: Период SMA.
        :return: Массив значений SMA.
        """
        hl2 = (high_array + low_array) * 0.5

        if window <= 1:
            return hl2

        # Создаем массив для результатов с NaN
        sma = np.full_like(hl2, np.nan)

        # Рассчитываем кумулятивную сумму
        cumsum = np.cumsum(hl2)

        # Создаем сдвинутый кумулятивный массив
        shifted_cumsum = np.zeros_like(cumsum)
        shifted_cumsum[window:] = cumsum[:-window]

        # Вычисляем SMA для валидных периодов
        valid = slice(window - 1, None)
        sma[valid] = (cumsum[valid] - shifted_cumsum[valid]) / window

        return sma

    def generatePMax(self, var_array, close_array, high_array, low_array, atr_period, atr_multiplier):
        """
        Генерация PMax (Profit Maximizer).

        :param var_array: Массив значений скользящего среднего.
        :param close_array: Массив значений close.
        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param atr_period: Период для расчета ATR.
        :param atr_multiplier: Множитель ATR.
        :return: Массив значений PMax.
        """
        try:
            atr = self.generateAtr(high_array, low_array, close_array, period=atr_period)
        except Exception as exp:
            print('exception in atr:', str(exp), flush=True)
            return []

        previous_final_upperband = 0
        previous_final_lowerband = 0
        final_upperband = 0
        final_lowerband = 0
        previous_var = 0
        previous_pmax = 0
        pmax = []
        pmaxc = 0

        for i in range(0, len(close_array)):
            if np.isnan(close_array[i]):
                pass
            else:
                atrc = atr[i]
                varc = var_array[i]

                if math.isnan(atrc):
                    atrc = 0

                basic_upperband = varc + atr_multiplier * atrc
                basic_lowerband = varc - atr_multiplier * atrc

                if basic_upperband < previous_final_upperband or previous_var > previous_final_upperband:
                    final_upperband = basic_upperband
                else:
                    final_upperband = previous_final_upperband

                if basic_lowerband > previous_final_lowerband or previous_var < previous_final_lowerband:
                    final_lowerband = basic_lowerband
                else:
                    final_lowerband = previous_final_lowerband

                if previous_pmax == previous_final_upperband and varc <= final_upperband:
                    pmaxc = final_upperband
                else:
                    if previous_pmax == previous_final_upperband and varc >= final_upperband:
                        pmaxc = final_lowerband
                    else:
                        if previous_pmax == previous_final_lowerband and varc >= final_lowerband:
                            pmaxc = final_lowerband
                        elif previous_pmax == previous_final_lowerband and varc <= final_lowerband:
                            pmaxc = final_upperband

                pmax.append(pmaxc)

                previous_var = varc

                previous_final_upperband = final_upperband

                previous_final_lowerband = final_lowerband

                previous_pmax = pmaxc

        return pmax

    def generate_signals(self, df, moving_average_length=10, atr_period=10, atr_multiplier=3, average_type='SMA',
                        ama_params=None):
        """
        Генерация сигналов на основе SMA или AMA.

        :param df: DataFrame с данными.
        :param moving_average_length: Период скользящего среднего.
        :param atr_period: Период ATR.
        :param atr_multiplier: Множитель ATR.
        :param average_type: Тип скользящего среднего ('SMA' или 'AMA').
        :param ama_params: Параметры для AMA (если используется).
        :return: DataFrame с добавленными сигналами.
        """
        high_array = df["high"].values
        low_array = df["low"].values
        close_array = df["close"].values
        df = df.copy()

        if average_type == 'SMA':
            ma_arr = self.generateSma(high_array, low_array, moving_average_length)
        elif average_type == 'VAR':
            ma_arr = self.generateVar(high_array, low_array, moving_average_length)
        elif average_type == 'AMA':
            if ama_params is None:
                raise ValueError("Для AMA необходимо указать параметры ama_params.")
            ma_arr = self.generateAma(high_array, low_array, close_array, **ama_params)
        else:
            raise ValueError("Неподдерживаемый тип скользящего среднего.")

        pmax = self.generatePMax(ma_arr, close_array, high_array, low_array, atr_period, atr_multiplier)
        df["pmax"] = pmax
        df["ma"] = ma_arr
        df["buy_signal"] = (df["ma"] > df["pmax"]) & (df["ma"].shift(1) < df["pmax"].shift(1))
        df["sell_signal"] = (df["ma"] < df["pmax"]) & (df["ma"].shift(1) > df["pmax"].shift(1))

        return df

def prepare_regime_params(optuna_params):
    """
    Преобразует параметры из формата Optuna в два словаря: базовые параметры режимов и параметры расчета.

    Args:
        optuna_params (dict): Словарь с параметрами из Optuna

    Returns:
        dict: Словарь с двумя ключами: 'base_params' (параметры режимов) и 'calc_params' (остальные параметры)
    """
    # Инициализируем словари для базовых параметров и параметров расчета
    start_params = {}
    base_params = {}
    calc_params = {}

    # Сначала обрабатываем параметры режимов (0-4)

    start_params['moving_average_length'] = optuna_params.get('moving_average_length', 14)
    start_params['atr_period'] = optuna_params.get('atr_period', 10)
    for regime in range(5):
        regime_key = f'regime_{regime}_'
        regime_params = {}

        # Основные параметры режима
        regime_params['average_type'] = optuna_params.get(f'{regime_key}average_type', 'SMA')
        regime_params['moving_average_length'] = optuna_params.get(f'{regime_key}ma_length', 50)
        regime_params['atr_period'] = optuna_params.get(f'{regime_key}atr_period', 14)
        regime_params['atr_multiplier'] = optuna_params.get(f'{regime_key}atr_multiplier', 3.0)

        # Параметры AMA, если они есть
        ama_atr_period = optuna_params.get(f'{regime_key}ama_atr_period')
        ama_min_period = optuna_params.get(f'{regime_key}ama_min_period')
        ama_max_period = optuna_params.get(f'{regime_key}ama_max_period')

        if regime_params['average_type'] == 'AMA' and all(p is not None for p in [ama_atr_period, ama_min_period, ama_max_period]):
            regime_params['ama_params'] = {
                'atr_period': int(ama_atr_period),
                'min_period': int(ama_min_period),
                'max_period': int(ama_max_period)
            }

        base_params[regime] = regime_params

    # Теперь собираем все остальные параметры в calc_params
    other_params = [
        'rsi_length', 'use_smoothing', 'smoothing_length', 'smoothing_type',
        'alma_sigma', 'rsi_overbought', 'rsi_oversold', 'use_knn',
        'knn_neighbors', 'knn_lookback', 'knn_weight', 'feature_count',
        'use_filter', 'filter_method', 'filter_strength', 'sma_length',
        'ema_length', 'rsi_helbuth'
    ]

    for param in other_params:
        if param in optuna_params:
            calc_params[param] = optuna_params[param]

    return {
        'start_params': start_params,
        'base_params': base_params,
        'calc_params': calc_params
    }


class AdaptiveTradingSystem:
    def __init__(self, regime_params: Dict[int, dict]):
        self.regime_params = regime_params

    # ──────────────────────────────────────────────────────────────
    def generate_adaptive_signals(
        self,
        df: pd.DataFrame,
        regime_series: pd.Series
    ) -> pd.DataFrame:

        df      = df.reset_index(drop=True)
        regimes = regime_series.reset_index(drop=True).astype(int).values
        n       = len(df)
        high    = df['high'].values
        low     = df['low'].values
        close   = df['close'].values

        # 0) создаём «коллектор» – единая точка доступа ко всем MA/ATR
        collector = TinkoffHistoricalDataCollector()

        # ────────────────────────────── 1. PRECOMPUTE  ────────────
        ma_cache:  Dict[int, np.ndarray] = {}
        atr_cache: Dict[int, np.ndarray] = {}

        for regime, p in self.regime_params.items():
            atype = p['average_type']
            L     = p['moving_average_length']
            P     = p['atr_period']

            # ---- MA ------------------------------------------------
            if atype == 'SMA':
                # готовая реализация из collector
                ma = collector.generateSma(high, low, window=L)

            elif atype == 'VAR':
                ma = collector.generateVar(high, low, moving_average_length=L)

            elif atype == 'EMA':
                ma = collector.generateEma(high, low, moving_average_length=L)

            elif atype == 'AMA':
                ama_pars = p.get('ama_params',                   # защита от None
                                 {'atr_period': 14,
                                  'min_period': 5,
                                  'max_period': 50})

                ma = collector.generateAma(
                    high, low, close,
                    atr_period=ama_pars['atr_period'],
                    min_period=ama_pars['min_period'],
                    max_period=ama_pars['max_period']
                )
            else:
                raise ValueError(f"Unknown MA type {atype!r}")

            ma_cache[regime] = ma

            # ---- ATR ----------------------------------------------
            atr_cache[regime] = collector.generateAtr(
                high, low, close, period=P
            )

        # ────────────────────────────── 2. МЭРДЖ ПО РЕЖИМАМ ───────
        var_all = np.empty(n, dtype=np.float64)
        atr_all = np.empty(n, dtype=np.float64)
        mul_all = np.empty(n, dtype=np.float64)

        for regime, p in self.regime_params.items():
            mask          = (regimes == regime)
            var_all[mask] = ma_cache[regime][mask]
            atr_all[mask] = atr_cache[regime][mask]
            mul_all[mask] = p['atr_multiplier']

        # Заполняем возможные NaN в начале серии MA одной первой валидной точкой
        if np.isnan(var_all[0]):
            first_valid = var_all[~np.isnan(var_all)][0]
            var_all[np.isnan(var_all)] = first_valid

        # ────────────────────────────── 3. PMax STATE MACHINE ─────
        pmax_all = np.empty(n, dtype=np.float64)

        prev_var = var_all[0]
        prev_atr = atr_all[0]
        prev_mul = mul_all[0]

        prev_fu = prev_var + prev_mul * prev_atr
        prev_fl = prev_var - prev_mul * prev_atr
        prev_p  = prev_fl                                # стартовое состояние
        pmax_all[0] = prev_p

        for i in range(1, n):
            v   = var_all[i]
            a   = atr_all[i]
            m   = mul_all[i]

            bu  = v + m * a
            bl  = v - m * a

            fu = bu if (bu < prev_fu or prev_var > prev_fu) else prev_fu
            fl = bl if (bl > prev_fl or prev_var < prev_fl) else prev_fl

            if prev_p == prev_fu:
                p = fu if v <= fu else fl
            else:  # prev_p == prev_fl
                p = fl if v >= fl else fu

            pmax_all[i] = p

            prev_var, prev_fu, prev_fl, prev_p = v, fu, fl, p

        # ────────────────────────────── 4. СИГНАЛЫ ────────────────
        v_prev = np.concatenate(([var_all[0]], var_all[:-1]))
        p_prev = np.concatenate(([pmax_all[0]], pmax_all[:-1]))

        buy  = (v_prev < p_prev) & (var_all > pmax_all)
        sell = (v_prev > p_prev) & (var_all < pmax_all)

        # ────────────────────────────── 5. ВЫХОДНОЙ DataFrame ──────
        out = df.copy()
        out['ma']   = var_all
        out['pmax'] = pmax_all
        out['buy_signal']    = buy
        out['sell_signal']   = sell
        out['regime']        = regimes

        return out
def extract_features(df: pd.DataFrame, window: int = 126):
    """
    Вычисляет устойчивые признаки для кластеризации рыночных режимов.
    """

    def calculate_macd(df, macd_fast_periods=[12], macd_slow_periods=[26], macd_signal_periods=[9]):
        """
        Быстрый расчет нормализованного MACD с использованием векторизованных операций
        """
        close = df['close']

        # Создаем множества для уникальных периодов
        unique_fast = set(macd_fast_periods)
        unique_slow = set(macd_slow_periods)


        # Предварительно вычисляем все необходимые EMA и скользящие средние
        ema_cache = {}
        rolling_cache = {}

        # Кешируем быстрые EMA
        for fp in unique_fast:
            ema_cache[f'ema_{fp}'] = close.ewm(span=fp, adjust=False).mean()

        # Кешируем медленные EMA и скользящие средние
        for sp in unique_slow:
            ema_cache[f'ema_{sp}'] = close.ewm(span=sp, adjust=False).mean()
            rolling_cache[f'rolling_{sp}'] = close.rolling(window=sp).mean()

        # Основной цикл вычислений
        for fp in macd_fast_periods:
            ema_fast = ema_cache[f'ema_{fp}']
            for sp in macd_slow_periods:
                ema_slow = ema_cache[f'ema_{sp}']
                rolling_mean = rolling_cache[f'rolling_{sp}']

                # Вычисляем MACD и нормализацию
                macd = ema_fast - ema_slow
                macd_norm = macd / rolling_mean

                # Сохраняем MACD только один раз для комбинации fp/sp

                # Обрабатываем сигнальные периоды
                for sig in macd_signal_periods:
                    # Вычисляем сигнальную линию
                    signal = macd.ewm(span=sig, adjust=False).mean()
                    signal_norm = signal / rolling_mean

        return pd.DataFrame([macd_norm, signal_norm, macd_norm - signal_norm]).T.fillna(0)

    def calculate_atr(df, atr_window=14):
        """
        Расчет ATR и его сдвигов.
        """
        high = df['high']
        low = df['low']
        close = df['close']

        tr1 = high - low
        tr2 = np.abs(high - close.shift(1))
        tr3 = np.abs(low - close.shift(1))
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(atr_window).mean()

        return pd.Series(atr).fillna(0)

    def calculate_rsi(df, rsi_period=14):
        """
        Расчет RSI и его сдвиги.
        """
        close = df['close']
        delta = close.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(rsi_period).mean()
        avg_loss = loss.rolling(rsi_period).mean()
        rs = avg_gain / (avg_loss + 1e-10)
        rsi = 100 - (100 / (1 + rs))

        return pd.Series(rsi).fillna(0)

    def calculate_bollinger_bands(df, bollinger_window=20):
        """
        Расчет Bollinger Bands (ширины полос) и сдвигов.
        """
        close = df['close']
        ma = close.rolling(bollinger_window).mean()
        std = close.rolling(bollinger_window).std()
        bb_width = (2 * std) / ma

        return pd.Series(bb_width).fillna(0)

    def detect_market_regime(df: pd.DataFrame, window: int = 30, n_clusters: int = 3) -> pd.Series:
        """
        Классифицирует рыночную фазу на основе кластеризации признаков: волатильность, автокорреляция, наклон тренда.
        Возвращает метку режима рынка для каждого окна.
        """
        from sklearn.cluster import KMeans
        from sklearn.preprocessing import StandardScaler
        import pandas as pd
        import numpy as np

        features = []

        for i in range(len(df) - window + 1):
            window_df = df.iloc[i:i+window]
            close = window_df['close'].values

            # Волатильность (стандартное отклонение)
            volatility = np.std(np.diff(close))

            # Наклон тренда (регрессия по времени)
            x = np.arange(window)
            y = close
            slope = np.polyfit(x, y, deg=1)[0]

            # Автокорреляция лаг-1
            autocorr = np.corrcoef(close[:-1], close[1:])[0, 1]

            features.append([volatility, slope, autocorr])

        features = np.array(features)
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)

        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(features_scaled)

        # Расширим метки до длины df
        regime_series = pd.Series(np.nan, index=df.index)
        regime_series.iloc[window - 1:] = labels

        return regime_series.fillna(0).ffill().astype(int)
    macd_trend = calculate_macd(df, macd_slow_periods=[window], macd_fast_periods=[window//3],
                                 macd_signal_periods=[window//6])
    atr = calculate_atr(df, atr_window=window)
    rel_volatility = atr / df["close"]
    rsi_ind = calculate_rsi(df, rsi_period=window//2)
    volume_ratio = df['volume'].rolling(window).apply(
        lambda x: x[-1]/x.mean(), raw=True
    ).fillna(1).values

    features = np.column_stack([
        macd_trend,
        rel_volatility,
        rsi_ind,
        volume_ratio
    ])

    return features

class FastRollingMode:
    def __init__(self, window_size):
        self.window = deque(maxlen=window_size)
        self.counts = {}

    def update(self, new_val):
        if len(self.window) == self.window.maxlen:
            old_val = self.window.popleft()
            self.counts[old_val] -= 1
            if self.counts[old_val] == 0:
                del self.counts[old_val]

        self.window.append(new_val)
        self.counts[new_val] = self.counts.get(new_val, 0) + 1
        return max(self.counts.items(), key=lambda x: x[1])[0]


def find_best_trial_equal_importance(
    trials: List[FrozenTrial],
    directions: List[str],
    weights: Optional[List[float]] = None,
) -> Tuple[float, int, Dict, List[float], List[float]]:
    """
    Best trial selection for multi-objective Optuna study.
    See docstring above for full description.
    """

    if not trials:
        raise ValueError("Список trials пуст.")

    # 1. Оставляем только completed-трейлы с валидными (finite) значениями
    def _is_finite(vals):
        return all(math.isfinite(v) for v in vals)

    trials = [
        t for t in trials
        if (t.values is not None) and _is_finite(t.values)
    ]

    if not trials:
        raise ValueError("Нет трейлов с конечными values (без nan/inf).")

    n_obj = len(directions)
    if any(len(t.values) != n_obj for t in trials):
        raise ValueError("Не у всех trials одинаковое число objectives.")

    # 2. Веса
    if weights is None:
        weights = [1.0 / n_obj] * n_obj
    else:
        total = sum(weights)
        if not np.isclose(total, 1.0):
            weights = [w / total for w in weights]

    # 3. min / max по каждому objective
    all_vals = np.array([t.values for t in trials], dtype=float)
    mins = np.min(all_vals, axis=0)
    maxs = np.max(all_vals, axis=0)

    # 4. span и защита от деления на 0
    spans = np.where(maxs - mins == 0.0, 1.0, maxs - mins)

    best_score = -np.inf
    best_trial = None
    best_norm_values = []

    for t in trials:
        norms = []
        for i, (v, d) in enumerate(zip(t.values, directions)):
            if d == "minimize":
                norm = (maxs[i] - v) / spans[i]
            elif d == "maximize":
                norm = (v - mins[i]) / spans[i]
            else:
                raise ValueError(f"Непонятное направление {d!r} (i={i})")
            norms.append(norm)

        aggregated = float(np.dot(weights, norms))

        if aggregated > best_score:
            best_score = aggregated
            best_trial = t
            best_norm_values = norms

    # Если все трейлы «не дотянули» (best_score остался −inf)
    if best_trial is None:
        raise RuntimeError("Не удалось выбрать лучший trial.")

    return (
        best_score,
        best_trial.number,
        best_trial.params,
        list(best_trial.values),
        best_norm_values,
    )

def _whittaker_smooth(y: np.ndarray, lam: float = 50.0) -> np.ndarray:
    """
    Whittaker–Eilers smoothing (penalized least squares, D2).
    Линейный безфазовый фильтр, хорошо убирает шум, не смещая тренд.
    """
    import scipy.sparse as sp
    import scipy.sparse.linalg as spla

    n = len(y)
    if n <= 2:
        return y.copy()

    E = sp.eye(n, format="csc")
    # Вторая разность (D2): размер (n-2) x n
    diagonals = [np.ones(n), -2*np.ones(n), np.ones(n)]
    D2 = sp.diags(diagonals, [0, 1, 2], shape=(n-2, n), format="csc")
    coef = E + lam * (D2.T @ D2)
    z = spla.spsolve(coef, y.astype(float))
    return z

def _rearrange_preserving_marginal(z_raw: np.ndarray, z_smooth: np.ndarray) -> np.ndarray:
    """
    Distribution-preserving smoothing via monotone rearrangement:
    - сортируем исходный z_raw -> z_sorted,
    - берём порядок (argsort) сглаженного z_smooth,
    - раскладываем z_sorted по этому порядку.
    В итоге: гладко по времени (за счёт порядка z_smooth), но эмпирическое
    распределение строго совпадает с исходным (z_raw).
    """
    if len(z_raw) == 0:
        return z_raw
    order = np.argsort(z_smooth)
    z_sorted = np.sort(z_raw)
    out = np.empty_like(z_raw)
    out[order] = z_sorted
    return out

def calculate_smoothed_target_qnorm(
    df: pd.DataFrame,
    *,
    batch_start: int = 0,
    epsilon: float = 1e-6,
    round_decimals: int = 1,

    # базовая нормализация по окну (buy->sell)
    tight_spread_thr: float = 1e-4,

    # сглаживание в z-домене
    smooth_method: Literal["gauss", "savgol", "whittaker"] = "gauss",
    gauss_sigma: float = 2.0,
    savgol_window: int = 11,   # нечётное
    savgol_poly: int = 3,
    whittaker_lambda: float = 50.0,

    # квантильная нормализация и глобальный маппинг
    clip_z: float = 2.5,       # клип в z-подобной шкале перед [-1,1]
    tanh_scale: float | None = None,

    # джиттер перед ECDF
    dequant_jitter: float = 1e-4,

    # опция "обязательно растянуть каждый батч" (робастная эквализация по квантилям)
    per_batch_equalize: bool = False,
    per_batch_q: float = 0.01,  # напр. 1% и 99% -> [-1, 1]

    random_state: int | None = 42,
) -> pd.DataFrame:
    """
    Возвращает df с колонками:
      - normalized_target ∈ [-1, 1] (единая глобальная шкала)
      - batch (int64): идентификатор события (buy→sell)

    Алгоритм:
      1) Внутри каждого события: base(0..1) -> ECDF -> z_raw ~ N(0,1).
      2) Сглаживаем (Gaussian/Savitzky–Golay/Whittaker) -> z_smooth.
      3) Rearrangement: z_preserved = rearrange(z_raw, order-of z_smooth).
         Это убирает шум, но полностью сохраняет исходное распределение.
      4) Глобальная квантильная нормализация: приводим все батчи к общей
         маргинальной функции Q_global(p).
      5) Единая глобальная шкала (median/MAD), клип до [-clip_z, clip_z],
         линейный маппинг в [-1, 1], опционально tanh.
      6) (Опционально) per-batch эквализация по квантилям до [-1, 1].
    """
    rng = np.random.default_rng(random_state)
    df = df.copy()
    df["normalized_target"] = np.nan
    df["batch"] = np.nan
    df["event_sell_time"] = pd.to_datetime(df["event_sell_time"], utc=True)

    if savgol_window % 2 == 0:
        savgol_window += 1
    use_savgol = (savgol_window >= savgol_poly + 2)

    all_indices: list[np.ndarray] = []
    all_z_preserved: list[np.ndarray] = []
    batch = batch_start

    buy_rows = df.index[df["buy_signal"] == True]
    for start_i in buy_rows:
        sell_time = df.at[start_i, "event_sell_time"]
        if pd.isna(sell_time):
            continue
        sell_rows = df.index[df["time"] == sell_time]
        if len(sell_rows) == 0:
            continue
        end_i = sell_rows[0]

        mask = (df.index >= start_i) & (df.index <= end_i)
        idx = df.index[mask]
        if idx.empty:
            continue

        high_s = df.loc[idx, "high"]
        low_s  = df.loc[idx, "low"]

        # Робастные полки только для базовой формы (0..1)
        max_p = np.round(high_s.quantile(0.92), round_decimals)
        min_p = np.round(low_s .quantile(0.08), round_decimals)
        if max_p - min_p < tight_spread_thr:
            max_p, min_p = float(high_s.max()), float(low_s.min())

        use_profit_norm = (max_p - min_p) < tight_spread_thr

        # 1) base 0..1
        if not use_profit_norm:
            base = (df.loc[idx, "close"] - min_p) / (max_p - min_p + 1e-12)
        else:
            buy_price = df.at[start_i, "close"]
            max_prof = (high_s.max() - buy_price) / max(buy_price, 1e-12)
            max_prof = max(max_prof, epsilon)
            base = (df.loc[idx, "close"] - buy_price) / (buy_price * max_prof)
            base = 0.5 + 0.5 * base

        base = np.clip(base.to_numpy(float), epsilon, 1 - epsilon)

        # Разбиваем дубликаты
        if dequant_jitter and len(base) > 0:
            base = base + rng.normal(scale=dequant_jitter, size=base.shape)
            base = np.clip(base, epsilon, 1 - epsilon)

        # 2) ECDF -> z_raw ~ N(0,1)
        n = len(base)
        ranks = pd.Series(base, index=idx).rank(method="first").to_numpy()
        ecdf = (ranks - 0.5) / max(n, 1)
        ecdf = np.clip(ecdf, epsilon, 1 - epsilon)
        z_raw = norm.ppf(ecdf)

        # 3) Сглаживание (без фазовых сдвигов)
        if n > 2:
            if smooth_method == "gauss":
                z_sm = gaussian_filter1d(z_raw, sigma=gauss_sigma, mode="nearest")
            elif smooth_method == "savgol" and use_savgol and n >= savgol_window:
                z_sm = savgol_filter(z_raw, window_length=savgol_window, polyorder=savgol_poly, mode="interp")
            elif smooth_method == "whittaker":
                z_sm = _whittaker_smooth(z_raw, lam=whittaker_lambda)
            else:
                # запасной вариант, если окно слишком короткое
                z_sm = z_raw.copy()
        else:
            z_sm = z_raw.copy()

        # 4) Rearrangement: сохраняем распределение z_raw, но используем порядок z_sm
        z_preserved = _rearrange_preserving_marginal(z_raw, z_sm)
        #z_preserved = np.where(z_preserved > 0, z_preserved * 1.2, z_preserved)  # Усиливаем положительные пики
        #z_preserved = np.clip(z_preserved, -clip_z * 1.5, clip_z * 1.5)

        all_indices.append(idx.to_numpy())
        all_z_preserved.append(z_preserved)
        df.loc[idx, "batch"] = batch
        batch += 1

    # Если событий не нашлось
    if len(all_indices) == 0:
        df["batch"] = df["batch"].fillna(batch_start).astype("int64", errors="ignore")
        df["normalized_target"] = df["normalized_target"].fillna(0.0)
        return df

    # 5) Глобальная квантильная нормализация: одна общая маргинальная функция
    z_pool = np.concatenate(all_z_preserved, axis=0)
    z_pool_sorted = np.sort(z_pool)
    N = len(z_pool_sorted)
    # сетка перцентилей соответствующая отсортированным значениям
    p_pool = (np.arange(N) + 0.5) / N

    def q_global(p: np.ndarray) -> np.ndarray:
        p = np.clip(p, p_pool[0], p_pool[-1])
        return np.interp(p, p_pool, z_pool_sorted)

    all_z_qn: list[np.ndarray] = []
    for z_preserved in all_z_preserved:
        n = len(z_preserved)
        # перцентиль внутри батча
        p_batch = (pd.Series(z_preserved).rank(method="first").to_numpy() - 0.5) / max(n, 1)
        z_qn = q_global(p_batch)  # теперь у батча та же маргиналка, что и у пула
        all_z_qn.append(z_qn)

    # 6) Единая глобальная шкала -> [-1, 1]
    g_med = np.median(z_pool)
    g_mad = np.median(np.abs(z_pool - g_med)) + 1e-12
    scale = 1.4826 * g_mad

    pos = 0
    for idx, z_qn in zip(all_indices, all_z_qn):
        z_g = (z_qn - g_med) / scale
        y = np.clip(z_g, -clip_z, clip_z) / clip_z
        if tanh_scale:
            y = np.tanh(y * tanh_scale) / np.tanh(tanh_scale)

        # 7) (опционально) робастная per-batch эквализация до [-1,1]
        if per_batch_equalize and len(y) >= 3:
            q = per_batch_q
            lo, hi = np.quantile(y, [q, 1 - q])
            if hi - lo > 1e-12:
                y = (y - lo) / (hi - lo)  # [0..1]
                y = 2.0 * np.clip(y, 0.0, 1.0) - 1.0  # [-1..1]
            y = np.clip(y, -1.0, 1.0)

        df.loc[idx, "normalized_target"] = y
        pos += len(idx)

    df["batch"] = df["batch"].astype("int64", errors="ignore")
    return df
'''def calculate_smoothed_target(
    df: pd.DataFrame,
    *,
    batch_start: int = 0,
    epsilon: float = 1e-6,
    round_decimals: int = 1,

    # сглаживание
    ema_window: int = 8,
    z_ema_window: int = 15,
    gauss_sigma: float = 2.0,
    savgol_window: int = 11,     # должно быть нечётным
    savgol_poly: int = 3,

    # rob-winsor / рескейл
    winsor_pct: float = 0.01,
    clip_z: float = 2.5,
    tanh_scale: float | None = None,

    # jitter
    dequant_jitter: float = 1e-4,

    tight_spread_thr: float = 1e-4,
    random_state: int | None = 42,   # чтобы результат был детерминирован
) -> pd.DataFrame:
    """
    Плавный регрессионный таргет [-1 … 1] без «гребёнки» и выбросов.
    """
    rng = np.random.default_rng(random_state)
    df   = df.copy()
    df["normalized_target"] = np.nan
    df["batch"]             = np.nan
    df["event_sell_time"]   = pd.to_datetime(df["event_sell_time"], utc=True)

    batch = batch_start
    for _, buy_row in df[df["buy_signal"]].iterrows():
        sell_row = df[df["time"] == buy_row["event_sell_time"]]
        if sell_row.empty:
            continue

        start_i, end_i = buy_row.name, sell_row.index[0]
        mask = (df.index >= start_i) & (df.index <= end_i)

        high_s, low_s = df.loc[mask, "high"], df.loc[mask, "low"]

        max_p = np.round(high_s.quantile(0.92), round_decimals)
        min_p = np.round(low_s .quantile(0.08), round_decimals)
        if max_p - min_p < tight_spread_thr:
            max_p, min_p = high_s.max(), low_s.min()

        use_profit_norm = (max_p - min_p) < tight_spread_thr

        # ── 1. базовый 0…1 ────────────────────────────────────────────────
        if not use_profit_norm:
            base = (df.loc[mask, "close"] - min_p) / (max_p - min_p + 1e-12)
        else:
            buy_price = df.at[start_i, "close"]
            max_prof  = (high_s.max() - buy_price) / max(buy_price, 1e-12)
            max_prof  = max(max_prof, epsilon)
            base = (df.loc[mask, "close"] - buy_price) / (buy_price * max_prof)
            base = 0.5 + 0.5 * base

        base = np.clip(base, epsilon, 1 - epsilon)

        # ── 2. micro-jitter, чтобы разбить дубликаты ─────────────────────
        if dequant_jitter:
            base += rng.normal(scale=dequant_jitter, size=base.shape)
            base = np.clip(base, epsilon, 1 - epsilon)

        # ── 3. EMA по базе ───────────────────────────────────────────────
        if ema_window > 1:
            base = (
                pd.Series(base, index=df.index[mask])
                .ewm(alpha=2 / (ema_window + 1), adjust=False)
                .mean()
                .to_numpy()
            )
            base = np.clip(base, epsilon, 1 - epsilon)

        # ── 4. ECDF → z-score (unique ranks) ─────────────────────────────
        ranks = pd.Series(base).rank(method="first").to_numpy()
        ecdf  = (ranks - 0.5) / len(ranks)
        ecdf  = np.clip(ecdf, epsilon, 1 - epsilon)
        z     = norm.ppf(ecdf)

        # ── 5. EMA + Gaussian ────────────────────────────────────────────
        if z_ema_window > 1:
            z = (
                pd.Series(z, index=df.index[mask])
                .ewm(span=z_ema_window, adjust=False)
                .mean()
                .to_numpy()
            )

        if gauss_sigma and len(z) > 1:
            z = gaussian_filter1d(z, sigma=gauss_sigma, mode="nearest")

        # ── 6. Savitzky-Golay (убираем «зубцы») ──────────────────────────
        if len(z) >= savgol_window and savgol_window >= savgol_poly + 2:
            if savgol_window % 2 == 0:                   # делаем нечётным
                savgol_window += 1
            z = savgol_filter(z, window_length=savgol_window,
                              polyorder=savgol_poly, mode="interp")

        # ── 7. robust-winsor + рескейл ───────────────────────────────────
        med = np.median(z)
        mad = np.median(np.abs(z - med)) + 1e-12
        z_r = (z - med) / (1.4826 * mad)

        if winsor_pct:
            lo, hi = np.quantile(z_r, [winsor_pct, 1 - winsor_pct])
            z_r = np.clip(z_r, lo, hi)

        z_r = np.clip(z_r, -clip_z, clip_z)
        z_scaled = z_r / clip_z        # → [-1 … 1]

        if tanh_scale:
            z_scaled = np.tanh(z_scaled * tanh_scale) / np.tanh(tanh_scale)

        # ── 8. запись ────────────────────────────────────────────────────
        df.loc[mask, ["normalized_target", "batch"]] = np.column_stack(
            [z_scaled, np.full(mask.sum(), batch)]
        )
        batch += 1
    #df['batch'] = df['batch'].astype('int')

    return df'''



def calculate_smoothed_multi_target(
    df: pd.DataFrame,
    horizons: list[int] = [0, 5, 10, 20],  # Horizons (0 for original)
    future_window_size: int = 10,  # Fixed window size for future max/min calculation to reduce correlation
    *,
    batch_start: int = 0,
    epsilon: float = 1e-6,
    round_decimals: int = 1,
    ema_window: int = 8,
    z_ema_window: int = 15,
    gauss_sigma: float = 2.0,
    savgol_window: int = 11,
    savgol_poly: int = 3,
    winsor_pct: float = 0.01,
    clip_z: float = 2.5,
    tanh_scale: float | None = None,
    dequant_jitter: float = 1e-4,
    tight_spread_thr: float = 1e-4,
    random_state: int | None = 42,
) -> pd.DataFrame:
    rng = np.random.default_rng(random_state)
    df = df.copy()
    # Initialize columns
    tgt_cols = ['normalized_target'] if 0 in horizons else []
    tgt_cols += [f"multi_target_{h}" for h in horizons if h != 0]
    for col in tgt_cols:
        df[col] = np.nan
    df["batch"] = np.nan
    df["event_sell_time"] = pd.to_datetime(df["event_sell_time"], utc=True)

    batch = batch_start
    for _, buy_row in df[df["buy_signal"]].iterrows():
        sell_row = df[df["time"] == buy_row["event_sell_time"]]
        if sell_row.empty:
            continue

        start_i, end_i = buy_row.name, sell_row.index[0]
        mask = (df.index >= start_i) & (df.index <= end_i)
        batch_df = df.loc[mask].copy()  # Copy for local mods
        batch_len = len(batch_df)
        batch_indices = batch_df.index

        buy_price = batch_df.at[batch_indices[0], "close"]

        for h in horizons:
            tgt_col = 'normalized_target' if h == 0 else f"multi_target_{h}"

            if h >= batch_len + future_window_size:  # Way too short
                df.loc[mask, tgt_col] = 0.0  # Neutral
                continue

            # Compute per-bar base using individual future windows
            base = np.zeros(batch_len)
            for i in range(batch_len):
                window_start = i + h
                # For h=0, include current bar in the window
                if h == 0:
                    window_start = i
                window_end = min(window_start + future_window_size, batch_len)

                if window_start >= batch_len or window_end <= window_start:
                    # Empty future window: neutral or conservative
                    base[i] = 0.5
                    continue

                slice_df = batch_df.iloc[window_start:window_end]
                fut_high = slice_df['high'].max()
                fut_low = slice_df['low'].min()

                # Round for discretization
                fut_high = np.round(fut_high, round_decimals)
                fut_low = np.round(fut_low, round_decimals)

                spread = fut_high - fut_low
                use_profit_norm = spread < tight_spread_thr

                if not use_profit_norm:
                    base[i] = (batch_df['close'].iloc[i] - fut_low) / (spread + 1e-12)
                else:
                    # Profit-based normalization, relative to buy_price
                    max_prof = max((fut_high - buy_price) / max(buy_price, 1e-12), epsilon)
                    base[i] = (batch_df['close'].iloc[i] - buy_price) / (buy_price * max_prof)
                    base[i] = 0.5 + 0.5 * base[i]

                base[i] = np.clip(base[i], epsilon, 1 - epsilon)

            # Now apply all smoothing steps to the per-bar base series
            if dequant_jitter:
                base += rng.normal(scale=dequant_jitter, size=base.shape)
                base = np.clip(base, epsilon, 1 - epsilon)

            if ema_window > 1:
                base = pd.Series(base, index=batch_indices).ewm(alpha=2 / (ema_window + 1), adjust=False).mean().to_numpy()
                base = np.clip(base, epsilon, 1 - epsilon)

            ranks = pd.Series(base).rank(method="first").to_numpy()
            ecdf = (ranks - 0.5) / len(ranks)
            ecdf = np.clip(ecdf, epsilon, 1 - epsilon)
            z = norm.ppf(ecdf)

            if z_ema_window > 1:
                z = pd.Series(z, index=batch_indices).ewm(span=z_ema_window, adjust=False).mean().to_numpy()

            if gauss_sigma and len(z) > 1:
                z = gaussian_filter1d(z, sigma=gauss_sigma, mode="nearest")

            if len(z) >= savgol_window and savgol_window >= savgol_poly + 2:
                if savgol_window % 2 == 0:
                    savgol_window += 1
                z = savgol_filter(z, window_length=savgol_window, polyorder=savgol_poly, mode="interp")

            med = np.median(z)
            mad = np.median(np.abs(z - med)) + 1e-12
            z_r = (z - med) / (1.4826 * mad)

            if winsor_pct:
                lo, hi = np.quantile(z_r, [winsor_pct, 1 - winsor_pct])
                z_r = np.clip(z_r, lo, hi)

            z_r = np.clip(z_r, -clip_z, clip_z)
            z_scaled = z_r / clip_z

            if tanh_scale:
                z_scaled = np.tanh(z_scaled * tanh_scale) / np.tanh(tanh_scale)

            # Write to DF
            df.loc[mask, tgt_col] = z_scaled

        df.loc[mask, "batch"] = batch
        batch += 1

    return df


def deep_elbow(imp: np.ndarray, win: int = 5, eps: float = 0.02) -> int:
    """
    Берём окно длиной win, считаем средний относительный спад.
    Первое место, где спад < eps, считаем плато.
    """
    if len(imp) <= win:
        return len(imp)
    dif = np.abs(np.diff(imp) / (imp[:-1] + 1e-9))
    # скользящее среднее
    m = np.convolve(dif, np.ones(win) / win, mode="valid")
    flat = np.nonzero(m < eps)[0]
    return int(flat[0] + win) if flat.size else len(imp)

# ───────────────────────────────────────────────────────
# 2.  корреляционная чистка (быстрая, исправленная)
# ───────────────────────────────────────────────────────
def corr_prune(df: pd.DataFrame, feats: list[str], thr=.95) -> list[str]:
    if len(feats) < 2 or thr >= 1:
        return feats
    X  = df[feats].apply(pd.to_numeric, errors='ignore')
    C  = X.corr().abs().to_numpy()
    keep = []
    for i in range(len(feats)):
        if not keep or C[i, keep].max() < thr:
            keep.append(i)
    return [feats[i] for i in keep]

# ───────────────────────────────────────────────────────
# 3.  универсальный быстрый селектор
# ───────────────────────────────────────────────────────
def fast_feature_select(
        res           : pd.DataFrame,      # feature / importance
        df_full       : pd.DataFrame,      # датасет для corr-prune
        target_col    : str = "target",
        *,
        method        : str = "elbow",     # elbow | deep_elbow | percentile | quantile | top_k
        top_k         : int = 150,         # для method="top_k"
        perc_limit    : float = .90,       # для method="percentile"
        quantile_q    : float = .10,       # для method="quantile"
        elbow_eps     : float = .05,       # (> flat %) для (shallow) elbow
        deep_win      : int = 5,           # окно для deep_elbow
        deep_eps      : float = .02,       # порог для deep_elbow
        corr_thr      : float = .95        # корреляционный порог
) -> list[str]:

    ranked = res.sort_values("importance", ascending=False).reset_index(drop=True)
    feats  = ranked.feature.to_numpy()
    imps   = ranked.importance.to_numpy()

    # ---------- 1) сколько оставить  ----------
    if method == "elbow":                 # одношаговое колено
        k = np.argmax(np.abs(np.diff(imps) / (imps[:-1] + 1e-9)) < elbow_eps) + 1
        if k == 1:        # колено не найдено
            k = len(imps)
    elif method == "deep_elbow":
        k = deep_elbow(imps, win=deep_win, eps=deep_eps)
    elif method == "percentile":          # кумулятивная доля
        cum = np.cumsum(imps)
        k   = np.searchsorted(cum / cum[-1], perc_limit) + 1
    elif method == "quantile":
        thr = np.quantile(imps, 1 - quantile_q)
        k   = int((imps >= thr).sum())
    elif method == "top_k":
        k = min(top_k, len(feats))
    else:
        raise ValueError("unknown method")

    selected = feats[:k].tolist()

    # ---------- 2) корреляционная чистка ----------
    selected = corr_prune(
        df_full.drop(columns=[target_col], errors='ignore'),
        selected,
        thr=corr_thr
    )

    return selected
    
def patch_feature_timings(cls):
    """
    Оборачивает все методы cls, начинающиеся на _feat_,
    и складывает затраченное время в self._timings[method_name].
    Вызывать сразу после объявления класса.
    """
    def timed(func):
        def wrapper(self, *args, **kwargs):
            t0 = time.perf_counter()
            result = func(self, *args, **kwargs)
            dt = time.perf_counter() - t0
            # заводим словарь при первом же вызове
            if not hasattr(self, "_timings"):
                self._timings = {}
            self._timings[func.__name__] = dt
            return result
        return wrapper

    for name, method in inspect.getmembers(cls, inspect.isfunction):
        if name.startswith("_feat_"):             # ← только расчётные функции
            setattr(cls, name, timed(method))

    return cls

def _slope(y):
        x = np.arange(len(y))
        # линейная регрессия «по формуле»
        xm, ym = x.mean(), y.mean()
        beta = ((x - xm) * (y - ym)).sum() / ((x - xm)**2).sum()
        return beta

@njit
def _rolling_entropy_exact_numba(x, window):
    """ Точная реализация rolling entropy с bins='auto' для каждого окна. """
    n = len(x)
    res = np.empty(n, dtype=np.float32)
    res[:] = np.nan
    if window < 2:
        return res
    for end in range(window - 1, n):
        win = x[end - window + 1 : end + 1]
        a_min = np.min(win)
        a_max = np.max(win)
        if a_min == a_max:
            res[end] = 0.0
            continue
        sorted_win = np.sort(win)
        idx25 = int(0.25 * window)
        idx75 = int(0.75 * window)
        q25 = sorted_win[idx25]
        q75 = sorted_win[idx75]
        iqr = q75 - q25
        sturges = int(np.ceil(np.log2(window) + 1))
        if iqr > 0:
            bin_width = 2.0 * iqr / (window ** (1.0 / 3.0))
            fd = int(np.ceil((a_max - a_min) / bin_width))
        else:
            fd = 1
        nbins = max(sturges, fd, 1)
        edges = np.empty(nbins + 1, dtype=np.float32)
        step = (a_max - a_min) / nbins
        edges[0] = a_min
        for i in range(1, nbins):
            edges[i] = a_min + i * step
        edges[nbins] = a_max
        counts = np.zeros(nbins, dtype=np.int32)  # int32 достаточно для window<=1e9
        for val in win:
            idx = np.searchsorted(edges, val, side='right') - 1
            if 0 <= idx < nbins:
                counts[idx] += 1
        ent = 0.0
        total = float(window)
        for c in counts:
            if c > 0:
                p = c / total
                ent -= p * np.log(p + 1e-10)
        res[end] = ent
    return res

@jit(nopython=True)
def rolling_autocorr(arr, window):
    n = len(arr)
    result = np.full(n, 0.0)
    for i in range(n):
        start = max(0, i - window + 1)
        w = i - start + 1
        if w <= 1:
            continue
        s = arr[start: i + 1]
        a = s[1:]
        b = s[:-1]
        n_pts = w - 1
        mean_a = np.sum(a) / n_pts
        mean_b = np.sum(b) / n_pts
        cov = np.dot(a, b) / n_pts - mean_a * mean_b
        var_a = np.dot(a, a) / n_pts - mean_a * mean_a
        var_b = np.dot(b, b) / n_pts - mean_b * mean_b
        if var_a <= 0 or var_b <= 0:
            result[i] = np.nan
        else:
            std_a = np.sqrt(var_a)
            std_b = np.sqrt(var_b)
            result[i] = cov / (std_a * std_b)
    return result
   

# ------------------------------------------------------------------------------
@patch_feature_timings
class FeatureCalculatorForRegression:
    """
    df  -- исходный OHLCV-DataFrame.
    required_features -- список имён колонок, которые нужны модели.
    params -- { primitive_name: {... гиперпараметры ...}, 'stat_window': int }.
    """

    _PRIMITIVES = {
        "MEDPRICE":               "_feat_base",
        "MACD":                   "_feat_macd",
        "MACD_Hist":              "_feat_macd",
        "Overbought_Oversold":    "_feat_overbought",
        "Overbought_Oversold_Index_mean": "_feat_overbought",
        "Price_MADist%":          "_feat_madist",
        "Mean_Reversion":         "_feat_mean_reversion",
        "Fear_Greed":             "_feat_fear_greed",
        "perc_var_open_close":    "_feat_price_variation",
        "pmax_norm":              "_feat_pmax_ma",
        "ma_norm":                "_feat_pmax_ma",
        "ma_pmax_norm_rage":      "_feat_pmax_ma",
        "ma_pmax_norm_rage_pct":  "_feat_pmax_ma",
        "slope_trend":            "_feat_slope",
        "ema_trend":              "_feat_ema_trend",
        "hp_trend":               "_feat_hp_trend",
        "trade_bars_counter":     "_feat_trade_duration",
        "ROC":                    "_feat_roc",
        "ATR_norm":               "_feat_atr",
        "BB_Width":               "_feat_bb_width",
        "Asset_Growth":           "_feat_asset_growth",
        "ema_acceleration":       "_feat_ema_acceleration",
        "price_change":           "_feat_price_change",
        "Asset_To_Equity_Ratio":  "_feat_asset_to_equity_ratio",
        "volume_ratio":           "_feat_fear_greed_index",
        "WILLR":                  "_feat_willr",
        "kf_trend":               "_feat_kf_trend",
        "Fractal_Dim":            "_feat_fractal_dim",
        "Peak_Exhaustion_Score":  "_feat_peak_exhaustion",
        "%B_BB":                  "_feat_bb_percent",
        "Kurtosis_roll":          "_feat_kurtosis_roll",
        "OBV_div":                "_feat_obv_div",
        "RSI_slope":              "_feat_rsi_slope",
        "Vol_Decay":              "_feat_vol_decay",
        "Accel_Decay":            "_feat_accel_decay",
        "Entropy_roll":           "_feat_entropy_roll",
        "Wavelet_Var_Ratio":      "_feat_wavelet_var",
        "Autocorr_Lag1":          "_feat_autocorr",
        "Beta_Market":            "_feat_beta",
        "PSC":                    "_feat_peak_squeeze_curvature",
        "PSC_raw":                "_feat_peak_squeeze_curvature",
        "PSC_z":                  "_feat_peak_squeeze_curvature",
        "PSC_sigmoid":            "_feat_peak_squeeze_curvature",
    }

    # СТАРАЯ: r"^ago_(\d+)_"
    # НОВАЯ: умеет и "ago50_", и "ago_50_"
    _LAG_RE  = re.compile(r"^ago_?(\d+)_")
    _STAT_RE = re.compile(r"_(mean|min|max|std|skew|kurt|quantile(\d{2}))$")
    _LOGSF   = "_logsf"

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        f64 = self.df.select_dtypes("float64").columns
        self.df[f64] = self.df[f64].astype(np.float32)
        if "time" in self.df:
            ts = pd.to_datetime(self.df["time"], utc=True, errors="coerce")
            self.df["hour"]        = ts.dt.hour.astype("int8")
            self.df["day_of_week"] = ts.dt.day_of_week.astype("int8")

    def calculate_features(
        self,
        required_features: Iterable[str],
        params: Mapping[str, Mapping[str, Any]] | None = None
    ) -> pd.DataFrame:
        saved_cols = ['regime', 'normalized_target', 'batch', 'time', 'open', 'close', 'high', 'low', 'volume', 'buy_signal', 
                      'sell_signal', 'event_sell_time', 'event_sell_price', 'event_time', 'event_price', 'event_sell_time', 
                      'event_sell_price', 'target', 'pnl', 'ma', 'pmax']
        self._params      = defaultdict(dict, params or {})
        self._stat_window = self._params.get("stat_window", 50)
        for col in required_features:
            self._ensure_column(col)
        out = self.df[list(required_features)].copy()
        f64 = out.select_dtypes("float64").columns
        out[f64] = out[f64].astype(np.float32)

        for mandatory_col in saved_cols:
            if mandatory_col in self.df.columns:
                out[mandatory_col] = self.df[mandatory_col]
            
        return out

    def calculate_all_possible_features(self, params: Mapping[str, Mapping[str, Any]] | None = None) -> pd.DataFrame:
        """
        Вычисляет все возможные фичи, исключая lag-версии для указанных колонок.
        Все бесконечные значения (np.inf/-np.inf) заменяются на 0.
        Порядок вычислений:
        1. Все базовые примитивы
        2. Lag-версии фич (кроме исключенных)
        3. Статистики для всех фич
        """
        # Инициализация параметров
        if not hasattr(self, '_params'):
            self._params = defaultdict(dict, params or {})
        self._stat_window = self._params.get("stat_window", 50)
        
        # Колонки, для которых не нужно создавать lag-версии
        EXCLUDE_FROM_LAGS = {
            'time', 'open', 'close', 'high', 'low', 'volume', 
            'ma', 'pmax', 'buy_signal', 'sell_signal', 'regime',
            'event_time', 'event_price', 'event_sell_time', 
            'event_sell_price', 'pnl', 'target', 'normalized_target',
            'batch', 'hour', 'day_of_week', 'trade_bars_counter'
        }
        
        # 1. Вычисляем все базовые примитивы
        all_primitives = list(self._PRIMITIVES.keys())
        for primitive in all_primitives:
            method_name = self._PRIMITIVES[primitive]
            primitive_params = self._params.get(primitive, {})
            try:
                getattr(self, method_name)(**primitive_params)
            except Exception as e:
                print(f"Ошибка при вычислении примитива {primitive}: {str(e)}")
    
        # 2. Добавляем lag-версии только для разрешенных фич
        numeric_cols = [
            col for col in self.df.select_dtypes(include=['float32', 'float64', 'int32', 'int64']).columns
            if col not in EXCLUDE_FROM_LAGS and  # Исключаем указанные колонки
            not self._LAG_RE.match(col) and      # Исключаем уже lag-фичи
            not col.endswith(self._LOGSF) and    # Исключаем logsf-фичи
            not self._STAT_RE.search(col)        # Исключаем статистики
        ]
        
        lag_periods = [1, 2, 3, 5, 10, 20, 50]  # Стандартные лаги
        
        for col in numeric_cols:
            for lag in lag_periods:
                lag_col = f"ago_{lag}_{col}"
                if lag_col not in self.df.columns:
                    self.df[lag_col] = self.df[col].shift(lag)
        
        # 3. Добавляем статистики для всех фич (кроме исключенных)
        all_cols_for_stats = [
            col for col in self.df.columns 
            if col not in EXCLUDE_FROM_LAGS and
            not col.endswith(self._LOGSF) and
            not self._STAT_RE.search(col)
        ]
        
        stats = ['mean', 'std', 'min', 'max', 'skew', 'kurt']
        
        for col in all_cols_for_stats:
            for stat in stats:
                stat_col = f"{col}_{stat}"
                if stat_col not in self.df.columns:
                    try:
                        self._add_stat(col, stat)
                    except Exception as e:
                        print(f"Ошибка при вычислении статистики {stat} для {col}: {str(e)}")
        
        # 4. Добавляем logsf-версии только для разрешенных фич
        main_cols_for_logsf = [
            col for col in numeric_cols 
            if not col.startswith('ago_') and
            not col.endswith(self._LOGSF) and
            col not in EXCLUDE_FROM_LAGS
        ]
        
        for col in main_cols_for_logsf:
            logsf_col = f"{col}{self._LOGSF}"
            if logsf_col not in self.df.columns:
                try:
                    self.df[logsf_col] = norm.logsf(self.df[col])
                except Exception as e:
                    print(f"Ошибка при вычислении logsf для {col}: {str(e)}")
    
        # 5. Заменяем бесконечные значения на 0
        numeric_cols_all = self.df.select_dtypes(include=['float32', 'float64', 'int32', 'int64']).columns
        self.df[numeric_cols_all] = self.df[numeric_cols_all].replace([np.inf, -np.inf], 0)
        
        # Сохраняем все оригинальные колонки
        for col in EXCLUDE_FROM_LAGS:
            if col in self.df.columns and col not in self.df:
                self.df[col] = self.df[col]
        
        return self.df.copy()

    def _ensure_column(self, name: str):
        if name in self.df:
            return

        # 1) lag-префикс "ago50_" или "ago_50_"
        m = self._LAG_RE.match(name)
        if m:
            lag  = int(m.group(1))
            base = name[m.end():]
            self._ensure_column(base)
            self.df[name] = self.df[base].shift(lag)
            return

        # 2) _logsf
        if name.endswith(self._LOGSF):
            base = name[:-len(self._LOGSF)]
            self._ensure_column(base)
            self.df[name] = norm.logsf(self.df[base])
            return

        # 3) статистический суффикс
        m = self._STAT_RE.search(name)
        if m:
            stat = m.group(1)
            base = name[:m.start()]
            self._ensure_column(base)
            self._add_stat(base, stat)
            return

        # 4) примитив
        prim = name
        if prim.startswith("Overbought_Oversold"):
            prim = "Overbought_Oversold"
        if prim.startswith("Fear_Greed"):
            prim = "Fear_Greed"
        if prim not in self._PRIMITIVES:
            raise KeyError(f"Не знаю, как получить примитив «{prim}» для «{name}»")
        getattr(self, self._PRIMITIVES[prim])(**self._params.get(prim, {}))
        if name not in self.df:
            raise RuntimeError(f"После _feat_{prim}() нет колонки «{name}»")

    def _add_stat(self, base: str, stat: str):
        col = f"{base}_{stat}"
        if col in self.df:
            return
        s = self.df[base]; w = self._stat_window
        if stat == "mean":
            self.df[col] = s.rolling(w).mean()
        elif stat == "std":
            self.df[col] = s.rolling(w).std()
        elif stat == "min":
            self.df[col] = s.rolling(w).min()
        elif stat == "max":
            self.df[col] = s.rolling(w).max()
        elif stat == "skew":
            self.df[col] = s.rolling(w).skew()
        elif stat == "kurt":
            self.df[col] = s.rolling(w).kurt()
        elif stat.startswith("quantile"):
            q = int(stat[-2:]) / 100
            self.df[col] = s.rolling(w).quantile(q)
        else:
            raise ValueError(f"Неизвестная stat «{stat}»")

    # ---------------------- ПРИМИТИВЫ ----------------------

    def _feat_base(self, medprice: int = 50):
        if "MEDPRICE" in self.df:
            return
        self.df["MEDPRICE"]      = (self.df["high"] + self.df["low"]) / 2
        self.df["MEDPRICE_std"] = self.df["MEDPRICE"].rolling(medprice).std()

    def _feat_macd(self, fast: int = 12, slow: int = 26, signal: int = 9):
        """
        Быстрый расчет нормализованного MACD с использованием векторизованных операций
        """
        if {"MACD","MACD_Hist"}.issubset(self.df.columns):
            return
            
        close = self.df['close']
        # Создаем множества для уникальных периодов
        ema_cache_fp = close.ewm(span=fast, adjust=False).mean()
        
        ema_cache_sp = close.ewm(span=slow, adjust=False).mean()
        rolling_cache = close.rolling(window=slow).mean()
        
        # Основной цикл вычислений
        ema_fast = ema_cache_fp
        ema_slow = ema_cache_sp
        rolling_mean = rolling_cache
        macd = ema_fast - ema_slow
        macd_norm = macd / rolling_mean
        self.df[f'MACD'] = macd_norm
        signal = macd.ewm(span=signal, adjust=False).mean()
        signal_norm = signal / rolling_mean
    
        # Сохраняем результаты
        self.df[f'MACD_Hist'] = macd_norm - signal_norm

    '''def _feat_macd(self, fast: int = 12, slow: int = 26, signal: int = 9):
        if {"MACD","MACD_Hist"}.issubset(self.df.columns):
            return
        c  = self.df["close"]
        ef = c.ewm(span=fast, adjust=False).mean()
        es = c.ewm(span=slow, adjust=False).mean()
        macd = (ef - es) / (c.rolling(slow).mean().add(1e-10))
        sig  = macd.ewm(span=signal, adjust=False).mean()
        self.df["MACD"]      = macd
        self.df["MACD_Hist"] = macd - sig'''

    def _feat_overbought(self, rsi_p: int = 14, stoch_p: int = 14):
        name = "Overbought_Oversold_Index"
        if name in self.df:
            return
        c   = self.df["close"]; d = c.diff()
        g   = d.clip(lower=0); l = (-d).clip(lower=0)
        rs  = g.rolling(rsi_p).mean() / (l.rolling(rsi_p).mean().add(1e-10))
        rsi = 100 - (100 / (1 + rs))
        lo  = self.df["low"].rolling(stoch_p).min()
        hi  = self.df["high"].rolling(stoch_p).max()
        st  = 100*(c - lo)/(hi - lo + 1e-10)
        self.df[name] = (rsi + st)/2

    def _feat_madist(self, span_lenght: int = 200):
        name = "Price_MADist%"
        if name in self.df:
            return
        ema = self.df["close"].ewm(span=span_lenght, adjust=False).mean()
        self.df[name] = (self.df["close"]/ema - 1)*100

    def _feat_mean_reversion(self, window: int = 20):
        name = "Mean_Reversion"
        if name in self.df:
            return
        ma = self.df["close"].rolling(window).mean()
        self.df[name] = self.df["close"] - ma

    def _feat_fear_greed(self, window: int = 14):
        name = "Fear_Greed_Index"
        if name in self.df:
            return
        v  = self.df["close"].pct_change().rolling(window).std()
        vc = self.df["volume"].pct_change().rolling(window).mean()
        tr = self.df["close"]/self.df["close"].rolling(window).mean()
        self.df[name] = (v + vc + tr)/3*100

    def _feat_price_variation(self):
        name = "perc_var_open_close"
        if name in self.df:
            return
        eps = 1e-10
        self.df[name] = (self.df["close"]-self.df["open"])/(self.df["open"]+eps)*100

    def _feat_pmax_ma(self,
        pmax_ma_length: int = 10,
        pmax_ma_length_roll: int = 50,
        pct_window: int = 5
    ):
        need = {
            "pmax_norm", "ma_norm",
            "ma_pmax_norm_rage", "ma_pmax_norm_rage_pct"
        }
        if need.issubset(self.df.columns):
            return
        if {"pmax","ma"}.difference(self.df.columns):
            raise ValueError("Нужны 'pmax' и 'ma'")
        c = self.df["close"]
        self.df["pmax_norm"]             = (c-self.df["pmax"])/self.df["pmax"]
        self.df["ma_norm"]               = (c-self.df["ma"])/self.df["ma"]
        self.df["ma_pmax_norm_rage"]     = self.df["ma_norm"] - self.df["pmax_norm"]
        # новый примитив — pct-динамика
        self.df["ma_pmax_norm_rage_pct"] = \
          self.df["ma_pmax_norm_rage"].pct_change(pct_window).fillna(0)

    def _feat_slope(self, slope_lag: int = 300, pct_window: int = 6):
        name = "slope_trend"
        if name in self.df:
            return
        r = self.df["close"].pct_change(pct_window).fillna(0)
        self.df[name] = r.rolling(slope_lag, min_periods=slope_lag)\
                         .apply(_slope, raw=True)

    def _feat_ema_trend(self, span: int = 300, pct_window: int = 6):
        name = "ema_trend"
        if name in self.df:
            return
        r = self.df["close"].pct_change(pct_window).fillna(0)
        e = r.ewm(span=span, adjust=False).mean()
        self.df[name] = e.diff().fillna(0)

    def _feat_asset_to_equity_ratio(self):
        """
        Вычисление коэффициента соотношения активов и собственного капитала.
        """
        name = "Asset_To_Equity_Ratio"
        asset = self.df['close']
        equity = self.df['low']
        # Добавляем в DataFrame
        self.df[name] = asset / (equity + 1e-10)

    def _feat_hp_trend(self, lamb: float = 1600):
        name = "hp_trend"
        if name in self.df:
            return
        y    = np.log(self.df["close"]).fillna(method="ffill")
        coef = lamb/(1+lamb)
        tr   = np.empty(len(y), dtype=float)
        tr[0] = y.iloc[0]
        for i in range(1, len(y)):
            tr[i] = coef*y.iloc[i] + (1-coef)*tr[i-1]
        self.df[name] = np.append([0], np.diff(tr))

    def _feat_kf_trend(self,
        pct_window: int = 6,
        obs_var: float = 1e-4, # σ² ε_t (шум наблюдения)
        level_var: float = 1e-5 # σ² η_t (шум уровня)
        ) -> pd.DataFrame:
        """
        Добавляет к DataFrame колонки:
        kf_trend — one-sided Калман-оценка тренда доходностей
        kf_trend_logsf — лог-survival-function (z-score) тренда
        Полностью каузально, обновляется тик-за-тиком.
        """

        name = 'kf_trend'
        # 1. Доходности
        r = self.df['close'].pct_change(pct_window).fillna(0)
        # 2. Local-level модель: y_t = μ_t + ε_t ;  μ_t = μ_{t-1} + η_t
        mod = sm.tsa.UnobservedComponents(r, level='llevel')
        
        # 3. Параметры модели в log-шкале (требование statsmodels)
        params = np.log([obs_var, level_var])
        
        # 4. Только forward-filter → нет look-ahead bias
        res = mod.filter(params)                       # <— односторонний Калман
        trend = pd.Series(res.filtered_state[0], index=self.df.index)
        
        # 5. Запись результата
        self.df['kf_trend'] = trend

    def _feat_willr(self, window=14):
        """
        Вычисление %R по методу Уильямса (WILLR).
        """
        name = 'WILLR'
        high = self.df['high']
        low = self.df['low']
        close = self.df['close']

        highest_high = high.rolling(window).max()
        lowest_low = low.rolling(window).min()
        
        willr = ((highest_high - close) / (highest_high - lowest_low)) * -100
        
        # Добавляем в DataFrame
        self.df[name] = willr

    def _feat_fear_greed_index(self, window: int = 14):
        """
        Расчет объема как отношение последнего объема к скользящему среднему.
        """
        name = "volume_ratio"
        if name in self.df:
            return
        s = self.df["volume"]
        self.df[name] = s / s.rolling(window).mean()

    def _feat_trade_duration(self):
        name = "trade_bars_counter"
        if name in self.df:
            return
        self.df[name] = np.nan
        entries = self.df.index[self.df["event_time"].notna()]
        last    = self.df.index[-1]
        for st in entries:
            sell = self.df.at[st, "event_sell_time"]
            ends = self.df.index[self.df["time"] == sell]
            end  = ends[0] if len(ends) else last
            s,e  = self.df.index.get_loc(st), self.df.index.get_loc(end)
            self.df.loc[self.df.index[s:e+1], name] = np.arange(e-s+1, dtype=np.float32)

    def _feat_roc(self, window: int = 5):
        name = "ROC"
        if name in self.df:
            return
        self.df[name] = self.df["close"].pct_change(window)

    def _feat_atr(self, atr_window: int = 14):
        name = "ATR_norm"
        if name in self.df:
            return
        h,l,c = self.df["high"], self.df["low"], self.df["close"]
        tr1 = h-l
        tr2 = (h-c.shift()).abs()
        tr3 = (l-c.shift()).abs()
        tr  = pd.concat([tr1,tr2,tr3], axis=1).max(axis=1)
        atr = tr.rolling(atr_window).mean()
        self.df[name] = atr/c

    def _feat_bb_width(self, bb_window: int = 20):
        name = "BB_Width"
        if name in self.df:
            return
        c   = self.df["close"]
        ma  = c.rolling(bb_window).mean()
        std = c.rolling(bb_window).std()
        self.df[name] = 2*std/ma

    def _feat_asset_growth(self, window: int = 3):
        name = "Asset_Growth"
        if name in self.df:
            return
        self.df[name] = self.df["close"].pct_change(window).fillna(0)*100

    def _feat_ema_acceleration(self, pct_window: int = 3, ema_window: int = 300):
        name = "ema_acceleration"
        if name in self.df:
            return
        r = self.df["close"].pct_change(pct_window).fillna(0)
        e = r.ewm(span=ema_window).mean()
        self.df[name] = e.diff(4)

    def _feat_price_change(self, window: int = 1):
        name = "price_change"
        if name in self.df:
            return
        self.df[name] = self.df["close"].pct_change(window).fillna(0)
        
    def _feat_peak_exhaustion(
        self,
        price_win: int = 60,    # окно "локального максимума"
        mom_win:   int = 10,    # окно для momentum
        vol_win:   int = 20,
        atr_win:   int = 14,
        z_win:     int = 100    # z-score нормализация
    ):
        """
        Peak-Exhaustion Score  ~ 0…1
        1 → почти наверху, импульс затух, объём падает, ATR высок.
        """
        name = "Peak_Exhaustion_Score"
        c = self.df["close"]
    
        # 1) расстояние до локального max
        roll_max = c.rolling(price_win).max()
        dist_max = (roll_max - c) / roll_max          # 0 — на max, >0 — ниже
    
        # 2) ослабевающий импульс
        roc_now  = c.pct_change(mom_win)
        roc_hist = roc_now.rolling(price_win).max()   # max импульса в окне
        momentum_div = 1 - (roc_now / (roc_hist + 1e-12))   # 0 → свежий high
    
        # 3) сушащийся объём
        vol_ratio = self.df["volume"] / \
            self.df["volume"].rolling(vol_win).mean()
    
        # 4) расширенный спред (ATR/price)
        tr  = pd.concat([
                self.df["high"]  - self.df["low"],
                (self.df["high"] - c.shift()).abs(),
                (self.df["low"]  - c.shift()).abs()
            ], axis=1).max(axis=1)
        atr = tr.rolling(atr_win).mean()
        atr_norm = atr / c
    
        # 5) агрегируем, переводим в z-score, squash σ → 0…1
        raw = (dist_max + momentum_div + (1/vol_ratio) + atr_norm) / 4
        z   = (raw - raw.rolling(z_win).mean()) / (raw.rolling(z_win).std() + 1e-9)
        self.df[name] = 1 / (1 + np.exp(-z))   # σ(z)
        
    def _feat_fractal_dim(self, short_win=20, long_win=40):
        """Вычисляет фрактальную размерность на основе отношения ATR разных периодов"""
        name = "Fractal_Dim"
        if name in self.df:
            return
        
        # Вычисляем ATR для короткого периода
        h, l, c = self.df['high'], self.df['low'], self.df['close']
        tr1 = h - l
        tr2 = (h - c.shift()).abs()
        tr3 = (l - c.shift()).abs()
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr_short = tr.rolling(short_win).mean()
        
        # Вычисляем ATR для длинного периода
        atr_long = tr.rolling(long_win).mean()
        
        # Вычисляем фрактальную размерность
        ratio = atr_long / (atr_short + 1e-10)  # Добавляем небольшое значение для избежания деления на 0
        self.df[name] = np.log(ratio) / np.log(2)
        
    def _feat_bb_percent(self, window=20, std_mult=2):
        name = "%B_BB"
        if name in self.df: return
        ma = self.df["close"].rolling(window).mean()
        std = self.df["close"].rolling(window).std()
        self.df[name] = (self.df["close"] - (ma - std_mult * std)) / (4 * std)
        
    def _feat_kurtosis_roll(self, window=50):
        name = "Kurtosis_roll"
        if name in self.df: return
        ret = self.df["close"].pct_change().fillna(0)
        self.df[name] = ret.rolling(window).kurt()
        
    def _feat_obv_div(self, window=10):
        name = "OBV_div"
        if name in self.df: return
        sign = np.sign(self.df["close"].diff())
        obv = (sign * self.df["volume"]).cumsum()
        price_chg = self.df["close"].pct_change(window)
        obv_chg = obv.pct_change(window)
        self.df[name] = price_chg - obv_chg
        
    def _feat_rsi_slope(self, rsi_p=14, diff_win=5):
        name = "RSI_slope"
        if name in self.df: return
        delta = self.df["close"].diff()
        gain = delta.clip(lower=0).rolling(rsi_p).mean()
        loss = -delta.clip(lower=0).rolling(rsi_p).mean()
        rsi = 100 - 100 / (1 + gain / (loss + 1e-10))
        self.df[name] = rsi.diff(diff_win)
        
    def _feat_vol_decay(self, window=20):
        name = "Vol_Decay"
        if name in self.df: return
        vol_ema = self.df["volume"].ewm(span=window).mean()
        self.df[name] = self.df["volume"] / vol_ema - 1
        
    def _feat_accel_decay(self, window=10):
        name = "Accel_Decay"
        if name in self.df: return
        vel = self.df["close"].diff(window)
        accel = vel.diff(window)
        self.df[name] = accel / (vel.abs() + 1e-10)
        
    '''def _feat_entropy_roll(self, window=50):
        name = "Entropy_roll"
        if name in self.df: 
            return
            
        ret = self.df["close"].pct_change().fillna(0)
        
        def _entropy(x):
            if len(x) < 2:
                return 0
            hist = np.histogram(x, bins='auto')[0]
            hist = hist / hist.sum()  # Нормализуем
            return -np.sum(hist * np.log(hist + 1e-10))
    
        self.df[name] = ret.rolling(window).apply(_entropy, raw=True)'''

    def _feat_entropy_roll(self, window: int = 50):
        name = "Entropy_roll"
        if name in self.df:
            return
        ret = self.df["close"].pct_change().fillna(0.0).to_numpy(dtype=np.float32)
        ent = _rolling_entropy_exact_numba(ret, window)
        self.df[name] = ent
    
    def _feat_wavelet_var(self, short_win=10, long_win=50):
        name = "Wavelet_Var_Ratio"
        if name in self.df:
            return
        ret = self.df["close"].pct_change().fillna(0)
        var_short = ret.rolling(short_win).var()
        var_long = ret.rolling(long_win).var()
        self.df[name] = var_short / (var_long + 1e-10)
        
    '''def _feat_autocorr(self, window=50):
        name = "Autocorr_Lag1"
        if name in self.df:
            return
        ret = self.df["close"].pct_change().fillna(0)
        self.df[name] = ret.rolling(window).apply(lambda x: x.autocorr(lag=1) if len(x) > 1 else 0, raw=False)'''

    def _feat_autocorr(self, window=50):
        name = "Autocorr_Lag1"
        if name in self.df:
            return
        ret = self.df["close"].pct_change().fillna(0)
        arr = ret.to_numpy()  # Use to_numpy() for compatibility
        autocorrs = rolling_autocorr(arr, window)
        self.df[name] = autocorrs
        
    def _feat_beta(self, window=50):
        name = "Beta_Market"
        if name in self.df or "market_close" not in self.df:
            return
        ret_stock = self.df["close"].pct_change().fillna(0)
        ret_market = self.df["market_close"].pct_change().fillna(0)
        cov = ret_stock.rolling(window).cov(ret_market)
        var_market = ret_market.rolling(window).var()
        self.df[name] = cov / (var_market + 1e-10)
        
    def _feat_peak_squeeze_curvature(self,
                                vel_win: int = 5,
                                acc_win: int = 5,
                                vol_win: int = 20,
                                atr_win: int = 14,
                                z_win : int = 60):
        """
        Возвращает 3 колонки:
        PSC_raw, PSC_z, PSC_sigmoid ∈ [0,1]
        """
        name = "PSC"
        cols_need = {"PSC_raw","PSC_z","PSC_sigmoid"}
        if cols_need.issubset(self.df.columns): return
        c = self.df['close']
    
        # 1) speed & accel
        speed  = c.pct_change(vel_win).fillna(0)
        accel  = speed.diff(acc_win).fillna(0)
        curvature = accel / (speed.abs() + 1e-10)
    
        # 2) squeeze = ATR_norm ↘ & HV_norm ↘
        h,l = self.df['high'], self.df['low']
        tr = pd.concat([h-l, (h-c.shift()).abs(), (l-c.shift()).abs()], axis=1).max(axis=1)
        atr = tr.rolling(atr_win).mean()
        hv  = c.pct_change().rolling(vol_win).std()
        squeeze = - (atr / (c+1e-10)).diff().clip(upper=0)   # падение ATR
        squeeze += - hv.diff().clip(upper=0)                 # падение HV
        squeeze /= 2
    
        # 3) агрегируем
        raw = 0.6*curvature + 0.4*squeeze
    
        # 4) z-score + σ(z)
        mu  = raw.rolling(z_win).mean()
        std = raw.rolling(z_win).std()
        z = (raw - mu)/(std + 1e-9)
        sigm = 1/(1+np.exp(-z))
    
        self.df[f'{name}_raw']     = raw
        self.df[f'{name}_z']       = z.clip(-5, 5)
        self.df[f'{name}_sigmoid'] = sigm


def calculate_metrics(test_data, y_test, y_pred, target_column='normalized_target'):
    """
    Функция для расчета метрик по данным теста и предсказаниям модели.

    Параметры:
    - test_data: pd.DataFrame — тестовые данные с колонками batch, high, close и другими.
    - y_test: pd.Series или np.array — фактические значения целевой переменной.
    - y_pred: pd.Series или np.array — предсказанные моделью значения.
    - target_column: str — название колонки целевой переменной в test_data.

    Возвращает:
    - avg_mse: float — среднеквадратическая ошибка.
    - avg_r2: float — средняя R².
    - std_r2: float — стандартное отклонение R².
    - corr_mean: float — средняя корреляция.
    - corr_std: float — стандартное отклонение корреляции.
    - avg_missed: float — средний процент упущенной прибыли.
    """
    # Инициализация метрик
    mse_scores, r2_scores, corr_scores, missed_pnl = [], [], [], []

    # Расчет корреляции целевой переменной и предсказаний
    y_pred_series = pd.Series(y_pred, index=y_test.index)
    corr_score = test_data[target_column].corr(y_pred_series)
    corr_scores.append(corr_score)

    # MSE и R2
    mse_scores.append(mean_squared_error(y_test, y_pred))
    r2_scores.append(r2_score(y_test, y_pred))

    # Расчет missed_pnl для каждого batch
    """for batch in test_data['batch'].unique():
        mask = test_data['batch'] == batch
        max_high = test_data.loc[mask, 'high'].max()
        pred = y_pred[mask]  # предполагается, что y_pred соответствует normalized_target
        sell_idx = np.argmin(pred)  # продажа на минимальном предсказанном значении
        sell_price = test_data.loc[mask].iloc[sell_idx]['close']
        missed = (max_high - sell_price) / (max_high - test_data.loc[mask].iloc[0]['close'])  # % упущенной прибыли
        missed_pnl.append(missed)"""

    # Усреднение метрик
    avg_mse = float(np.mean(mse_scores))
    avg_r2 = float(np.mean(r2_scores))
    std_r2 = float(np.std(r2_scores))
    corr_mean = float(np.mean(corr_scores))
    corr_std = float(np.std(corr_scores))
    #avg_missed = float(np.mean(missed_pnl))

    # Проверка на корректность результатов
    if np.isfinite([avg_mse, avg_r2, std_r2, corr_mean, corr_std]).all(): #avg_missed
        return avg_mse, avg_r2, std_r2, corr_mean, corr_std, #avg_missed
    else:
        return float('inf'), float('inf'), float('inf'), float('inf'), float('inf'), float('inf')

def to_dense(X):
    """Преобразует разреженную матрицу в плотную."""
    if issparse(X):
        return X.toarray()
    return X
    
def prepare_data(df, target_col):
    """
    Подготавливает данные: разделяет на числовые и категориальные признаки, создает конвейер преобразования.
    """
    if type(target_col) == str:
        df.dropna(inplace=True)
        X = df.drop([target_col, 'batch'], axis=1)
        y = df[target_col]
    elif type(target_col) == list:
        df.dropna(inplace=True)
        X = df.drop(target_col+['batch'], axis=1)
        y = df[target_col]

    # Разделение на числовые и категориальные признаки
    numeric_features = X.select_dtypes(include=['int64', 'float64', 'float32', 'int32']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    # Создание конвейера преобразования
    preprocessing = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('scaler', RobustScaler()),
                ('normalize', PowerTransformer(method='yeo-johnson')),
            ]), numeric_features),
            ('cat', Pipeline([
                ('onehot', OneHotEncoder(handle_unknown='ignore')),
            ]), categorical_features)
        ]
    )

    return X, y, preprocessing


def calculate_indicators(df, features, params=None, mode=None, multy=False):
    
    fc = FeatureCalculatorForRegression(df)
    if mode==None:
        df_features = fc.calculate_features(params=params, required_features=features)
    else:
        df_features = fc.calculate_all_possible_features()
    if multy == False:
        df1 = df_features[df_features['normalized_target'].notna()]
    else:
        df1 = df_features[df_features['multi_target_5'].notna()]
    features = ['open', 'close', 'high', 'low', 'volume', 'buy_signal', 'sell_signal','event_sell_time','event_sell_price',
                'event_time','event_price','event_sell_time','event_sell_price','target', 'pnl', 'ma', 'pmax'] #time
    df1['regime'] = df1['regime'].astype('object')
    df1[['batch','trade_bars_counter']] = df1[['batch', 'trade_bars_counter']].astype('int')
    df1 = df1.drop(features, axis=1)
    #df1 = df1.dropna()
    return df1, fc._timings

def sample_feature_params(params) -> dict:
        """
        Draws *one* sample of the whole feature-engineering hyper-parameter set.
        Rule of thumb for ranges:
          • lower bound = ‘sane minimum‘ from domain knowledge
          • upper bound = ‘sane maximum’
        Adjust them if you feel the search space is too wide or too narrow.
        """
        # ---- helpers for monotone constraints ----------------------------------
        fast  = params['macd_fast']
        slow  = params['macd_slow']
    
        slope_lag_min = params['slope_lag_min']
        slope_lag     = params['slope_lag']
    
        # ---- finally compose the nested dict -----------------------------------
        return {
            'base': {
                'medprice': params['medprice']
            },
            'macd': {
                'fast'      : fast,
                'slow'      : slow,
                'signal'    : params['macd_signal'],
                'macd_roll' : params['macd_roll']
            },
            'overbought': {
                'rsi_p'         : params['rsi_p'],
                'stoch_p'       : params['stoch_p'],
                'oversold_roll' : params['oversold_roll']
            },
            'madist': {
                'span_lenght'   : params['madist_span'],
                'madist_lenght' : params['madist_len']
            },
            'mean_reversion': {
                'window' : params['mr_window']
            },
            'fear_greed': {
                'greed_pct'    : params['fg_greed_pct'],
                'volume_ratio_scr' : params['fg_vol_ratio'],
                'window'       : params['fg_window'],
                'greed_roll'   : params['fg_roll']
            },
            'price_variation': {
                'variation_lenght': params['pv_len']
            },
            'pmax_ma': {
                'pmax_ma_lenght'      : params['pmax_len'],
                'pmax_ma_lenght_roll' : params['pmax_roll']
            },
            'slope': {
                'slope_lag'     : slope_lag,
                'slope_lag_min' : slope_lag_min,
                'sloap_pct'     : params['slope_pct'],
                'sloap_roll'    : params['slope_roll']
            },
            # _trade_duration_features – no params
        }

def build_feature_params(
    flat_params: Dict[str, Any],
    extra_alias: Optional[Dict[str, Tuple[str, str | None]]] = None
) -> Dict[str, Dict[str, Any]]:
    """
    Преобразует «плоский» словарь от Optuna в структуру,
    которую понимает FeatureCalculatorForRegression.
    """

    # 1. Базовая явная таблица соответствий
    alias: Dict[str, Tuple[str, str | None]] = {
        'hp_lamb'          : ('hp_trend'           , 'lamb'),
        'ea_pct'           : ('ema_acceleration'   , 'pct_window'),
        'ea_ema'           : ('ema_acceleration'   , 'ema_window'),
        'mr_window'        : ('Mean_Reversion'     , 'window'),
        'ag_window'        : ('Asset_Growth'       , 'window'),
        'medprice'         : ('MEDPRICE'           , 'medprice'),
        'bb_window'        : ('BB_Width'           , 'bb_window'),
        'macd_fast'        : ('MACD'               , 'fast'),
        'macd_slow'        : ('MACD'               , 'slow'),
        'macd_signal'      : ('MACD'               , 'signal'),
        'fg_window'        : ('Fear_Greed'         , 'window'),
        'atr_window'       : ('ATR_norm'           , 'atr_window'),
        'vr_window'        : ('volume_ratio'       , 'window'),
        'madist_span'      : ('Price_MADist%'      , 'span_lenght'),
        'slope_lag'        : ('slope_trend'        , 'slope_lag'),
        'slope_lag_min'    : ('slope_trend'        , 'slope_lag'),
        'rsi_p'            : ('Overbought_Oversold', 'rsi_p'),
        'stoch_p'          : ('Overbought_Oversold', 'stoch_p'),
        'pmax_len'         : ('pmax_norm'          , 'pmax_ma_length'),
        'pmax_roll'        : ('pmax_norm'          , 'pmax_ma_length_roll'),
        'pc_window'        : ('pmax_norm'          , 'pct_window'),
        'ema_trend_span'   : ('ema_trend'          , 'span'),
        'ema_trend_pct'    : ('ema_trend'          , 'pct_window'),
        'stat_window'      : ('stat_window', None),
        # --- новые алиасы ---
        'roc_window'        : ('ROC'          , 'window'),
        'willr_window'      : ('WILLR'          , 'window'),
        'fractal_short_win' : ('Fractal_Dim', 'short_win'),
        'fractal_long_win'  : ('Fractal_Dim', 'long_win'),
        'peak_price_win'    : ('Peak_Exhaustion_Score', 'price_win'),
        'peak_mom_win'      : ('Peak_Exhaustion_Score', 'mom_win'),
        'peak_vol_win'      : ('Peak_Exhaustion_Score', 'vol_win'),
        'peak_atr_win'      : ('Peak_Exhaustion_Score', 'atr_win'),
        'peak_z_win'        : ('Peak_Exhaustion_Score', 'z_win'),
        'bb_window'         : ('%B_BB', 'window'),
        'bb_std_mult'       : ('%B_BB', 'std_mult'),
        'kurt_window'       : ('Kurtosis_roll', 'window'),
        'obv_window'        : ('OBV_div', 'window'),
        'rsi_slope_rsi_p'   : ('RSI_slope', 'rsi_p'),
        'rsi_diff_win'      : ('RSI_slope', 'diff_win'),
        'voldec_window'     : ('Vol_Decay', 'window'),
        'acceldec_window'   : ('Accel_Decay', 'window'),
        'ent_window'        : ('Entropy_roll', 'window'),
        'wlt_short_win'     : ('Wavelet_Var_Ratio', 'short_win'),
        'wlt_long_win'      : ('Wavelet_Var_Ratio', 'long_win'),
        'acorr_window'      : ('Autocorr_Lag1', 'window'),
        'beta_window'       : ('Beta_Market', 'window'),
        'psc_vel_win'       : ('PSC', 'vel_win'),
        'psc_acc_win'       : ('PSC', 'acc_win'),
        'psc_vol_win'       : ('PSC', 'vol_win'),
        'psc_atr_win'       : ('PSC', 'atr_win'),
        'psc_z_win'         : ('PSC', 'z_win'),
    }

    # 2. Пользовательские переопределения
    if extra_alias:
        alias.update(extra_alias)

    # 3. Автоматический разбор префиксов (fallback)
    prefix_map: Dict[str, str] = {
        'macd'        : 'MACD',
        'hp'          : 'hp_trend',
        'ea'          : 'ema_acceleration',
        'mr'          : 'Mean_Reversion',
        'ag'          : 'Asset_Growth',
        'bb'          : 'BB_Width',
        'fg'          : 'Fear_Greed',
        'atr'         : 'ATR_norm',
        'vr'          : 'volume_ratio',
        'madist'      : 'Price_MADist%',
        'slope'       : 'slope_trend',
        'pmax'        : 'pmax_norm',
        'ema_trend'   : 'ema_trend',
        'rsi'         : 'Overbought_Oversold',
        'stoch'       : 'Overbought_Oversold',
        'fractal'     : 'Fractal_Dim',
        'peak'        : 'Peak_Exhaustion_Score',
        'bb'          : '%B_BB',
        'kurt'        : 'Kurtosis_roll',
        'obv'         : 'OBV_div',
        'rsi_slope'   : 'RSI_slope',
        'voldec'      : 'Vol_Decay',
        'acceldec'    : 'Accel_Decay',
        'entropy'     : 'Entropy_roll',
        'wavelet'     : 'Wavelet_Var_Ratio',
        'acorr'       : 'Autocorr_Lag1',
        'beta'        : 'Beta_Market',
        'psc'         : 'PSC',
    }

    nested: Dict[str, Dict[str, Any]] = defaultdict(dict)

    for key, val in flat_params.items():

        # 3.1 Явное соответствие
        if key in alias:
            prim, arg = alias[key]
            if prim == 'stat_window' or arg is None:
                nested['stat_window'] = val
            else:
                nested[prim][arg] = val
            continue

        # 3.2 Игнорируем вспомогательные ключи вида *_min, *_max, если
        #     они не нужны никакому примитиву.
        if key.endswith('_min') or key.endswith('_max'):
            continue

        # 3.3 Fallback-разбор <prefix>_<arg>
        if '_' in key:
            prefix, arg = key.split('_', 1)
            if prefix in prefix_map:
                nested[prefix_map[prefix]][arg] = val
                continue

        # 3.4 Неизвестный ключ — игнорируем или логируем
        # print(f'Warning: parameter "{key}" was not mapped')

    return {p: d for p, d in nested.items()}

_ORIG_INTERP = F.interpolate


def _collapse_pred_to_bt(y_pred: torch.Tensor) -> torch.Tensor:
    """
    Приводим предсказание к виду (B, T), считая последнюю ось временем (горизонтом).
    Все промежуточные оси (кроме batch=ось 0 и time=последняя ось) усредняем.
    Пример: (B, 1, 4, 10) -> mean по осям (1,2) -> (B,10)
    (B, 10) -> ок
    (B, 1, 10) -> squeeze -> (B,10)
    """
    if not isinstance(y_pred, torch.Tensor):
        raise TypeError(f"y_pred must be a tensor, got {type(y_pred)}")

    # Сначала уберём все единичные оси
    if any(s == 1 for s in y_pred.shape[1:-1]):
        # squeeze не трогает последнюю ось, если она не равна 1
        y_pred = y_pred.squeeze()
        # Если squeeze убрал не только единичные, но и привёл к (B, T) — хорошо.

    if y_pred.dim() == 1:
        # (B,) — интерпретируем как T=1, сделаем (B,1)
        y_pred = y_pred.unsqueeze(-1)
        return y_pred

    if y_pred.dim() == 2:
        # (B, T) — уже как надо
        return y_pred

    # Если размерностей больше 2: считаем last dim = time, batch = 0
    # Все промежуточные оси схлопываем усреднением
    reduce_dims = tuple(range(1, y_pred.dim() - 1))
    if len(reduce_dims) > 0:
        y_pred = y_pred.mean(dim=reduce_dims)
    # На выходе (B, T)
    if y_pred.dim() != 2:
        # На всякий случай добьёмся (B, T)
        y_pred = y_pred.view(y_pred.size(0), -1)
    return y_pred


def _install_safe_interpolate_patch():
    """
    Патч делает F.interpolate детерминированным при включённом torch.use_deterministic_algorithms(True)
    для CUDA и режимов linear/bilinear/bicubic, прогоняя вычисление на CPU.
    Идемпотентен и не меняет сигнатуру.
    """
    if getattr(F.interpolate, "_is_deterministic_wrapper", False):
        return

    _orig_interpolate = F.interpolate

    # Какие режимы считаем потенциально недетерминируемыми на CUDA
    _CUDA_UNSAFE_MODES = {"linear", "bilinear", "bicubic"}  # 1d/2d/2d

    def _needs_cpu_fallback(input, mode):
        if not torch.is_tensor(input):
            return False
        if input.is_cuda and mode in _CUDA_UNSAFE_MODES:
            # upsample_linear1d_backward_out_cuda и др. — недетерминируемы
            return True
        return False

    @functools.wraps(_orig_interpolate)
    def _deterministic_interpolate(
        input: torch.Tensor,
        size=None,
        scale_factor=None,
        mode="nearest",
        align_corners=None,
        recompute_scale_factor=None,
        antialias=False,
    ):
        # Если не нужно, просто вызовем оригинал
        if not _needs_cpu_fallback(input, mode):
            return _orig_interpolate(
                input,
                size=size,
                scale_factor=scale_factor,
                mode=mode,
                align_corners=align_corners,
                recompute_scale_factor=recompute_scale_factor,
                antialias=antialias,
            )

        # CUDA + linear/bilinear/bicubic → CPU fallback
        x = input
        dev = x.device
        orig_dtype = x.dtype

        # Для стабильности переводим в float32 на CPU
        x_cpu = x.detach().to("cpu", dtype=torch.float32).requires_grad_(x.requires_grad)

        y_cpu = _orig_interpolate(
            x_cpu,
            size=size,
            scale_factor=scale_factor,
            mode=mode,
            align_corners=align_corners,
            recompute_scale_factor=recompute_scale_factor,
            antialias=antialias,
        )

        # Возвращаем на исходное устройство и тип
        y = y_cpu.to(dev, dtype=orig_dtype)

        return y

    _deterministic_interpolate._is_deterministic_wrapper = True  # type: ignore[attr-defined]
    F.interpolate = _deterministic_interpolate


_install_safe_interpolate_patch()


def _unpack_pf_batch(batch):
    """
    Унифицированная распаковка батча из TimeSeriesDataSet.to_dataloader(...)
    Возвращает: x (dict), y (Tensor|None), weight (Tensor|None)
    """
    if isinstance(batch, (list, tuple)):
        if len(batch) == 3:
            x, y, weight = batch
        elif len(batch) == 2:
            x, y = batch
            weight = None
        else:
            raise ValueError(f"Unexpected batch tuple length: {len(batch)}")
    elif isinstance(batch, dict):
        # На всякий случай поддержим dict → возьмём таргет из decoder_target, если есть
        x = batch
        y = batch.get("decoder_target", None)
        weight = None
    else:
        raise TypeError(f"Unexpected batch type: {type(batch)}")
    return x, y, weight


# F.interpolate = _deterministic_interpolate


def _extract_pred_tensor(y_pred):
    # извлекаем тензор предикта из любых обёрток
    if isinstance(y_pred, dict):
        for key in ("prediction", "output", "decoder_output"):
            if key in y_pred and torch.is_tensor(y_pred[key]):
                return y_pred[key]
        # если не нашли — попробуем fallback: первый тензор в dict
        for v in y_pred.values():
            if torch.is_tensor(v):
                return v
        raise ValueError("Could not extract prediction tensor from dict y_pred.")

    if isinstance(y_pred, (list, tuple)):
        # обычно y_pred[0] — предсказание
        return y_pred[0]

    if torch.is_tensor(y_pred):
        return y_pred

    raise TypeError(f"Unsupported y_pred type: {type(y_pred)}")


class EventTimeSeriesSplit(BaseCrossValidator):
    """
    Кросс-валидация по событиям (batch), хронологическая, с эмбарго.
    groups: массив той же длины, что и df, со значениями batch
    times:  массив pd.Timestamp (или sortable), та же длина, что и df
    """

    def __init__(self, n_splits: int = 5, embargo_events: int = 1, min_train_events: int = 5):
        self.n_splits = n_splits
        self.embargo_events = embargo_events
        self.min_train_events = min_train_events

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X, y=None, groups=None, times: Optional[pd.Series] = None) -> Iterator[
        Tuple[np.ndarray, np.ndarray]]:
        if groups is None or times is None:
            raise ValueError("Pass groups=batch and times=time columns")

        groups = np.asarray(groups)
        times = pd.to_datetime(times)

        # порядок событий по старт-времени
        df_tmp = pd.DataFrame({"group": groups, "time": times}).reset_index(names="row_idx")
        first_time = df_tmp.groupby("group")["time"].min().sort_values()
        uniq_groups = first_time.index.to_numpy()

        n_events = len(uniq_groups)
        if n_events < (self.n_splits + self.min_train_events):
            # уменьшаем число сплитов, если событий мало
            eff_splits = max(1, n_events - self.min_train_events)
        else:
            eff_splits = self.n_splits

        # на каждой итерации расширяем train вправо
        for split_idx in range(1, eff_splits + 1):
            # доля событий для валидации
            val_events = max(1, n_events // (eff_splits + 1))
            train_end = n_events - (eff_splits - split_idx + 1) * val_events

            if train_end < self.min_train_events:
                continue

            # эмбарго
            embargoed_end = max(0, train_end - self.embargo_events)

            train_groups = uniq_groups[:embargoed_end]
            val_groups = uniq_groups[train_end: train_end + val_events]

            train_idx = df_tmp.index[df_tmp["group"].isin(train_groups)].to_numpy()
            val_idx = df_tmp.index[df_tmp["group"].isin(val_groups)].to_numpy()

            # индексы исходной X (если это DataFrame — у вас совпадают позиции с row_idx)
            yield (train_idx, val_idx)


class MinimalRichProgressBar(RichProgressBar):
    def on_validation_start(self, trainer, pl_module):
        pass

    def on_validation_batch_start(self, trainer, pl_module, batch, batch_idx):
        pass

    def on_validation_end(self, trainer, pl_module):
        pass


class NoValidationBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        # возвращаем полностью отключённый tqdm для валидации
        return tqdm_class(disable=True)

class CustomTFT(TemporalFusionTransformer):
    def __init__(self, *args, **kwargs):
        self.mask_prob = kwargs.pop("mask_prob", 0.05)
        super().__init__(*args, **kwargs)
        self._val_preds = []
        self._val_trues = []
        self._val_gids = []
        self.scheduled_prob = 0.0
        
        safe_val = torch.tensor(
            torch.finfo(torch.float16).min,
            dtype=torch.float32
        )
    
        m = self.multihead_attn.attention
        if hasattr(m, "mask_bias") and not isinstance(m.mask_bias, torch.Tensor):
            delattr(m, "mask_bias")
        m.register_buffer("mask_bias", safe_val)

    def on_epoch_start(self, trainer, pl_module):
        if trainer.max_epochs > 0:
            self.scheduled_prob = min(1.0, trainer.current_epoch / (trainer.max_epochs * 0.8))

    def on_train_epoch_end(self):
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    def training_step(self, batch, batch_idx):
        x, y, weight = _unpack_pf_batch(batch)
    
        if torch.rand(1).item() < self.mask_prob and "encoder_target" in x:
            enc = x["encoder_target"]
            noise = torch.normal(0, 0.15, size=enc.shape, device=enc.device)
            x = {**x, "encoder_target": noise}
            del enc, noise
    
        if torch.rand(1).item() < self.scheduled_prob and y is not None:
            with torch.no_grad():
                out = self(x)
                y_pred = self.loss.to_prediction(out)
                y_bt = _collapse_pred_to_bt(y_pred)
                dec_tgt = y_bt if y_bt.dim() == 2 else y_bt.unsqueeze(-1)
                x['decoder_target'] = dec_tgt.detach()
                del out, y_pred, y_bt, dec_tgt
    
        batch = (x, y, weight) if weight is not None else (x, y)
        result = super().training_step(batch, batch_idx)
        
        if batch_idx % 50 == 0:  # Rare for speed
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        return result
    
    def validation_step(self, batch, batch_idx):
        x, y, weight = _unpack_pf_batch(batch)
    
        with torch.no_grad():
            out_temp = self(x)
            y_pred_temp = self.loss.to_prediction(out_temp)
            y_bt_temp = _collapse_pred_to_bt(y_pred_temp)
            del out_temp, y_pred_temp

        if "decoder_target" in x:
            enc_tgt = x.get("encoder_target")
            if enc_tgt is not None:
                batch_mean = enc_tgt.mean(dim=-1, keepdim=True)
                batch_std = enc_tgt.std(dim=-1, keepdim=True) + 1e-8
                mean_tensor = batch_mean.expand_as(x["decoder_target"])
                std_tensor = (0.1 * batch_std).expand_as(x["decoder_target"])
                noise_fill = torch.normal(mean_tensor, std_tensor)
                del batch_mean, std_tensor, mean_tensor
            else:
                dec_tgt = x["decoder_target"]
                device = dec_tgt.device
                noise_fill = torch.full_like(dec_tgt, self.global_target_mean)
                batch_size = dec_tgt.size(0)
                batch_std = torch.full((batch_size,), self.global_target_std, device=device).unsqueeze(-1)
            
            if torch.rand(1).item() < self.scheduled_prob:
                decoder_fill = y_bt_temp
            else:
                jitter_size = noise_fill.shape
                additional_jitter = torch.randn(jitter_size, device=noise_fill.device) * (0.05 * batch_std.expand_as(noise_fill))
                decoder_fill = noise_fill + additional_jitter
                del additional_jitter, noise_fill
            
            decoder_fill = torch.clamp(decoder_fill, -1.0, 1.0)
            x["decoder_target"] = decoder_fill
            del y_bt_temp, batch_std
    
        out = self(x)
        y_pred_raw = self.loss.to_prediction(out)
        del out
    
        y_pred_metrics = torch.clamp(y_pred_raw, -1.0, 1.0)
        del y_pred_raw
    
        y_actual = y if y is not None else x.get("decoder_target")
        if y_actual is None:
            return
    
        try:
            y_pred_aligned, y_actual_aligned = _align_pred_target(y_pred_metrics, y_actual)
            del y_pred_metrics, y_actual
        except Exception:
            return
    
        self._val_preds.append(y_pred_aligned.detach())
        self._val_trues.append(y_actual_aligned.detach())
        del y_pred_aligned, y_actual_aligned
    
        gid = x.get("group_ids")
        if gid is not None and isinstance(gid, torch.Tensor):
            self._val_gids.append(gid.detach())
        else:
            self._val_gids.append(None)
        
        if batch_idx % 50 == 0:  # Rare for speed
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    def on_validation_epoch_end(self):
        if len(self._val_preds) == 0:
            return
    
        yp = torch.cat(self._val_preds, dim=0)
        yt = torch.cat(self._val_trues, dim=0)
    
        self._val_preds.clear()
        self._val_trues.clear()
    
        se = (yp - yt) ** 2
        val_mse = float(se.mean().item())
        val_mse_std = float(se.std(unbiased=False).item())
        del yp, yt
    
        self.log("val_mse", val_mse, prog_bar=True, on_step=False, on_epoch=True)
        self.log("val_mse_std", val_mse_std, prog_bar=False, on_step=False, on_epoch=True)
    
        has_any_gid = any(g is not None for g in self._val_gids)
        if has_any_gid:
            gid_list = []
            valid = True
            for g in self._val_gids:
                if g is None:
                    valid = False
                    break
                gid_list.append(g)
    
            if valid and len(gid_list) > 0:
                gid_all = torch.cat(gid_list, dim=0).numpy().ravel()
                se_np = se.numpy().ravel()
                del se
    
                if gid_all.shape[0] == se_np.shape[0]:
                    uniq = np.unique(gid_all)
                    g_mse = [se_np[gid_all == u].mean() for u in uniq if (gid_all == u).any()]
                    if len(g_mse) > 0:
                        g_mse = np.asarray(g_mse, dtype=float)
                        val_mse_group_mean = float(g_mse.mean())
                        val_mse_group_std = float(g_mse.std(ddof=0))
                        self.log("val_mse_group_mean", val_mse_group_mean, prog_bar=False, on_step=False, on_epoch=True)
                        self.log("val_mse_group_std", val_mse_group_std, prog_bar=True, on_step=False, on_epoch=True)
                    del g_mse, uniq
    
                del gid_all, se_np
    
        self._val_gids.clear()
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    def log(self, name, value, *args, **kwargs):
        if value is None:
            return
        super().log(name, value, *args, **kwargs)


def _worker_init_fn(worker_id: int, seed: int):
    """
    Глобальная функция для инициализации worker-а DataLoader-а.
    pickle её «видит» и может передать в подпроцессы.
    """
    set_seeds(seed + worker_id)


def _extract_tensor(x, role="pred"):
    """
    Извлекает torch.Tensor из различных контейнеров/структур.
    - dict: сперва пробуем ключи, характерные для предсказаний/таргета
    - tuple/list: берём первый тензор или первый элемент, приводимый к тензору
    - tensor: возвращаем как есть
    """
    if isinstance(x, torch.Tensor):
        return x

    if isinstance(x, dict):
        # Наиболее типичные ключи в pytorch-forecasting / lightning шагах
        preferred_keys = [
            "prediction", "pred", "output", "y_pred", "yhat", "y", "target"
        ]
        for k in preferred_keys:
            if k in x and isinstance(x[k], torch.Tensor):
                return x[k]
        # Если значения-словари/кортежи — попробуем рекурсивно
        for v in x.values():
            if isinstance(v, torch.Tensor):
                return v
            if isinstance(v, (list, tuple, dict)):
                try:
                    t = _extract_tensor(v, role=role)
                    if isinstance(t, torch.Tensor):
                        return t
                except Exception:
                    pass
        raise TypeError(f"Cannot extract tensor from dict for role={role}. Keys={list(x.keys())}")

    if isinstance(x, (list, tuple)):
        for item in x:
            if isinstance(item, torch.Tensor):
                return item
        # если нет прямого тензора — попробуем рекурсивно
        for item in x:
            if isinstance(item, (list, tuple, dict)):
                try:
                    t = _extract_tensor(item, role=role)
                    if isinstance(t, torch.Tensor):
                        return t
                except Exception:
                    pass
        raise TypeError(f"Cannot extract tensor from {type(x)} for role={role}")

    # Последняя попытка — у объектов некоторых библиотек есть .values или .tensor
    for attr in ("values", "tensor", "data"):
        if hasattr(x, attr):
            v = getattr(x, attr)
            if isinstance(v, torch.Tensor):
                return v

    raise TypeError(f"Unsupported type for tensor extraction (role={role}): {type(x)}")


def _maybe_squeeze_last(x):
    """
    Безопасно убираем последнюю размерность, если она равна 1.
    Если x не тензор — возвращаем как есть.
    """
    if not isinstance(x, torch.Tensor):
        return x
    if x.dim() > 0 and x.size(-1) == 1:
        return x.squeeze(-1)
    return x


def _align_pred_target(y_pred, y_actual):
    """
    Приводим предсказания и таргет к совместимым формам для MSE:
    - Извлекаем тензоры из возможных контейнеров.
    - Сводим предсказание к (B, T_pred) с последней осью как временем.
    - Таргет сводим к (B,) или (B, T_act).
    - Если таргет (B,) — берём последний горизонт из предсказаний.
    - Если таргет (B, T_act) — подгоняем по времени (обрезаем/проверяем равенство).
    """
    # 1) Достаём тензоры
    y_pred = _extract_tensor(y_pred, role="pred")
    y_actual = _extract_tensor(y_actual, role="target")

    # 2) Сжимаем последнюю единичную ось
    y_pred = _maybe_squeeze_last(y_pred)
    y_actual = _maybe_squeeze_last(y_actual)

    # Быстрый путь: формы совпали
    if isinstance(y_pred, torch.Tensor) and isinstance(y_actual, torch.Tensor):
        if y_pred.shape == y_actual.shape:
            return y_pred, y_actual

    # 3) Приводим предсказание к (B, T_pred)
    y_pred_bt = _collapse_pred_to_bt(y_pred)  # (B, T_pred)

    # 4) Приведём таргет к (B,) или (B, T_act)
    if y_actual.dim() == 1:
        # (B,) — ожидаем 1 шаг на таргет → берём последний горизонт из предсказаний
        if y_pred_bt.dim() != 2 or y_pred_bt.size(0) != y_actual.size(0):
            raise ValueError(f"Batch mismatch: pred={tuple(y_pred_bt.shape)} vs target={tuple(y_actual.shape)}")
        y_pred_aligned = y_pred_bt[:, -1]  # последний шаг горизонта
        return y_pred_aligned, y_actual

    if y_actual.dim() == 2:
        # (B, T_act)
        if y_pred_bt.size(0) != y_actual.size(0):
            raise ValueError(f"Batch mismatch: pred={tuple(y_pred_bt.shape)} vs target={tuple(y_actual.shape)}")
        T_pred = y_pred_bt.size(1)
        T_act = y_actual.size(1)
        if T_pred == T_act:
            return y_pred_bt, y_actual
        if T_pred > T_act:
            # Обрежем последние T_act шагов, чтобы соответствовать таргету
            y_pred_bt = y_pred_bt[:, -T_act:]
            return y_pred_bt, y_actual
        # Если предсказаний по времени меньше, чем в таргете — это логическая ошибка настройки
        raise ValueError(
            f"Prediction horizon shorter than target: pred T={T_pred}, target T={T_act} "
            f"(pred shape={tuple(y_pred_bt.shape)}, target shape={tuple(y_actual.shape)})"
        )

    # Случай редкий: если таргет внезапно >2D — пробуем схлопнуть по всем, кроме батча
    if y_actual.dim() > 2:
        # Схлопнём таргет к (B, T_act) по последней оси
        reduce_dims = tuple(range(1, y_actual.dim() - 1))
        if len(reduce_dims) > 0:
            y_actual_bt = y_actual.mean(dim=reduce_dims)
        else:
            y_actual_bt = y_actual
        # Рекурсивно выровняем теперь как (B, ?)
        return _align_pred_target(y_pred_bt, y_actual_bt)

    # Если таргет скалярный (редко, но вдруг), расширим до (B,) повтором
    if y_actual.dim() == 0:
        y_actual = y_actual.expand(y_pred_bt.size(0))
        y_pred_aligned = y_pred_bt[:, -1]
        return y_pred_aligned, y_actual

    # Если сюда дошли — что-то совсем нетипичное
    raise ValueError(
        f"Shapes still mismatch after alignment: pred={tuple(y_pred.shape)} vs target={tuple(y_actual.shape)}"
    )


# Фиксируем seeds для воспроизводимости и стабильности
def set_seeds(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    pl.seed_everything(seed, verbose=False)


class PeakFriendlyHuber(Metric):
  
    def __init__(
        self,
        delta: float = 0.5,
        peak_thr: float = 0.85,
        peak_weight: float = 1.6,  # Увеличено до 1.6 для stronger поощрения пиков
        contrast_weight: float = 0.02,
        center_band: float = 0.3,
        clip_scale: float = 1.5,  # Новый: scale для soft-clip (tanh * scale, чтобы не сжимать середину сильно)
    ):
        super().__init__()
        self.delta = float(delta)
        self.peak_thr = float(peak_thr)
        self.peak_weight = float(peak_weight)
        self.contrast_weight = float(contrast_weight)
        self.center_band = float(center_band)
        self.clip_scale = float(clip_scale)  # Новый параметр
        self.mse = MeanSquaredError()

    @staticmethod
    def _smooth_l1(diff, delta):
        absd = diff.abs()
        return torch.where(absd < delta, 0.5 * (diff ** 2) / delta, absd - 0.5 * delta)

    def to_prediction(self, y_pred):
        y_pred = _extract_tensor(y_pred, role="pred")
        return super().to_prediction(y_pred)

    def to_quantiles(self, y_pred, quantiles=None, **kwargs):
        y_pred = _extract_tensor(y_pred, role="pred")
        return super().to_quantiles(y_pred, quantiles=quantiles, **kwargs)

    def loss(self, y_pred, y_actual, **kwargs):
        y_pred = _extract_tensor(y_pred, role="pred")
        y_actual = _extract_tensor(y_actual, role="target")
        y_pred, y_actual = _align_pred_target(y_pred, y_actual)

        # Soft-clip (без изменений)
        y_pred = torch.tanh(y_pred * self.clip_scale) / self.clip_scale

        diff = y_pred - y_actual
        base = self._smooth_l1(diff, self.delta)
        
        # Адаптивный peak_weight: средний по батчу, scale от доли пиков
        if self.peak_weight > 1.0:
            with torch.no_grad():
                peak_mag = torch.relu(y_actual.abs() - self.peak_thr)
                peak_frac = (peak_mag > 0).float().mean()  # Доля пиков в батче
                adaptive_weight = 1.0 + (self.peak_weight - 1.0) * peak_frac  # Больше веса, если много пиков
                w = adaptive_weight * torch.clamp(peak_mag / (1.0 - self.peak_thr + 1e-8), 0.0, 1.0) + 1.0
            huber_term = (base * w).mean()
        else:
            huber_term = base.mean()
        
        # Лёгкий «anti-flatness» у центра: штрафим чрезмерно малую амплитуду,
        # но только там, где таргет далеко от 0.
        if self.contrast_weight > 0.0:
            with torch.no_grad():
                far_mask = (y_actual.abs() >= self.center_band).float()
                near_mask = (y_actual.abs() < self.center_band).float()
            
            # Прямая амплитуда предсказания
            far_amp = (y_pred.abs() * far_mask).sum() / (far_mask.sum() + 1e-8)
            near_amp = (y_pred.abs() * near_mask).sum() / (near_mask.sum() + 1e-8)
            
            # Хотим far_amp >= near_amp + margin; введём небольшой margin
            margin = 0.05
            contrast = torch.relu((near_amp + margin) - far_amp)
            loss = huber_term + self.contrast_weight * contrast
        else:
            loss = huber_term
        
        return loss

    def __call__(self, y_pred, y_actual, **kwargs):
        return self.loss(y_pred, y_actual, **kwargs)

    def update(self, y_pred, y_actual, **kwargs):
        y_pred = _extract_tensor(y_pred, role="pred")
        y_actual = _extract_tensor(y_actual, role="target")
        y_pred, y_actual = _align_pred_target(y_pred, y_actual)
        self.mse.update(y_pred, y_actual)

    def compute(self):
        return self.mse.compute()

    def reset(self):
        self.mse.reset()

    def name(self):
        return "PeakFriendlyHuber"

class TFTAdapter(pl.LightningModule):
    """
    Обёртка над TemporalFusionTransformer,
    чтобы Trainer воспринимал модель нужного типа
    и наш tft.training_step видел непустой self.trainer.
    """

    def __init__(self, tft: TemporalFusionTransformer):
        super().__init__()
        self.tft = tft

    def on_fit_start(self) -> None:
        # вызовется перед стартом Trainer.fit
        # прикрепляем Trainer к внутреннему tft
        self.tft.trainer = self.trainer
        # и логгеры
        self.tft.log = self.log
        self.tft.log_dict = self.log_dict

    def forward(self, *args, **kwargs):
        return self.tft(*args, **kwargs)

    def training_step(self, batch, batch_idx):
        self.tft.trainer = self.trainer
        result = self.tft.training_step(batch, batch_idx)
        if isinstance(result, dict):
            return result.get("loss")
        elif isinstance(result, tuple) and len(result) >= 2:
            log_dict, out = result[:2]  # Берем первые два элемента
            if isinstance(log_dict, dict):
                return log_dict.get("loss")
        # Если ни один вариант не подошел
        raise ValueError(f"Unexpected return type from tft.training_step: {type(result)}")

    def validation_step(self, batch, batch_idx):
        # обеспечим корректную ссылку на тренер внутри tft (если нужно)
        self.tft.trainer = self.trainer
        self.tft.validation_step(batch, batch_idx)
        return

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        # Просто передаём вызов внутреннему TFT, без dataloader_idx (не нужен для TFT)
        return self.tft.predict_step(batch, batch_idx)

    def predict(self, *args, **kwargs):
        return self.tft.predict(*args, **kwargs)

    def configure_optimizers(self):
        return self.tft.configure_optimizers()


def to_dense(X):
    """Преобразование sparse matrix в dense numpy array"""
    if hasattr(X, 'toarray'):
        return X.toarray()
    return np.asarray(X)

def prepare_data_transformer(df, target_col):
    df = df.dropna(subset=[target_col])
    if 'time' in df.columns and 'time_idx' not in df.columns:
        df = df.rename(columns={'time': 'time_idx'})
    X = df.drop(columns=target_col)
    y = df[target_col].copy()

    exclude = ['time_idx', 'batch']
    numeric_features = [
        c for c in X.select_dtypes(include=['int64', 'float64', 'float32', 'int32']).columns
        if c not in exclude
    ]
    categorical_features = [
        c for c in X.select_dtypes(include=['object', 'category']).columns
        if c not in exclude
    ]

    preprocessing = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('scaler', RobustScaler()),
                ('yeo', PowerTransformer(method='yeo-johnson'))
            ]), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ],
        remainder='passthrough'  # passthrough → тут окажутся сначала все num→cat, а потом time_idx и batch
    )
    # Возвращаем дополнительные списки для передачи в модель
    return X, y, preprocessing, numeric_features, categorical_features

def split_features_batch_time(X_array, n_transformed):
    """
    X_array: np.ndarray после преобразований shape=(N, n_transformed + 2)
    n_transformed: сколько колонок ушло на num+cat
    возвращает (features, time_raw, batch_raw)
    """
    features = X_array[:, :n_transformed]
    batch_raw = X_array[:, n_transformed].ravel()
    time_raw = X_array[:, n_transformed + 1].ravel()
    return features, batch_raw, time_raw


def tft_output_transformer(x):
    # Больше НЕ клипуем внутри графа. Пусть модель учится выходить за [-1,1],
    # а мы ограничим при расчёте метрик и при возврате пользователю.
    return x


def get_early_stopping_callback(patience=10, min_delta=0.001):
    return EarlyStopping(
        monitor="train_loss",
        patience=patience,
        min_delta=min_delta,
        mode="min",
        verbose=True
    )


class SequenceTransformerRegressor(BaseEstimator, RegressorMixin):
    """
    Temporal Fusion Transformer для последовательностей.
    Интегрируется в Pipeline аналогично LSTM.
    Адаптировано для стабильного обучения и алготрейдинга (реального времени).
    """

    def __init__(self,
                 seq_len: int = 10,
                 pred_len: int = 1,
                 hidden_size: int = 64,  # Увеличено для лучшей емкости
                 hidden_continuous_size: int = 24,
                 epochs: int = 100,  # Увеличено для более долгого обучения
                 batch_size: int = 128,  # Увеличено для стабильности
                 learning_rate: float = 1e-3,  # Увеличено для более быстрого старта
                 patience: int = 15,  # Увеличено для терпимости
                 seed: int = 42,
                 dropout: float = 0.25,  # Уменьшено для меньшей регуляризации
                 weight_decay: float = 1e-4,  # Новый: для регуляризации
                 verbose: int = 2,
                 mask_prob: float = 0.1,  # Уменьшено, чтобы меньше шумить
                 infer_stride: int = 2,
                 ckpt_path=None,
                 preprocessing=None,
                 numeric_features=None,
                 categorical_features=None,
                 remainder_columns=None,
                 min_encoder_length: int = 1):  # Больше логов
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.hidden_size = hidden_size
        self.hidden_continuous_size = hidden_continuous_size
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.patience = patience
        self.seed = seed
        self.verbose = verbose
        self._model = None
        self._trainer = None
        self._n_transformed = None
        self._n_feat = None
        self._train_dataset = None
        self._feature_columns = None
        self._dropout = dropout
        self.norm_eps = 1e-6
        self.norm_window = max(10, self.seq_len // 2)
        self.ckpt_path = ckpt_path
        self.weight_decay = weight_decay  # Новый
        self.mask_prob = mask_prob
        self.infer_stride = infer_stride
        self.global_target_mean = 0.0  # Будем вычислять в fit
        self.global_target_std = 1.0  # Fallback global scale
        self.global_target_min = None
        self.global_target_max = None
        self.global_range = None
        self.soft_clip_scale =  None
        self.preprocessing = preprocessing
        self.numeric_features = numeric_features or []
        self.categorical_features = categorical_features or []
        self.remainder_columns = remainder_columns or ['batch', 'time']
        self.use_tanh_post = False
        self.train_q_lo = None
        self.train_q_hi = None
        self.clip_scale = 1.5
        self.future_fill_mode = "repeat" 
        self.smooth_window = max(5, self.seq_len // 5)  # For savgol
        self.smooth_poly = 2
        self.ema_alpha = 0.1
        self.infer_batch_size = 512  # Новый: большой батч для inference
        self.infer_stride = infer_stride if infer_stride is not None else 4 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'# Добавьте это, если нужно tanh в предикте
        self.min_encoder_length = max(1, min_encoder_length)  # Не меньше 1

    def fit(self, X, y):
        set_seeds(self.seed)

        X = np.asarray(X)
        if self._n_transformed is None:
            self._n_transformed = X.shape[1] - 2
            self._n_feat = X.shape[1]

        feat, batch_raw, time_raw = split_features_batch_time(X, self._n_transformed)

        df = pd.DataFrame(feat, columns=[f"f{i}" for i in range(feat.shape[1])])
        self._feature_columns = df.columns.tolist()

        df["batch"] = pd.Series(batch_raw).astype("int64")
        df["time_raw"] = pd.to_datetime(time_raw, utc=True, errors="coerce")

        if isinstance(y, (pd.Series, pd.DataFrame)):
            y = y.reset_index(drop=True)
        df["target"] = pd.Series(y, index=df.index).astype(float)

        before = len(df)
        df = df.dropna(subset=["time_raw", "target"]).reset_index(drop=True)
        if self.verbose and len(df) < before:
            print(f"Dropped {before - len(df)} rows with invalid time/target")

        df = df.sort_values(["batch", "time_raw"]).reset_index(drop=True)
        df["time_idx"] = df.groupby("batch").cumcount()

        min_len = max(1, int(self.pred_len))  # Изменено: позволяем короткие батчи (encoder может быть < seq_len)
        gsize = df.groupby("batch").size()
        valid_batches = gsize[gsize >= min_len].index
        if len(valid_batches) == 0:
            raise ValueError(f"No batches with length >= {min_len}. Reduce pred_len.")
        if self.verbose and len(valid_batches) < gsize.index.nunique():
            dropped = sorted(list(set(gsize.index) - set(valid_batches)))
            print(f"Warning: dropped {len(dropped)} short batches: {dropped[:8]}{' ...' if len(dropped) > 8 else ''}")
        df = df[df["batch"].isin(valid_batches)].reset_index(drop=True)

        batch_starts = df.groupby("batch")["time_raw"].min().sort_values()
        uniq_batches = batch_starts.index.to_numpy()
        n_total = len(uniq_batches)
        val_frac = 0.3 if n_total >= 10 else 0.1  # Увеличено для лучшего обобщения
        n_val = max(1, int(round(n_total * val_frac)))

        embargo = 1 if n_total >= 8 else 0

        train_end = max(0, n_total - n_val - embargo)
        train_batches = uniq_batches[:train_end]
        val_batches = uniq_batches[-n_val:]

        if len(train_batches) == 0 and n_total > 1:
            train_batches = uniq_batches[:-1]
            val_batches = uniq_batches[-1:]

        train_df = df[df["batch"].isin(train_batches)].copy()
        val_df = df[df["batch"].isin(val_batches)].copy()

        self.global_target_mean = train_df["target"].mean()
        self.global_target_std = train_df["target"].std() + 1e-8
        self.global_target_min = train_df["target"].min()
        self.global_target_max = train_df["target"].max()
        self.global_range = self.global_target_max - self.global_target_min + 1e-8
        self.soft_clip_scale = max(1.0, self.global_target_std * 1.5)  # Adaptive soft clip for stable [-1,1] without compression

        #self.clip_scale = max(1.0, self.global_target_std * 1.2)   # Adaptive to train variance

        train_targets = train_df["target"].values
        #if len(train_targets) > 0:
        #   self.train_q_lo, self.train_q_hi = np.quantile(train_targets, [0.01, 0.99])

        if self.verbose:
            print(f"Train batches: {len(np.unique(train_batches))}")
            print(f"Val batches: {len(np.unique(val_batches))}")
            print(f"Train rows: {len(train_df)}, Val rows: {len(val_df)}")

        full_dataset = TimeSeriesDataSet(
            df,
            time_idx="time_idx",
            target="target",
            group_ids=["batch"],
            max_encoder_length=int(self.seq_len),
            min_encoder_length=0,#int(self.min_encoder_length),  # Новый: позволяем короткие encoder
            max_prediction_length=int(self.pred_len),
            time_varying_unknown_reals=self._feature_columns,
            target_normalizer=None,
            allow_missing_timesteps=True,
            add_relative_time_idx=True,
            add_target_scales=False,
            add_encoder_length=True,
            min_prediction_length=1,
        )

        train_dataset = TimeSeriesDataSet.from_dataset(
            full_dataset, train_df, predict=False, stop_randomization=True
        )
        val_dataset = TimeSeriesDataSet.from_dataset(
            full_dataset, val_df, predict=False, stop_randomization=True
        ) if len(val_df) > 0 else None

        train_dl = train_dataset.to_dataloader(
            train=True,
            batch_size=int(self.batch_size),
            shuffle=True,
            num_workers=0,
            worker_init_fn=None,
            drop_last=False,
            persistent_workers=False,
        )
        val_dl = None
        if val_dataset is not None and len(val_dataset) > 0:
            val_dl = val_dataset.to_dataloader(
                train=False,
                batch_size=int(self.batch_size),
                shuffle=False,
                num_workers=0,
                worker_init_fn=None,
                drop_last=False,
                persistent_workers=False,
            )

        tft = CustomTFT.from_dataset(
            train_dataset,
            hidden_size=int(self.hidden_size),
            output_size=1,
            loss=PeakFriendlyHuber(
                delta=0.5,
                peak_thr=0.85,  # можно затем подвинуть 0.8..0.9
                peak_weight=1.3,  # аккуратно: 1.3..1.6
                contrast_weight=0.03,  # очень маленькая добавка
                center_band=0.3,  # что считать «центром»
                clip_scale=1.5
            ),
            optimizer="adam",
            learning_rate=float(self.learning_rate),  # оставьте тот, на котором MSE был лучше (у вас 1e-4 давал ~0.186)
            lstm_layers=3,
            hidden_continuous_size=self.hidden_continuous_size,
            attention_head_size=4,
            dropout=float(self._dropout),
            reduce_on_plateau_patience=5,
            reduce_on_plateau_min_lr=1e-6,
            weight_decay=float(self.weight_decay),
            mask_prob=float(self.mask_prob),
            output_transformer=tft_output_transformer,
        )

        class GCCallback(pl.Callback):
            def __init__(self):
                super().__init__()
                self.last_mem = psutil.Process().memory_info().rss / 1e6
                
            def _check_gc(self):
                current_mem = psutil.Process().memory_info().rss / 1e6
                if current_mem - self.last_mem > 50:
                    gc.collect()
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                self.last_mem = current_mem
                
            def on_train_epoch_end(self, trainer, pl_module):
                self._check_gc()
                
            def on_validation_epoch_end(self, trainer, pl_module):
                self._check_gc()
            
            def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
                if batch_idx % 10 == 0:
                    self._check_gc()
            
            def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
                if batch_idx % 10 == 0:
                    self._check_gc()

        callbacks = []
        if val_dl is not None:
            callbacks.append(
                get_early_stopping_callback(patience=self.patience, min_delta=1e-3))  # Новый: больше patience
            checkpoint_callback = ModelCheckpoint(monitor="train_loss", mode="min", save_top_k=1, verbose=True)
            callbacks.append(checkpoint_callback)
        else:
            checkpoint_callback = None

        logger = TensorBoardLogger(save_dir="lightning_logs/", name="my_model") if self.verbose > 0 else False
        if self.verbose > 0:
            callbacks.append(LearningRateMonitor(logging_interval="step"))
            callbacks.append(NoValidationBar(refresh_rate=20))
        callbacks.append(GCCallback())

        self._model = TFTAdapter(tft)
        self._trainer = pl.Trainer(
            max_epochs=int(self.epochs),
            enable_checkpointing=(checkpoint_callback is not None),
            callbacks=callbacks,
            logger=logger,
            enable_model_summary=True,
            gradient_clip_val=1.0,
            gradient_clip_algorithm="norm",
            deterministic=True,
            benchmark=False,
            accelerator="gpu" if torch.cuda.is_available() else "cpu",
            precision=32,
            limit_val_batches=1.0 if val_dl is not None else 0.0,
            enable_progress_bar=self.verbose > 0,
            log_every_n_steps=50,
            num_sanity_val_steps=0,
        )

        self._trainer.fit(
            self._model,
            train_dataloaders=train_dl,
            val_dataloaders=val_dl if val_dl is not None else None,
            ckpt_path=self.ckpt_path if self.ckpt_path and self.epochs > 0 else None,
        )

        if checkpoint_callback is not None and checkpoint_callback.best_model_path:
            best_path = checkpoint_callback.best_model_path
            if self.verbose:
                print(f"Loaded best model from {best_path} with val_loss={checkpoint_callback.best_model_score}")
            self._model = TFTAdapter.load_from_checkpoint(best_path, tft=tft)

        self._train_dataset = full_dataset

        gc.collect()

        #return self
        try:
            y_train = train_df["target"].to_numpy(dtype=float)
            # robust percentiles — перестрахуемся от выносов: 1% и 99%
            self._cal_p_low = float(np.nanpercentile(y_train, 1))
            self._cal_p_high = float(np.nanpercentile(y_train, 99))
            # амплитуды «типичных пиков»
            top_mask = y_train >= self._cal_p_high
            bot_mask = y_train <= self._cal_p_low
            self._cal_mean_top = float(np.nanmean(y_train[top_mask])) if np.any(top_mask) else float(self.global_target_max)
            self._cal_mean_bot = float(np.nanmean(y_train[bot_mask])) if np.any(bot_mask) else float(self.global_target_min)
            # защита от вырождения
            if not np.isfinite(self._cal_mean_top): self._cal_mean_top = float(self.global_target_max)
            if not np.isfinite(self._cal_mean_bot): self._cal_mean_bot = float(self.global_target_min)
        except Exception:
            # безопасные фолбэки
            self._cal_p_low, self._cal_p_high = self.global_target_min, self.global_target_max
            self._cal_mean_top, self._cal_mean_bot = self.global_target_max, self.global_target_min
        # ----------------------------------------------
        return self

    def predict(self, X):
        if self._train_dataset is None or self._model is None:
            raise RuntimeError("Model is not fitted yet")
    
        # Full suppress context (no logs/output, including PL/Torch/PF/Seed/GPU/TPU/HPU messages)
        @contextlib.contextmanager
        def suppress_all():
            with open(os.devnull, "w") as devnull, contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
                # Подавление всех возможных логгеров
                root_logger = logging.getLogger()
                old_level = root_logger.level
                root_logger.setLevel(logging.CRITICAL + 1)  # Выше CRITICAL, чтобы ничего не логировалось
                
                # Специфические логгеры (расширенный список)
                for logger_name in [
                    "pytorch_lightning", "lightning.pytorch", "lightning", 
                    "torch", "pytorch_forecasting", "optuna", 
                    "sklearn", "joblib", "numpy", "pandas", 
                    "scipy", "matplotlib", "seaborn", "plotly", 
                    "shap", "statsmodels", "torchmetrics",
                    "lightning.pytorch.utilities.migration.utils",  # Для Attribute 'loss' warnings
                    "lightning.pytorch.utilities.migration", 
                    "lightning.pytorch.utilities", 
                    "lightning.pytorch"
                ]:
                    logging.getLogger(logger_name).setLevel(logging.CRITICAL + 1)
                
                # Подавление всех предупреждений (расширенный список)
                warnings.filterwarnings("ignore")
                warnings.filterwarnings("ignore", category=DeprecationWarning)
                warnings.filterwarnings("ignore", category=UserWarning)
                warnings.filterwarnings("ignore", category=RuntimeWarning)
                warnings.filterwarnings("ignore", category=FutureWarning)
                warnings.filterwarnings("ignore", message=".*Attribute 'loss' is an instance of nn.Module.*")
                warnings.filterwarnings("ignore", message=".*Attribute 'logging_metrics' is an instance of nn.Module.*")
                warnings.filterwarnings("ignore", message=".*GPU available.*")
                warnings.filterwarnings("ignore", message=".*TPU available.*")
                warnings.filterwarnings("ignore", message=".*HPU available.*")
                warnings.filterwarnings("ignore", message=".*This Pipeline instance is not fitted yet.*")
                warnings.filterwarnings("ignore", message=".*Using an existing study with name.*")
                warnings.filterwarnings("ignore", message=".*A value is trying to be set on a copy of a slice.*")
                warnings.filterwarnings("ignore", message=".*The behavior of DataFrame concatenation.*")
                warnings.filterwarnings("ignore", message=".*torch.utils.checkpoint: the use_reentrant parameter.*")
                
                # Окружение (расширенное)
                os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
                os.environ['LITMODELS_DISABLE_TIP'] = '1'
                os.environ['HYDRA_FULL_ERROR'] = '1'
                os.environ['TQDM_DISABLE'] = '0'  # Не подавлять tqdm (если вызван снаружи)
                os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Для синхронизации CUDA, но без логов
                os.environ['TORCH_USE_DETERMINISTIC_ALGORITHMS'] = '1'  # Без логов
                
                try:
                    yield
                finally:
                    root_logger.setLevel(old_level)
                    warnings.resetwarnings()
    
        with suppress_all():
            set_seeds(self.seed)
            pl.seed_everything(self.seed, verbose=False, workers=True)
    
            if self.preprocessing is not None and not isinstance(X, np.ndarray):
                X = self.preprocessing.transform(X)
    
            X = np.asarray(X)
            N = X.shape[0]
            if N < self.min_encoder_length:
                return [float(np.nan)] * N
    
            feat, _, time_raw = split_features_batch_time(X, self._n_transformed)
            df = pd.DataFrame(feat, columns=self._feature_columns)
            df["__row_id"] = np.arange(N)
            df["time_raw"] = pd.to_datetime(time_raw, utc=True, errors="coerce")
            df["batch"] = np.int64(0)
            df["target"] = self.global_target_mean
    
            df = df.dropna(subset=["time_raw"]).reset_index(drop=True)
            df_sorted = df.sort_values("time_raw").reset_index(drop=True)
            df_sorted["time_idx"] = np.arange(len(df_sorted), dtype=np.int64)
    
            eff_N = len(df_sorted)
            if eff_N < self.min_encoder_length:
                out = np.full(N, np.nan, dtype=float)
                return out.tolist()
    
            step_ns = int(60 * 1e9)
            if eff_N >= 2:
                diffs = df_sorted["time_raw"].view("int64").astype("int64").to_numpy()
                diffs = np.diff(diffs)
                step_ns = int(np.nan_to_num(np.median(diffs), nan=step_ns))
                if step_ns <= 0:
                    step_ns = int(60 * 1e9)
    
            out_raw = np.full(eff_N, np.nan, dtype=float)
    
            tft = self._model.tft
            orig_log = tft.log
            tft.log = lambda *args, **kwargs: None
    
            try:
                eff_encoder_len = min(self.seq_len, eff_N)
                M = max(0, eff_N - eff_encoder_len + 1)
                stride = self.pred_len  # Keep original stride to preserve logic
                window_starts = np.arange(0, M, stride)
                K = len(window_starts)
    
                if K == 0:
                    K = 1
                    window_starts = np.array([0])
                    eff_encoder_len = eff_N
    
                num_feat_cols = len(self._feature_columns)
    
                # Vectorized construction of all_feats
                enc_indices = window_starts[:, np.newaxis] + np.arange(eff_encoder_len)
                all_enc_feats = feat[enc_indices]  # (K, eff_encoder_len, num_feat_cols)
                last_enc_feats = all_enc_feats[:, -1, :]
                all_fut_feats = np.repeat(last_enc_feats[:, np.newaxis, :], self.pred_len, axis=1)  # (K, pred_len, num_feat_cols)
                all_feats = np.concatenate([all_enc_feats, all_fut_feats], axis=1).reshape(-1, num_feat_cols)
    
                # Vectorized all_time_idx
                orig_time_idx = df_sorted["time_idx"].values
                all_enc_time_idx = orig_time_idx[enc_indices]  # (K, eff_encoder_len)
                last_time_idx = all_enc_time_idx[:, -1]
                fut_offsets = np.arange(1, self.pred_len + 1)
                all_fut_time_idx = last_time_idx[:, np.newaxis] + fut_offsets  # (K, pred_len)
                all_time_idx = np.concatenate([all_enc_time_idx, all_fut_time_idx], axis=1).reshape(-1)
    
                # Vectorized all_batch
                batch_per_window = np.arange(K)[:, np.newaxis]
                all_enc_batch = np.repeat(batch_per_window, eff_encoder_len, axis=1)  # (K, eff_encoder_len)
                all_fut_batch = np.repeat(batch_per_window, self.pred_len, axis=1)  # (K, pred_len)
                all_batch = np.concatenate([all_enc_batch, all_fut_batch], axis=1).reshape(-1)
    
                # all_target (constant)
                all_target = np.full(K * (eff_encoder_len + self.pred_len), self.global_target_mean, dtype=np.float32)
    
                # Vectorized all_time_raw using int64 ns
                orig_time_raw_int = df_sorted["time_raw"].view("int64").values
                all_enc_time_int = orig_time_raw_int[enc_indices]  # (K, eff_encoder_len)
                last_time_int = all_enc_time_int[:, -1]
                fut_offsets_ns = fut_offsets * step_ns
                all_fut_time_int = last_time_int[:, np.newaxis] + fut_offsets_ns  # (K, pred_len)
                all_time_int_flat = np.concatenate([all_enc_time_int.reshape(-1), all_fut_time_int.reshape(-1)])
                all_time_raw = pd.to_datetime(all_time_int_flat, unit='ns', utc=True).values  # object array of Timestamp
    
                pred_df = pd.DataFrame(all_feats, columns=self._feature_columns)
                pred_df["time_idx"] = all_time_idx
                pred_df["batch"] = all_batch
                pred_df["target"] = all_target
                pred_df["time_raw"] = all_time_raw
    
                pred_df["batch"] = pred_df["batch"].astype("int64")
                pred_df["time_idx"] = pred_df["time_idx"].astype("int64")
                pred_df["target"] = pred_df["target"].astype("float32")
    
                sliding_dataset = TimeSeriesDataSet.from_dataset(
                    self._train_dataset,
                    pred_df,
                    predict=True,
                    stop_randomization=True,
                    min_prediction_length=self.pred_len,
                    max_prediction_length=self.pred_len,
                )
    
                test_dl = sliding_dataset.to_dataloader(
                    train=False,
                    batch_size=K,  # Full batch for max speed (K small due to stride=pred_len)
                    num_workers=0,
                    persistent_workers=False,
                    pin_memory=False,
                )
    
                preds = tft.predict(
                    test_dl,
                    mode="prediction",
                    return_x=False,
                    trainer_kwargs={
                        "logger": False, 
                        "enable_progress_bar": False, 
                        "enable_model_summary": False, 
                        "enable_checkpointing": False, 
                        "accelerator": "gpu" if self.device == "cuda" else "cpu"
                    },
                )
    
                if isinstance(preds, torch.Tensor):
                    preds = preds.detach().cpu()
                    if preds.dim() == 3 and preds.size(-1) == 1:
                        preds = preds.squeeze(-1)
                    if preds.dim() == 2:
                        yhat_stride = preds[:, 0].numpy()  # First step per window
                    elif preds.dim() == 1:
                        yhat_stride = preds.numpy()
                    elif preds.dim() == 0:
                        yhat_stride = np.array([preds.item()])
                    else:
                        yhat_stride = np.full(K, self.global_target_mean)
                else:
                    yhat_stride = np.full(K, self.global_target_mean)
    
                # Fill out_raw at window ends
                for j, i in enumerate(window_starts):
                    out_raw[i + eff_encoder_len - 1] = yhat_stride[j]
    
                # Causal interpolation for missed points (linear from past, no future look)
                s = pd.Series(out_raw)
                s = s.interpolate(method='linear', limit_direction='forward')  # Only forward for causality
                s = s.ffill()  # Fill initial with first pred
                out_raw = s.values
    
                # Global map to [-1,1] (consistent scale, preserves relative peaks)
                out = (out_raw - self.global_target_min) / self.global_range * 2 - 1
                out = np.tanh(out * self.soft_clip_scale) / np.tanh(self.soft_clip_scale)
                out = np.clip(out, -1.0, 1.0)
    
                # Fill any remaining nans (unlikely) with mapped mean
                nan_mask = np.isnan(out)
                mapped_mean = (self.global_target_mean - self.global_target_min) / self.global_range * 2 - 1
                out[nan_mask] = mapped_mean
    
            finally:
                tft.log = orig_log
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
    
            original_pos = df_sorted["__row_id"].to_numpy()
            out_final = np.full(N, np.nan, dtype=float)
            out_final[original_pos] = out
    
            return out_final.tolist()

    def get_params(self, deep=True):
        # Чтобы sklearn корректно передавал параметры в Pipeline
        return {
            "seq_len": self.seq_len,
            "pred_len": self.pred_len,
            "hidden_size": self.hidden_size,
            "epochs": self.epochs,
            "batch_size": self.batch_size,
            "learning_rate": self.learning_rate,
            "patience": self.patience,
            "seed": self.seed,
            "dropout": self._dropout,
            "weight_decay": self.weight_decay,
            "verbose": self.verbose,
            "ckpt_path": self.ckpt_path
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

def _tensor_state_dict_to(dtype: torch.dtype, state: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    out = {}
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            # важно: сохраняем в CPU и ровно в dtype (float32 для идентичности)
            out[k] = v.detach().to('cpu', dtype=dtype)
        else:
            out[k] = v
    return out


def _save_bytes_compressed_zip(file_path: str, bytes_map: Dict[str, bytes], compression=zipfile.ZIP_DEFLATED, compresslevel=9):
    with zipfile.ZipFile(file_path, mode='w', compression=compression, compresslevel=compresslevel) as zf:
        for name, b in bytes_map.items():
            zf.writestr(name, b)


def _load_bytes_from_zip(file_path: str) -> Dict[str, bytes]:
    out = {}
    with zipfile.ZipFile(file_path, mode='r') as zf:
        for name in zf.namelist():
            out[name] = zf.read(name)
    return out


def _safe_json_dump(obj: Dict[str, Any]) -> bytes:
    def to_builtin(x):
        import numpy as _np
        import pandas as _pd
        if isinstance(x, (_np.integer,)):
            return int(x)
        if isinstance(x, (_np.floating,)):
            return float(x)
        if isinstance(x, (_np.ndarray,)):
            return x.tolist()
        if isinstance(x, (_pd.Timestamp,)):
            return x.isoformat()
        return x

    def convert(v):
        if isinstance(v, dict):
            return {k: convert(val) for k, val in v.items()}
        if isinstance(v, (list, tuple)):
            return [convert(i) for i in v]
        return to_builtin(v)

    return json.dumps(convert(obj), ensure_ascii=False, separators=(",", ":")).encode("utf-8")


def _safe_json_load(b: bytes) -> Dict[str, Any]:
    return json.loads(b.decode("utf-8"))


def _extract_tsd_feature_lists(tds) -> Dict[str, Any]:
    def g(name, default=None):
        return getattr(tds, name, default)

    lists = {}
    for name in [
        "time_varying_known_reals",
        "time_varying_unknown_reals",
        "static_reals",
        "time_varying_known_categoricals",
        "time_varying_unknown_categoricals",
        "static_categoricals",
        "target_categoricals",
        "known_reals",
        "unknown_reals",
        "known_categoricals",
        "unknown_categoricals",
        "reals",
        "categoricals",
    ]:
        val = g(name, None)
        if val is not None:
            try:
                lists[name] = list(val)
            except Exception:
                pass
    return lists


def save_transformer(
    pipeline: Pipeline,
    path: str,
    *,
    # ЖЁСТКО: float32 по умолчанию для идентичности. Не выставляйте True, пока не сравните предикты.
    float16_weights: bool = False,
    preprocessing_filename: str = "preprocessing.joblib.lzma",
    model_zip_filename: str = "model_weights.zip",
    meta_json_filename: str = "meta.json"
):
    os.makedirs(path, exist_ok=True)

    if not isinstance(pipeline, Pipeline):
        raise TypeError("Expected sklearn Pipeline")

    steps_dict = dict(pipeline.named_steps)
    if "preprocessing" not in steps_dict or "model" not in steps_dict:
        raise ValueError("Pipeline must have 'preprocessing' and 'model' steps")

    preprocessing = steps_dict["preprocessing"]
    model: SequenceTransformerRegressor = steps_dict["model"]

    if model._model is None or model._train_dataset is None:
        raise RuntimeError("Model must be fitted before saving")

    # 1) preprocessing
    prep_path = os.path.join(path, preprocessing_filename)
    with lzma.open(prep_path, "wb", preset=9) as f:
        joblib.dump(preprocessing, f)

    # 2) параметры TDS
    tds = model._train_dataset
    def g(name, default=None):
        return getattr(tds, name, default)

    train_dataset_params = {
        "time_idx": g("time_idx", "time_idx"),
        "target": g("target", "target"),
        "group_ids": list(g("group_ids", ["batch"])),
        "max_encoder_length": int(g("max_encoder_length", model.seq_len)),
        "max_prediction_length": int(g("max_prediction_length", model.pred_len)),
        "target_normalizer": None,
        "allow_missing_timesteps": bool(g("allow_missing_timesteps", True)),
        "add_relative_time_idx": bool(g("add_relative_time_idx", True)),
        "add_target_scales": bool(g("add_target_scales", False)),
        "add_encoder_length": bool(g("add_encoder_length", True)),
        "min_prediction_length": int(g("min_prediction_length", 1)),
        "min_encoder_length": int(g("min_encoder_length", 0)),
    }
    feature_lists = _extract_tsd_feature_lists(tds)

    # 3) веса TFT — строго в float32 (если float16_weights=False)
    tft = model._model.tft
    state = tft.state_dict()
    dtype = torch.float16 if float16_weights else torch.float32
    state = _tensor_state_dict_to(dtype, state)
    buf = io.BytesIO()
    torch.save(state, buf, _use_new_zipfile_serialization=True)
    buf.seek(0)
    weights_zip_path = os.path.join(path, model_zip_filename)
    _save_bytes_compressed_zip(weights_zip_path, {"tft_state_dict.pt": buf.getvalue()})

    # 4) метаданные
    meta = {
        "class": "SequenceTransformerRegressor",
        "params": {
            "seq_len": model.seq_len,
            "pred_len": model.pred_len,
            "hidden_size": model.hidden_size,
            "hidden_continuous_size": model.hidden_continuous_size,
            "epochs": model.epochs,
            "batch_size": model.batch_size,
            "learning_rate": model.learning_rate,
            "patience": model.patience,
            "seed": model.seed,
            "dropout": model._dropout,
            "weight_decay": model.weight_decay,
            "verbose": model.verbose,
            "mask_prob": model.mask_prob,
            "infer_stride": model.infer_stride,
            "ckpt_path": None,
            "min_encoder_length": model.min_encoder_length,
            "lstm_layers": 3,
            "attention_head_size": 4,
            "output_size": 1,
        },
        "feature_columns": model._feature_columns,
        "n_transformed": model._n_transformed,
        "n_feat": model._n_feat,
        "global_stats": {
            "global_target_mean": model.global_target_mean,
            "global_target_std": model.global_target_std,
            "global_target_min": model.global_target_min,
            "global_target_max": model.global_target_max,
            "global_range": model.global_range,
            "soft_clip_scale": model.soft_clip_scale,
            "clip_scale": model.clip_scale,
        },
        "train_dataset_params": train_dataset_params,
        "feature_lists": feature_lists,
        "torch_dtype": "float16" if float16_weights else "float32",
        "preproc_feature_names_out": list(getattr(preprocessing, "get_feature_names_out", lambda: [])()),
    }

    meta_path = os.path.join(path, meta_json_filename)
    with open(meta_path, "wb") as f:
        f.write(_safe_json_dump(meta))

    weights_size_mb = os.path.getsize(weights_zip_path) / (1024 * 1024)
    prep_size_mb = os.path.getsize(prep_path) / (1024 * 1024)
    meta_size_kb = os.path.getsize(meta_path) / 1024.0
    print(f"Saved: weights={weights_size_mb:.2f} MB, preprocessing={prep_size_mb:.2f} MB, meta={meta_size_kb:.1f} KB → dir={path}")

# Требуется наличие в окружении:
# - SequenceTransformerRegressor
# - TimeSeriesDataSet
# - CustomTFT, TFTAdapter
# - tft_output_transformer
# - PeakFriendlyHuber
# - to_dense
# - вспомогательные функции _load_bytes_from_zip, _safe_json_load из предыдущей версии


def load_transformer_exact(
    path: str,
    *,
    preprocessing_filename: str = "preprocessing.joblib.lzma",
    model_zip_filename: str = "model_weights.zip",
    meta_json_filename: str = "meta.json",
    device: Optional[str] = None,
) -> Pipeline:
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

    warnings.filterwarnings("ignore", message=".*Attribute 'loss' is an instance of `nn.Module`.*")
    warnings.filterwarnings("ignore", message=".*Attribute 'logging_metrics' is an instance of `nn.Module`.*")

    # 1) preprocessing
    prep_path = os.path.join(path, preprocessing_filename)
    with lzma.open(prep_path, "rb") as f:
        preprocessing = joblib.load(f)

    # 2) meta
    meta_path = os.path.join(path, meta_json_filename)
    if not os.path.isfile(meta_path):
        raise FileNotFoundError(meta_path)
    with open(meta_path, "rb") as f:
        meta = json.loads(f.read().decode("utf-8"))

    params = meta["params"]
    feature_columns = meta["feature_columns"]
    n_transformed = meta["n_transformed"]
    n_feat = meta["n_feat"]
    global_stats = meta["global_stats"]
    tds_params = meta["train_dataset_params"]
    feature_lists = meta.get("feature_lists", {})
    saved_torch_dtype = meta.get("torch_dtype", "float32")

    # 3) регрессор
    model = SequenceTransformerRegressor(
        seq_len=params["seq_len"],
        pred_len=params["pred_len"],
        hidden_size=params["hidden_size"],
        hidden_continuous_size=params["hidden_continuous_size"],
        epochs=params["epochs"],
        batch_size=params["batch_size"],
        learning_rate=params["learning_rate"],
        patience=params["patience"],
        seed=params["seed"],
        dropout=params["dropout"],
        weight_decay=params["weight_decay"],
        verbose=params["verbose"],
        mask_prob=params["mask_prob"],
        infer_stride=params["infer_stride"],
        ckpt_path=None,
        preprocessing=preprocessing,
        min_encoder_length=params.get("min_encoder_length", 1),
    )

    model._feature_columns = feature_columns
    model._n_transformed = n_transformed
    model._n_feat = n_feat
    model.global_target_mean = float(global_stats["global_target_mean"])
    model.global_target_std = float(global_stats["global_target_std"])
    model.global_target_min = float(global_stats["global_target_min"])
    model.global_target_max = float(global_stats["global_target_max"])
    model.global_range = float(global_stats["global_range"])
    model.soft_clip_scale = float(global_stats["soft_clip_scale"])
    model.clip_scale = float(global_stats.get("clip_scale", 1.5))
    model.device = device

    # 4) синтетический df
    max_enc = int(tds_params["max_encoder_length"])
    max_pred = int(tds_params["max_prediction_length"])
    num_feat_cols = len(feature_columns)
    rows = max_enc + max_pred

    synth_df_base = pd.DataFrame(
        np.zeros((rows, num_feat_cols), dtype=np.float32),
        columns=feature_columns,
    )
    synth_df_base["time_idx"] = np.arange(rows, dtype=np.int64)
    synth_df_base["target"] = np.zeros(rows, dtype=np.float32)
    synth_df_base["batch"] = 0

    # Попытка №1: «обычная» конструкция TDS как при обучении
    tds_kwargs = dict(
        time_idx=tds_params["time_idx"],
        target=tds_params["target"],
        group_ids=tds_params["group_ids"],
        max_encoder_length=tds_params["max_encoder_length"],
        max_prediction_length=tds_params["max_prediction_length"],
        target_normalizer=None,
        allow_missing_timesteps=tds_params["allow_missing_timesteps"],
        add_relative_time_idx=tds_params["add_relative_time_idx"],
        add_target_scales=tds_params["add_target_scales"],
        add_encoder_length=tds_params["add_encoder_length"],
        min_prediction_length=tds_params["min_prediction_length"],
        min_encoder_length=tds_params.get("min_encoder_length", 0),
    )

    if "time_varying_unknown_reals" in feature_lists:
        tds_kwargs["time_varying_unknown_reals"] = list(feature_lists["time_varying_unknown_reals"])
    else:
        tds_kwargs["time_varying_unknown_reals"] = list(feature_columns)

    for key in [
        "time_varying_known_reals",
        "static_reals",
        "time_varying_known_categoricals",
        "time_varying_unknown_categoricals",
        "static_categoricals",
        "known_reals",
        "unknown_reals",
        "known_categoricals",
        "unknown_categoricals",
    ]:
        if key in feature_lists:
            tds_kwargs[key] = list(feature_lists[key])

    dataset = TimeSeriesDataSet(synth_df_base.copy(), **tds_kwargs)

    # Проверяем порядок reals
    saved_reals = feature_lists.get("reals")
    rebuild_forced = False
    if saved_reals is not None:
        # В PF список dataset.reals может быть кортежами/объектами -- приводим к строкам
        ds_reals = list(getattr(dataset, "reals", []))
        ds_reals = [str(x) for x in ds_reals]
        saved_reals_str = [str(x) for x in saved_reals]
        if ds_reals != saved_reals_str:
            rebuild_forced = True

    if rebuild_forced:
        # Попытка №2: Форсируем порядок каналов reals.
        # Для этого отключим автоматические добавления и сами сгенерируем служебные колонки
        # encoder_length и relative_time_idx в synth_df.
        synth_df = synth_df_base.copy()
        # relative_time_idx: от 0 до rows-1
        synth_df["relative_time_idx"] = np.arange(rows, dtype=np.int64)
        # encoder_length: длина encoder для каждой позиции; для синтетики можно установить константу max_enc
        # PF ожидает целочисленную encoder_length, соответствующую длине энкодера на каждом шаге.
        # Для инициализации архитектуры достаточно положить валидные числа.
        enc_len = np.zeros(rows, dtype=np.int64)
        enc_len[:max_enc] = np.arange(1, max_enc + 1, dtype=np.int64)
        enc_len[max_enc:] = max_enc
        synth_df["encoder_length"] = enc_len

        # Теперь задаём TDS без add_* фичей, и передаём time_varying_unknown_reals в порядке saved_reals.
        # saved_reals начинается с ["encoder_length","relative_time_idx", ... f0..fN]
        tds_kwargs_forced = dict(
            time_idx=tds_params["time_idx"],
            target=tds_params["target"],
            group_ids=tds_params["group_ids"],
            max_encoder_length=tds_params["max_encoder_length"],
            max_prediction_length=tds_params["max_prediction_length"],
            target_normalizer=None,
            allow_missing_timesteps=tds_params["allow_missing_timesteps"],
            add_relative_time_idx=False,
            add_target_scales=tds_params["add_target_scales"],
            add_encoder_length=False,
            min_prediction_length=tds_params["min_prediction_length"],
            min_encoder_length=tds_params.get("min_encoder_length", 0),
            time_varying_unknown_reals=list(saved_reals),  # порядок каналов фиксируем здесь
            # Не задаём known/unknown cats/reals дополнительно, чтобы не нарушить порядок
        )
        dataset = TimeSeriesDataSet(synth_df, **tds_kwargs_forced)

        # Контроль: проверим снова порядок
        ds_reals2 = [str(x) for x in list(getattr(dataset, "reals", []))]
        if ds_reals2 != [str(x) for x in saved_reals]:
            raise RuntimeError(
                f"Failed to force reals order. Got {ds_reals2[:8]}..., expected {list(saved_reals)[:8]}..."
            )

    # 5) TFT 1-в-1
    tft = CustomTFT.from_dataset(
        dataset,
        hidden_size=int(model.hidden_size),
        output_size=int(params.get("output_size", 1)),
        loss=PeakFriendlyHuber(
            delta=0.5,
            peak_thr=0.85,
            peak_weight=1.3,
            contrast_weight=0.03,
            center_band=0.3,
            clip_scale=1.5,
        ),
        optimizer="adam",
        learning_rate=float(model.learning_rate),
        lstm_layers=int(params.get("lstm_layers", 3)),
        hidden_continuous_size=int(model.hidden_continuous_size),
        attention_head_size=int(params.get("attention_head_size", 4)),
        dropout=float(model._dropout),
        reduce_on_plateau_patience=5,
        reduce_on_plateau_min_lr=1e-6,
        weight_decay=float(model.weight_decay),
        mask_prob=float(model.mask_prob),
        output_transformer=tft_output_transformer,
    )

    model._model = TFTAdapter(tft)
    model._train_dataset = dataset

    # 6) загрузка весов
    weights_zip_path = os.path.join(path, model_zip_filename)
    with zipfile.ZipFile(weights_zip_path, mode='r') as zf:
        if "tft_state_dict.pt" not in zf.namelist():
            raise RuntimeError("tft_state_dict.pt not found in weights zip")
        state = torch.load(io.BytesIO(zf.read("tft_state_dict.pt")), map_location="cpu")

    missing, unexpected = tft.load_state_dict(state, strict=False)
    if missing or unexpected:
        warnings.warn(f"load_state_dict: missing={missing}, unexpected={unexpected}")

    # 7) eval + dropout off
    tft.eval()
    for m in tft.modules():
        if isinstance(m, torch.nn.Dropout):
            m.p = 0.0

    model._model.to(device)

    restored = Pipeline([
        ("preprocessing", preprocessing),
        ("to_dense", FunctionTransformer(to_dense)),
        ("model", model),
    ])
    return restored

In [2]:
ticker = 'OZPH'

with open('final_cols.pkl', 'rb') as f:
    final_cols = pickle.load(f)

with open('final_params.pkl', 'rb') as f:
    final_params = pickle.load(f)

warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*DataLoader will create.*")
    
path = r'C:\Users\aleksandrovva1\Desktop\data science\0-trade\t\data_long' #test_files_15_2  data_long
file_name = [i for i in os.listdir(r'C:\Users\aleksandrovva1\Desktop\data science\0-trade\t\data_long') if ticker == i.split('_')[0]][0]
with open('phase_ful_tickers_params.txt', 'r') as file:
    phase_df = json.load(file)
df_phase = pd.read_parquet(os.path.join(path, file_name))
window = int(phase_df[ticker]['params']['moving_average_length']*9.5)
features = extract_features(df_phase, window=window)
scaled = joblib.load("scaler_global.pkl").transform(features)
labels = joblib.load("kmeans_global.pkl").predict(scaled)

regime_series = pd.Series(labels, index=df_phase.index)
window_size = int(phase_df[ticker]['params']['atr_period']*5.5)

smoother = FastRollingMode(window_size=window_size)
smoothed = [smoother.update(x) for x in labels]
smoothed_regime = pd.Series(smoothed, index=df_phase.index)

regime_params = prepare_regime_params(phase_df[ticker]['params'])

CV = AdaptiveTradingSystem(regime_params['base_params'])

pipeline_trans = load_transformer_exact(f'C:/Users/aleksandrovva1/Desktop/data science/0-trade/t/transformers/{ticker}', device='cpu')    

df_phase = CV.generate_adaptive_signals(df_phase, regime_series=smoothed_regime)

buy_signals = df_phase[df_phase['buy_signal']]
sell_signals = df_phase[df_phase['sell_signal']]
for _, buy in buy_signals.iterrows():
    sell = sell_signals[sell_signals.time > buy.time].head(1)
    if not sell.empty:
        df_phase.loc[buy.name, "event_time"] = buy.time
        df_phase.loc[buy.name, "event_price"] = buy.close
        df_phase.loc[buy.name, "event_sell_time"] = sell.time.values[0]
        df_phase.loc[buy.name, "event_sell_price"] = sell.close.values[0]

df_phase['pnl'] = ((df_phase['event_sell_price'] * (1 - 0.003)) / (df_phase['event_price'] * (1 + 0.003)) - 1) * 100
df_phase['regime'] = smoothed_regime
df_phase = calculate_target(df_phase, threshold=1.9)
df_phase = calculate_smoothed_target_qnorm(df_phase, 
                                       smooth_method='whittaker', whittaker_lambda=10, savgol_window=15, savgol_poly=3, 
                                       per_batch_equalize=True, per_batch_q=0.01
                                      )
db_path_phase = f"sqlite:///C:/Users/aleksandrovva1/Desktop/data science/0-trade/t/out_try/{ticker}.db"
'''study_phase = optuna.create_study(study_name=f'feature_hyperparam_search_{ticker}', directions=['minimize', 'minimize', 'minimize', 'maximize', 'minimize'], storage=db_path_phase, load_if_exists=True)
directions = ['minimize', 'minimize', 'minimize', 'maximize', 'minimize']

best_score, best_num, best_params, best_values, best_norm = \
    find_best_trial_equal_importance(study_phase.trials, directions)
combined_ml_df, timings = calculate_indicators(df_phase, final_cols[ticker], params=build_feature_params(final_params[ticker]), multy=False)
TGT_COL = ['normalized_target']
unique_batches = combined_ml_df['batch'].unique()
columns_for_model = final_cols[ticker] + TGT_COL + ['batch', 'regime'] + ['time']
combined_ml_df['predd'] = np.nan
for i in tqdm(combined_ml_df.batch.unique()):
    index = combined_ml_df[combined_ml_df['batch']==i].index
    gh_test = combined_ml_df[columns_for_model][combined_ml_df[columns_for_model]['batch'] == i]
    X_test, _, _, _, _ = prepare_data_transformer(gh_test, 'normalized_target')
    
    # Локальный перехват предупреждений от pipeline
    with warnings.catch_warnings(record=True) as caught_warnings:
        warnings.simplefilter("always")  # Чтобы catch_warnings перехватывал их
        combined_ml_df.loc[index, 'predd'] = pipeline_trans.predict(X_test)
    
    # Игнорируем только релевантные предупреждения (опционально, для чистоты)
    for warning in caught_warnings:
        if "Pipeline instance is not fitted yet" in str(warning.message):
            continue  # Просто пропускаем (не показываем)
    
    # Остальной код (shift, pct_change и т.д.) без изменений
    combined_ml_df.loc[index, 'predd_shift_5'] = combined_ml_df.loc[index]['predd'].shift(5)
    combined_ml_df.loc[index, 'predd_pct'] = combined_ml_df.loc[index]['predd'].pct_change(3)
    combined_ml_df.loc[index, 'predd_var'] = combined_ml_df.loc[index]['predd'].rolling(10).var()'''

'study_phase = optuna.create_study(study_name=f\'feature_hyperparam_search_{ticker}\', directions=[\'minimize\', \'minimize\', \'minimize\', \'maximize\', \'minimize\'], storage=db_path_phase, load_if_exists=True)\ndirections = [\'minimize\', \'minimize\', \'minimize\', \'maximize\', \'minimize\']\n\nbest_score, best_num, best_params, best_values, best_norm =     find_best_trial_equal_importance(study_phase.trials, directions)\ncombined_ml_df, timings = calculate_indicators(df_phase, final_cols[ticker], params=build_feature_params(final_params[ticker]), multy=False)\nTGT_COL = [\'normalized_target\']\nunique_batches = combined_ml_df[\'batch\'].unique()\ncolumns_for_model = final_cols[ticker] + TGT_COL + [\'batch\', \'regime\'] + [\'time\']\ncombined_ml_df[\'predd\'] = np.nan\nfor i in tqdm(combined_ml_df.batch.unique()):\n    index = combined_ml_df[combined_ml_df[\'batch\']==i].index\n    gh_test = combined_ml_df[columns_for_model][combined_ml_df[columns_for_model][\'batch\'] == i]\n  

In [3]:
with open('final_cols.pkl', 'rb') as f:
    final_cols = pickle.load(f)

with open('final_params.pkl', 'rb') as f:
    final_params = pickle.load(f)

def optimize_for_ticker(df_path, db_path, study_name, ticker):
  
    path = r'C:\Users\aleksandrovva1\Desktop\data science\0-trade\t\data_long' #test_files_15_2  data_long
    with open('phase_ful_tickers_params.txt', 'r') as file:
        phase_df = json.load(file)
    df_phase = pd.read_parquet(os.path.join(path, df_path))
    window = int(phase_df[ticker]['params']['moving_average_length']*9.5)
    features = extract_features(df_phase, window=window)
    scaled = joblib.load("scaler_global.pkl").transform(features)
    labels = joblib.load("kmeans_global.pkl").predict(scaled)
    
    regime_series = pd.Series(labels, index=df_phase.index)
    window_size = int(phase_df[ticker]['params']['atr_period']*5.5)
    
    smoother = FastRollingMode(window_size=window_size)
    smoothed = [smoother.update(x) for x in labels]
    smoothed_regime = pd.Series(smoothed, index=df_phase.index)
    
    regime_params = prepare_regime_params(phase_df[ticker]['params'])
    
    CV = AdaptiveTradingSystem(regime_params['base_params'])

    pipeline_trans = load_transformer_exact(f'C:/Users/aleksandrovva1/Desktop/data science/0-trade/t/transformers/{ticker}', device='cpu')    
    
    df_phase = CV.generate_adaptive_signals(df_phase, regime_series=smoothed_regime)
    
    buy_signals = df_phase[df_phase['buy_signal']]
    sell_signals = df_phase[df_phase['sell_signal']]
    for _, buy in buy_signals.iterrows():
        sell = sell_signals[sell_signals.time > buy.time].head(1)
        if not sell.empty:
            df_phase.loc[buy.name, "event_time"] = buy.time
            df_phase.loc[buy.name, "event_price"] = buy.close
            df_phase.loc[buy.name, "event_sell_time"] = sell.time.values[0]
            df_phase.loc[buy.name, "event_sell_price"] = sell.close.values[0]
    
    df_phase['pnl'] = ((df_phase['event_sell_price'] * (1 - 0.003)) / (df_phase['event_price'] * (1 + 0.003)) - 1) * 100
    df_phase['regime'] = smoothed_regime
    df_phase = calculate_target(df_phase, threshold=1.9)
    df_phase = calculate_smoothed_target_qnorm(df_phase, 
                                           smooth_method='whittaker', whittaker_lambda=10, savgol_window=15, savgol_poly=3, 
                                           per_batch_equalize=True, per_batch_q=0.01
                                          )
    db_path_phase = f"sqlite:///C:/Users/aleksandrovva1/Desktop/data science/0-trade/t/out_try/{ticker}.db"
    study_phase = optuna.create_study(study_name=f'feature_hyperparam_search_{ticker}', directions=['minimize', 'minimize', 'minimize', 'maximize', 'minimize'], storage=db_path_phase, load_if_exists=True)
    directions = ['minimize', 'minimize', 'minimize', 'maximize', 'minimize']

    best_score, best_num, best_params, best_values, best_norm = \
        find_best_trial_equal_importance(study_phase.trials, directions)
    combined_ml_df, timings = calculate_indicators(df_phase, final_cols[ticker], params=build_feature_params(final_params[ticker]), multy=False)

    TGT_COL = ['normalized_target']
    unique_batches = combined_ml_df['batch'].unique()
    columns_for_model = final_cols[ticker] + TGT_COL + ['batch', 'regime'] + ['time']

    if combined_ml_df.isna().sum().any() == True:
        nan_batch = combined_ml_df[combined_ml_df.isna().any(axis=1)]['batch'].unique()
        combined_ml_df = combined_ml_df[~combined_ml_df['batch'].isin(nan_batch)]
    combined_ml_df['predd'] = np.nan
            
    for i in combined_ml_df.batch.unique():
        index = combined_ml_df[combined_ml_df['batch']==i].index
        gh_test = combined_ml_df[columns_for_model][combined_ml_df[columns_for_model]['batch'] == i]
        X_test, _, _, _, _ = prepare_data_transformer(gh_test, 'normalized_target')
        
        # Локальный перехват предупреждений от pipeline
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")  # Чтобы catch_warnings перехватывал их
            combined_ml_df.loc[index, 'predd'] = pipeline_trans.predict(X_test)
        
        # Игнорируем только релевантные предупреждения (опционально, для чистоты)
        for warning in caught_warnings:
            if "Pipeline instance is not fitted yet" in str(warning.message):
                continue  # Просто пропускаем (не показываем)
        
        # Остальной код (shift, pct_change и т.д.) без изменений
        combined_ml_df.loc[index, 'predd_shift_5'] = combined_ml_df.loc[index]['predd'].shift(5)
        combined_ml_df.loc[index, 'predd_pct'] = combined_ml_df.loc[index]['predd'].pct_change(3)
        combined_ml_df.loc[index, 'predd_var'] = combined_ml_df.loc[index]['predd'].rolling(10).var()

    def make_batch_bins(df, target_col, batch_col, K):
        """
        Для каждого batch отдельно строим K бинов от min до max и возвращаем Series[0..K-1].
        """
        def bin_series(s):
            if s.max()==s.min():
                return pd.Series(np.zeros(len(s),dtype=int), index=s.index)
            # K равных интервалов между [min,max]
            bins = np.linspace(s.min(), s.max(), K+1)
            return pd.cut(s, bins=bins, labels=False, include_lowest=True)
        return df.groupby(batch_col)[target_col].apply(bin_series)
    
    def make_group(batches):
        vals, idx = np.unique(batches, return_index=True)
        order = np.argsort(idx)
        vals  = vals[order]
        return batches.value_counts().reindex(vals).to_numpy()
    
    def pred_converter(y_vl_bins):
        indices = y_vl_bins.index.get_level_values(1).values
        values = y_vl_bins.values
        return pd.Series(values, index=indices)
    
    def objective(trial):
        K = 15  # Фиксированное K для бинов
    
        # Гиперпараметры для LGBMRanker
        params = {
            'objective': 'lambdarank',
            'metric': 'ndcg',
            'ndcg_eval_at': [3],
            'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 3000),
            'max_depth': trial.suggest_int('max_depth', 3, 13),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('lgbm_reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('lgbm_reg_lambda', 0.0, 10.0),
            'n_estimators': trial.suggest_int('n_estimators', 50, 6000),
            'random_state': 42,
            'verbosity': -1
        }
    
        # Кросс-валидация
        ndcg_scores, r2_scores, corr_scores = [], [], []
    
        TGT_COLS = ['normalized_target']
        unique_batches = combined_ml_df['batch'].unique()
        columns_for_model = final_cols[ticker] + TGT_COLS + ['batch', 'regime', 'predd', 'predd_var', 'predd_shift_5', 'predd_pct']
        combined_ml_opt = combined_ml_df[columns_for_model].copy()
    
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        for train_idx, test_idx in kf.split(combined_ml_df['batch'].unique()):
            train_batches = combined_ml_opt['batch'].unique()[train_idx]
            test_batches = combined_ml_opt['batch'].unique()[test_idx]
    
            train_data = combined_ml_opt[combined_ml_opt['batch'].isin(train_batches)].reset_index(drop=True)
            test_data = combined_ml_opt[combined_ml_opt['batch'].isin(test_batches)].reset_index(drop=True)
    
            # Подготовка данных
            X_train, y_train, preprocessing = prepare_data(train_data, 'normalized_target')
            X_test, y_test, _ = prepare_data(test_data, 'normalized_target')
    
            # Бины для ранжирования
            y_tr_bins = make_batch_bins(train_data, 'normalized_target', 'batch', K)
            y_vl_bins = make_batch_bins(test_data, 'normalized_target', 'batch', K)
    
            # Группы для LGBMRanker
            grp_tr = make_group(train_data['batch'])
            grp_vl = make_group(test_data['batch'])
    
            pipeline = Pipeline([
                ('preprocessing', preprocessing),
                ('to_dense', FunctionTransformer(to_dense)),  # Преобразование в плотную матрицу
                ('model', LGBMRanker(**params))
            ])
    
            # Предварительно трансформируем X_train и X_test
            X_tr_t = pipeline.named_steps['to_dense'].transform(pipeline.named_steps['preprocessing'].fit_transform(X_train, y_tr_bins))
            X_vl_t = pipeline.named_steps['to_dense'].transform(pipeline.named_steps['preprocessing'].transform(X_test))
    
            # Фитим ranker вручную
            pipeline.named_steps['model'].fit(
                X_tr_t, y_tr_bins,
                group=grp_tr,
                eval_set=[(X_vl_t, y_vl_bins)],
                eval_group=[grp_vl],
                eval_at=[3],
            )
    
            # Предсказание
            y_pred_val = pipeline.named_steps['model'].predict(X_vl_t)
            y_pred_s = pd.Series(y_pred_val, index=test_data.index)
    
            # Вычисление NDCG@3 по батчам
            ndcgs = []
            for b in test_batches:
                mask = (test_data['batch'] == b)
                true_labels = pred_converter(y_vl_bins)[mask].to_numpy()
                preds = y_pred_s[mask].to_numpy()
                k = min(3, len(preds))
                #print(preds, k)
                
                if k >= 1:  # Add this check to skip empty batches
                    ndcgs.append(ndcg_score([true_labels], [preds], k=k))
                else:
                    # Optional: Append a default value like 0 if you want to penalize empty batches explicitly
                    # ndcgs.append(0.0)
                    # Or log a warning: print(f"Skipping empty batch {b} with {len(preds)} points")
                    pass  # Skipping is fine; average will ignore it
            
            avg_ndcg = np.mean(ndcgs) if ndcgs else 0.0  # Fallback to 0 if all batches are empty (or keep your inf logic)
    
            # Для сравнения: R2 и корреляция на bin-labels vs scores
            true_bins_all = pred_converter(y_vl_bins).to_numpy()
            r2_scores.append(r2_score(true_bins_all, y_pred_val))
            corr = pearsonr(true_bins_all, y_pred_val)[0]
            corr_scores.append(corr if not np.isnan(corr) else 0.0)
    
            ndcg_scores.append(avg_ndcg)
    
        if not ndcg_scores:
            return float('inf'), float('inf'), float('inf'), float('inf'), float('inf')
    
        if corr_scores == []:
            return float('inf'), float('inf'), float('inf'), float('inf'), float('inf')
    
        avg_ndcg = float(np.mean(ndcg_scores))
        avg_r2 = float(np.mean(r2_scores))      # we want it *higher*
        std_r2 = float(np.std(r2_scores))
        corr_mean = float(np.mean(corr_scores))
        corr_std = float(np.std(corr_scores))
    
        if np.isfinite([avg_ndcg, avg_r2, std_r2, corr_mean, corr_std]).all() == True:
            return -avg_ndcg, -avg_r2, std_r2, corr_mean, corr_std  # Primary: maximize NDCG (negative for minimize)
        else:
            return float('inf'), float('inf'), float('inf'), float('inf'), float('inf')

          
    study = optuna.create_study(
          study_name=study_name,
          directions=['minimize', 'minimize', 'minimize', 'maximize', 'minimize'],
          storage=db_path,
          load_if_exists=True
      )
    study.optimize(
          objective,
          n_trials=200 - len([trial for trial in study.trials if trial.values is not None]),
          n_jobs=3,
          show_progress_bar=False
      )
    
    return study

In [4]:
filtered_index = list(phase_df.keys())
canceled_keys = [i for i in filtered_index if i not in [f.split('.')[0] for f in os.listdir(r'C:\Users\aleksandrovva1\Desktop\data science\0-trade\t\lgbmranker_params')]]

# Преобразуем в список и перемешиваем для более равномерного распределения
tickers_list = np.random.permutation(canceled_keys)

# Разделяем на 3 части
split_indices = np.array_split(tickers_list, 4)

# Получаем три группы
group1 = split_indices[0].tolist()
group2 = split_indices[1].tolist()
group3 = split_indices[2].tolist()
group4 = split_indices[3].tolist()

In [5]:
group1 = ['VKCO',
 'LNZL',
 'NMTP',
 'SVAV',
 #'CNTLP',
 'MRKC',
 #'FESH',
 #'BANEP',
 'RBCM',
 'RENI',
 'IRKT',
 'PRFN',
 'PLZL',
 'TGKB',
 #'HEAD',
 'X5',
 'NTZL',
 'MAGN',
 'KZIZ',
 'KLSB',
 'PMSBP',
 'MDMG']

tickers = []
for i in tqdm([i for i in os.listdir(r'C:\Users\aleksandrovva1\Desktop\data science\0-trade\t\data_long')]): #test_files_15_2  data_long
    if i.split('_')[0] in group1:
        tickers.append((i.replace("'", "").split('_')[0], i.replace("'", "").replace(",", ""), i.replace("'", "").split('_')[0]+'.db'))
for ticks in tqdm(tickers):
    db_path = f"sqlite:///C:/Users/aleksandrovva1/Desktop/data science/0-trade/t/lgbmranker_params/{ticks[2]}"
    study_name=f'feature_hyperparam_search_{ticks[0]}'
    optimize_for_ticker(ticks[1], db_path, study_name, ticks[0])

  0%|          | 0/166 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

[I 2025-10-01 14:07:53,111] Using an existing study with name 'feature_hyperparam_search_IRKT' instead of creating a new one.


In [6]:
ticks

('IRKT', 'IRKT_industrials_CANDLE_INTERVAL_15_MIN.pq', 'IRKT.db')

In [15]:
group1 = ['NVTK',
 'TATN',
 'BSPB',
 'MRKU',
 'SELG',
 'OZPH',
 'ETLN',
 'LKOH',
 'MGTSP',
 'FRHC',
 'UWGN',
 'ABRD',
 'RKKE',
 'AFKS',
 #'OBNEP',
 'YDEX',
 'CNRU',
 'SBER',
 'UGLD',
 'KROT',
 'PMSB',
 'TGKBP',
 'LSRG']

group2 = ['NLMK',
 #'GECO',
 'MGNT',
 'NKHP',
 'SBERP',
 'RNFT',
 'LNZLP',
 'LSNG',
 #'KZIZP',
 'OBNE',
 'TTLK',
 'TGKJ',
 'BANE',
 'SNGSP',
 'LSNGP',
 #'MRKZ',
 'VRSB',
 'MRKV',
 'SPBE',
 'TRNFP',
 'MSTT',
 'DVEC']

group3 = ['OKEY',
 'MTLRP',
 'KZOSP',
 'LIFE',
 'UNAC',
 'UPRO',
 #'FLOT',
 #'GEMC',
 'MRKP',
 'TGKN',
 'NOMP',
 'NSVZ',
 'RAGR',
 'LENT',
 'TATNP',
 'AFLT',
 'TRMK',
 'ROSN',
 'MTLR',
 'MOEX',
 #'GTRK',
 'CHMK']

grop = group1 + group2 + group3


for i in os.listdir(r'C:\Users\aleksandrovva1\Desktop\data science\0-trade\t\lgbmregressor_params'):
    
    if i in os.listdir(r'C:\Users\aleksandrovva1\Desktop\data science\0-trade\t\lgbmranker_params'):
        pass
    elif i.split('.')[0] in grop:
        print(i)

SNGSP.db
SPBE.db
TATN.db
TGKBP.db
TGKJ.db
TGKN.db
TRMK.db
TRNFP.db
TTLK.db
UGLD.db
UNAC.db
UPRO.db
UWGN.db
VRSB.db
YDEX.db


In [ ]:
from scipy.stats import pearsonr
def prepare_data(df, target_col):
    """
    Подготавливает данные: разделяет на числовые и категориальные признаки, создает конвейер преобразования.
    """
    if type(target_col) == str:
        df.dropna(inplace=True)
        X = df.drop([target_col, 'batch'], axis=1)
        y = df[target_col]
    elif type(target_col) == list:
        df.dropna(inplace=True)
        X = df.drop(target_col+['batch'], axis=1)
        y = df[target_col]

    # Разделение на числовые и категориальные признаки
    numeric_features = X.select_dtypes(include=['int64', 'float64', 'float32', 'int32']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    # Создание конвейера преобразования
    preprocessing = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('scaler', RobustScaler()),
                ('normalize', PowerTransformer(method='yeo-johnson')),
            ]), numeric_features),
            ('cat', Pipeline([
                ('onehot', OneHotEncoder(handle_unknown='ignore')),
            ]), categorical_features)
        ]
    )

    return X, y, preprocessing
#TGT_COLS = ['normalized_target', 'multi_target_5', 'multi_target_10', 'multi_target_20']
TGT_COLS = ['normalized_target']
unique_batches = combined_ml_df['batch'].unique()
columns_for_model = final_cols[ticker] + TGT_COLS + ['batch', 'regime', 'predd', 'predd_var', 'predd_shift_5', 'predd_pct']
gh_t = combined_ml_df[columns_for_model].copy()

mse_scores, r2_scores, corr_scores = [], [], []

kf = KFold(n_splits=3, shuffle=True, random_state=42)

if len(TGT_COLS)!=1:
    for train_idx, test_idx in kf.split(gh_t['batch'].unique()):
        train_batches = gh_t['batch'].unique()[train_idx]
        test_batches = gh_t['batch'].unique()[test_idx]
    
        train_data = gh_t[gh_t['batch'].isin(train_batches)]
        test_data = gh_t[gh_t['batch'].isin(test_batches)]
    
        # Подготовка данных (как в вашем коде)
        X_train, y_train, preprocessing = prepare_data(train_data, TGT_COLS)
        X_test, y_test, _ = prepare_data(test_data, TGT_COLS)
    
        # Определение модели (как в вашем коде)
        base_model = LGBMRegressor(**study_model_base.trials[best_num_model].params, random_state=42, verbosity=-1)  # objective='quantile', alpha=0.25 если нужно
        #multi_model = MultiOutputRegressor(base_model, n_jobs=-1)
    
        # Пайплайн (как в вашем коде)
        pipeline_reg = Pipeline([
            ('preprocessing', preprocessing),
            ('to_dense', FunctionTransformer(to_dense)),  # Преобразование в плотную матрицу
            ('model', base_model)
        ])
    
        # Обучение
        pipeline_reg.fit(X_train, y_train)
    
        # Предсказания: y_pred — (n_samples, n_targets)
        y_pred = pipeline.predict(X_test)
    
        # Вычисление метрик (MSE и R2 — как в вашем коде, средние по горизонтам)
        mse_per_horizon = [mean_squared_error(y_test.to_numpy()[:, i], y_pred[:, i]) for i in range(y_test.shape[1])]
        r2_per_horizon = [r2_score(y_test.to_numpy()[:, i], y_pred[:, i]) for i in range(y_test.shape[1])]
        mse_scores.append(np.mean(mse_per_horizon))
        r2_scores.append(np.mean(r2_per_horizon))
    
        # Вычисление корреляции: per-horizon (между y_test[:,i] и y_pred[:,i])
        corr_per_horizon = []
        for i in range(y_test.shape[1]):
            # Используем pearsonr для обработки NaN/infs; берем [0] для коэффициента
            corr_i, _ = pearsonr(y_test.to_numpy()[:, i], y_pred[:, i])
            corr_per_horizon.append(corr_i if not np.isnan(corr_i) else 0.0)  # Обработка NaN
        corr_scores.append(np.mean(corr_per_horizon))  # Средняя корреляция по горизонтам
    
        # Опционально: Аггрегированная корреляция (среднее предсказание vs original normalized_target)
        # Это для вашего "corr" в коде: коррелируем mean(y_pred, axis=1) с 'normalized_target'
        y_pred_mean = np.mean(y_pred, axis=1)
        original_target = df_phase.loc[test_data.index, 'normalized_target']  # Или test_data['normalized_target'] если доступно
        corr_mean_pred, _ = pearsonr(original_target, y_pred_mean)
        print(f"Fold corr_mean_pred (mean pred vs normalized_target): {corr_mean_pred}")  # Для логирования, не добавляем в scores
    
        # Ваш закомментированный код для missed_pnl (можно раскомментировать и адаптировать)
        # missed_pnl = []  # % missed from max
        # for batch in test_data['batch'].unique():
        #     mask = test_data['batch'] == batch
        #     max_high = test_data.loc[mask, 'high'].max()
        #     pred = y_pred[mask]  # (n_bars_in_batch, n_targets)
        #     pred_mean_batch = np.mean(pred, axis=1)  # Среднее по горизонтам для сигнала
        #     sell_idx = np.argmin(pred_mean_batch)  # sell at min pred (low confidence)
        #     sell_price = test_data.loc[mask].iloc[sell_idx]['close']
        #     missed = (max_high - sell_price) / (max_high - test_data.loc[mask].iloc[0]['close'])  # % missed
        #     missed_pnl.append(missed)
        # avg_missed = np.mean(missed_pnl)
        # print(f"Fold avg_missed: {avg_missed}")
    
    # Финальные метрики (как в вашем коде)
    avg_mse = float(np.mean(mse_scores))
    avg_r2 = float(np.mean(r2_scores))      # we want it *higher*
    std_r2 = float(np.std(r2_scores))
    corr_mean = float(np.mean(corr_scores))  # Теперь это средняя per-horizon corr
    corr_std = float(np.std(corr_scores))
else:
    for train_idx, test_idx in kf.split(gh_t['batch'].unique()):
        train_batches = gh_t['batch'].unique()[train_idx]
        test_batches = gh_t['batch'].unique()[test_idx]
    
        train_data = gh_t[gh_t['batch'].isin(train_batches)]
        test_data = gh_t[gh_t['batch'].isin(test_batches)]
    
        # Подготовка данных
        X_train, y_train, preprocessing = prepare_data(train_data, 'normalized_target')
        X_test, y_test, _ = prepare_data(test_data, 'normalized_target')
    
        pipeline_reg = Pipeline([
                    ('preprocessing', preprocessing),
                    ('to_dense', FunctionTransformer(to_dense)),  # Преобразование в плотную матрицу
                    ('model', LGBMRegressor(max_depth = 13,
                                            n_estimators = 140, 
                                            random_state=42, 
                                            verbosity=-1))#study_model.trials[best_num_model].params |
                ])
    
        pipeline_reg.fit(X_train, y_train)
        y_pred = pipeline_reg.predict(X_test)
    
        #missed_pnl = []  # % missed from max
        #for batch in test_data['batch'].unique():
        #    mask = test_data['batch'] == batch
        #    max_high = test_data.loc[mask, 'high'].max()
        #    pred = y_pred[mask]  # assume pred ~ normalized_target
        #    sell_idx = np.argmin(pred)  # sell at min pred (low confidence)
        #    sell_price = test_data.loc[mask].iloc[sell_idx]['close']
        #    missed = (max_high - sell_price) / (max_high - test_data.loc[mask].iloc[0]['close'])  # % missed
        #    missed_pnl.append(missed)
        #avg_missed = np.mean(missed_pnl)
        #corr_score = df_phase.loc[gh[gh['batch'].isin(test_batches)].index]['normalized_target'].corr(pd.Series(y_pred, index=X_test.index))
        corr = df_phase.loc[test_data.index, 'normalized_target']\
                               .corr(pd.Series(y_pred, index=X_test.index))
        mse_scores.append(mean_squared_error(y_test, y_pred))
        r2_scores.append(r2_score(y_test, y_pred))
        corr_scores.append(corr)
    avg_mse = float(np.mean(mse_scores))
    avg_r2 = float(np.mean(r2_scores))      # we want it *higher*
    std_r2 = float(np.std(r2_scores))
    corr_mean = float(np.mean(corr_scores))  # Теперь это средняя per-horizon corr
    corr_std = float(np.std(corr_scores))

print(f"Avg MSE: {avg_mse}")
print(f"Avg R2: {avg_r2}")
print(f"Std R2: {std_r2}")
print(f"Mean Corr (per-horizon): {corr_mean}")
print(f"Std Corr: {corr_std}")

from lightgbm import LGBMRanker
from sklearn.metrics import ndcg_score
n_splits = 3
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
TGT_COLS = ['normalized_target']
unique_batches = combined_ml_df['batch'].unique()
columns_for_model = final_cols[ticker] + TGT_COLS + ['batch', 'regime', 'predd', 'predd_var', 'predd_shift_5', 'predd_pct']
gh_t = combined_ml_df[columns_for_model].copy()

ranker_params = {
    'objective':'lambdarank',#lambdarank
    'metric':'ndcg', #ndcg
    'ndcg_eval_at':[3],
    'learning_rate':0.15,
    'num_leaves':128,
    'n_estimators':1000,
    'subsample':0.8,
    'colsample_bytree':0.8,
    'random_state':42
}

K = 15  
global_y_min = gh_t['normalized_target'].min()
global_y_max = gh_t['normalized_target'].max()

mse_list, r2_list, corr_list, ndcg_list = [], [], [], []

def make_batch_bins(df, target_col, batch_col, K):
    """
    Для каждого batch отдельно строим K бинов от min до max и возвращаем Series[0..K-1].
    """
    def bin_series(s):
        if s.max()==s.min():
            return pd.Series(np.zeros(len(s),dtype=int), index=s.index)
        # K равных интервалов между [min,max]
        bins = np.linspace(s.min(), s.max(), K+1)
        return pd.cut(s, bins=bins, labels=False, include_lowest=True)
    return df.groupby(batch_col)[target_col].apply(bin_series)

def make_group(batches):
        vals, idx = np.unique(batches, return_index=True)
        order = np.argsort(idx)
        vals  = vals[order]
        return batches.value_counts().reindex(vals).to_numpy()

# уникальные батчи
all_batches = gh_t['batch'].unique()

for train_idx, test_idx in kf.split(all_batches):
    train_batches = gh_t['batch'].unique()[train_idx]
    test_batches = gh_t['batch'].unique()[test_idx]
    
    train_df = gh_t[gh_t['batch'].isin(train_batches)].reset_index(drop=True)
    val_df   = gh_t[gh_t['batch'].isin(test_batches)] .reset_index(drop=True)
    
    # готовим X,y и препроцессор
    
    X_tr, y_tr, preprocessing = prepare_data(train_df, 'normalized_target')
    X_vl, y_vl, _            = prepare_data(val_df,  'normalized_target')

    K = 15  
    y_tr_bins = make_batch_bins(train_df, 'normalized_target', 'batch', K)
    y_vl_bins = make_batch_bins(val_df,  'normalized_target', 'batch', K)

    
    # группы для LGBMRanker
    # group = число строк в каждом batch (по порядку уникальных batch'ей в train_df)
    grp_tr = make_group(train_df['batch'])
    grp_vl = make_group(val_df ['batch'])
    
    pipeline = Pipeline([
        ('preproc',  preprocessing),
        ('todense',  FunctionTransformer(to_dense, feature_names_out="one-to-one")),
        ('ranker',   LGBMRanker(**ranker_params)),
    ])

    # 5) предварительно трансформируем X_tr и X_vl
    X_tr_t = pipeline.named_steps['todense']\
                      .transform(pipeline.named_steps['preproc']\
                      .fit_transform(X_tr, y_tr_bins))
    X_vl_t = pipeline.named_steps['todense']\
                      .transform(pipeline.named_steps['preproc']\
                      .transform   (X_vl))

    # 6) фитим ranker вручную
    pipeline.named_steps['ranker'].fit(
        X_tr_t, y_tr_bins,
        group      = grp_tr,
        eval_set   = [(X_vl_t, y_vl_bins)],
        eval_group = [grp_vl],
        eval_at    = [3],
        #verbose    = False,
    )
    
    # предсказание
    y_pred_val = pipeline.predict(X_vl)
    
    y_pred_s = pd.Series(y_pred_val, index=val_df.index)

    ndcgs = []
    
    def pred_converter(df):
        indices = y_vl_bins.index.get_level_values(1).values
        values = y_vl_bins.values  # или df['название_столбца'].values
        return pd.Series(values, index=indices)

    
    for b in val_df['batch'].unique():
        mask = (val_df['batch'] == b)
        # true_rank должен быть Series или numpy того же размера
        true_labels = pred_converter(y_vl_bins)[mask].to_numpy()
        preds       = y_pred_s[mask].to_numpy()
        k = min(3, len(preds))
        ndcgs.append(ndcg_score([true_labels], [preds], k=k))
    
    avg_ndcg = np.mean(ndcgs)
    
    # 2) для сравнения: MSE, R2, корреляция
    mse_list.append(mean_squared_error(pred_converter(y_vl_bins), y_pred_val))
    r2_list.append(r2_score(pred_converter(y_vl_bins), y_pred_val))
    corr = pearsonr(pred_converter(y_vl_bins), y_pred_val)[0]
    corr_list.append(corr if not np.isnan(corr) else 0.0)
    ndcg_list.append(avg_ndcg)

# итог
print(f"CV results over {n_splits} folds:")
print(f" avg NDCG@3 = {np.mean(ndcg_list):.4f} ± {np.std(ndcg_list):.4f}")
print(f" avg Corr   = {np.mean(corr_list):.4f} ± {np.std(corr_list):.4f}")
print(f" avg R2     = {np.mean(r2_list):.4f} ± {np.std(r2_list):.4f}")
print(f" avg MSE    = {np.mean(mse_list):.4e} ± {np.std(mse_list):.4e}")

predict = pipeline_reg.predict(combined_ml_df[columns_for_model].drop(TGT_COLS+['batch', 'normalized_target'], axis=1))#.drop('normalized_target', axis=1))
if len(TGT_COLS)!=1:
    predict_series = pd.Series(np.mean(predict, axis=1), index=combined_ml_df.index)
    predict_series = predict_series
else:
    predict_series = pd.Series(predict, index=combined_ml_df.index)
reg_pred= predict_series.reindex(df_phase.index)

predict = pipeline.predict(combined_ml_df[columns_for_model].drop(TGT_COLS+['batch', 'normalized_target'], axis=1))#.drop('normalized_target', axis=1))
if len(TGT_COLS)!=1:
    predict_series = pd.Series(np.mean(predict, axis=1), index=combined_ml_df.index)
    predict_series = predict_series
else:
    predict_series = pd.Series(predict, index=combined_ml_df.index)
    predict_series = 2* ((predict_series +15) / (5 + 15))-1
rank_pred = predict_series.reindex(df_phase.index)
w_reg =  0.940107707282538
df_phase['predicted_p'] = (reg_pred * w_reg) + (rank_pred * (1.0-w_reg))

In [ ]:
print(mean_squared_error(df_phase.loc[combined_ml_df[combined_ml_df['batch'].isin(test_batches)].index]['normalized_target'], df_phase.loc[combined_ml_df[combined_ml_df['batch'].isin(test_batches)].index]['predicted_p']))
pred11s = df_phase['predicted_p'].dropna()
print(mean_squared_error(df_phase[df_phase.index.isin(pred11s.index)]['normalized_target'], pred11s))